In [1]:
%matplotlib qt5
import numpy as np
from numpy import linalg as la
import matplotlib.pylab as plt
from scipy.optimize import fsolve
import math
import statsmodels.api as sm
from scipy.stats import norm
import seaborn as sns
from scipy import stats
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms
from utils import *

## Fixed connectivity probability

### Connectivity

In [2]:
### define the network parameters of the adjacency matrix
nn = [50,100,150,250,750,800,1200]
g, gamma = 6.0, 1/4.0#1.5,1/1.#
NE = nn[-1]
NI = int(gamma*NE)
N  = NE+NI
ALPHAE, ALPHAI = NE/N, NI/N
c = 0.2 ### sparsity, identical for excitatory and inhibitory neuron populations
KE, KI = int(c*NE), int(c*NI) ### fixed out-degree
# J = 1/np.sqrt(N)*0.5  ### TODO: make sure this scalar with David&Stefano's paper
J = 1/np.sqrt(1500)*0.5  ### TODO: make sure this scalar with David&Stefano's paper
print('number of connected E/I neurons:',KE,KI)
print('non-zero J:',J)
ji,je = g*J,J 

### define the network parameters of the diluted Gaussian matrix 
ge, gi = np.sqrt(je**2*c*(1-c)*N), np.sqrt(ji**2*c*(1-c)*N) 
hat_sigmae, hat_sigmai = np.sqrt(c*(1-c)), np.sqrt(c*(1-c))### standard deviation of the adjacency matrix
sigmae,sigmai = np.sqrt(c*(1-c)*J**2*N), np.sqrt(c*(1-c)*(-g*J)**2*N)### with magnitude of the coupling
JE,JI = je*c*NE, ji*c*NI 
lambda0 = JE-JI 
print('lambda0:',lambda0)
print('ge,gi:',ge,gi)
print('JE,JI:',JE,JI)

number of connected E/I neurons: 240 60
non-zero J: 0.012909944487358056
lambda0: -1.5491933384829668
ge,gi: 0.2 1.2000000000000002
JE,JI: 3.0983866769659336 4.6475800154489


In [3]:
ntau   = 10#21#
trials = 30+6#6
tau_series = np.linspace(0,0.225,ntau)# np.linspace(0.25,0.4,ntau)#
eigvchn_series, eigrvec_series, eiglvec_series = np.zeros((trials,ntau,N),dtype=complex), np.zeros((trials,ntau,N,2)), np.zeros((trials,ntau,N,2))
eiglvec0_series, norml0_series = np.zeros((trials,ntau,N,2)), np.zeros((trials,ntau+1,N)) 
### normalization for left and right eigenvectors 
norms_for_lvec, norms_for_rvec = np.zeros((trials,ntau+1,2)), np.zeros((trials,ntau+1,2))
### also have the reconstructed left and right eigenvectors 
eigrvec_series_rec, eiglvec_series_rec = np.zeros((trials,ntau,N,2)), np.zeros((trials,ntau,N,2))
htau = tau_series[1]-tau_series[0]
### mean connectivity
nvec, mvec = np.zeros((N,1)), np.ones((N,1))
nvec[:NE,0], nvec[NE:,0] = N*JE/NE, -N*JI/NI
Jbar = mvec@nvec.T/N 
## TEST THE EIGENVALUES OF THE MEAN MATRIX 
eigvJ0, eigvecJ0 = la.eig(Jbar)
print('eigvJ0:',eigvJ0[0],' theory:',JE-JI)
### mean left and right eigenvectors
leigvec0, reigvec0 = np.zeros((N,N)), np.zeros((N,N))
norm_left = np.zeros(2)
## first eigenvector
leigvec0[:,0], reigvec0[:,0] = nvec[:,0]/(JE-JI)/np.sqrt(N), mvec[:,0]/np.sqrt(N)
norm_left[0]  = la.norm(leigvec0[:,0])
leigvec0[:,0] = leigvec0[:,0]/norm_left[0]
norml0_series[:,0,0] = np.sum(leigvec0[:,0]*reigvec0[:,0])
## second eigenvector
kk = np.sqrt(NE*JI**2+NI*JE**2)
reigvec0[:NE,1], reigvec0[NE:,1] = JI/kk,JE/kk 
leigvec0[:NE,1], leigvec0[NE:,1] = -kk/(JE-JI)/NE,kk/(JE-JI)/NI 
norm_left[1]  = la.norm(leigvec0[:,1])
leigvec0[:,1] = leigvec0[:,1]/norm_left[1]
norml0_series[:,0,1] = np.sum(leigvec0[:,1]*reigvec0[:,1])
outerproduct = np.sum(leigvec0[:,0]*reigvec0[:,1])#*norm_left
### compute the square of the random connectivity 
Z2E = N*J**2*hat_sigmae**2*tau_series*ALPHAE-N*g*J**2*hat_sigmae*hat_sigmai*tau_series*ALPHAI
Z2I = -N*g*J**2*hat_sigmae*hat_sigmai*tau_series*ALPHAE+N*g**2*J**2*hat_sigmai**2*tau_series*ALPHAI
DeltaZ2E, DeltaZ2I = Z2E[1]-Z2E[0],Z2I[1]-Z2I[0] ### for practical use
print('DeltaZ2E,DeltaZ2I:',DeltaZ2E,DeltaZ2I)
DeltaZ2 = np.zeros((N,N))
DeltaZ2[:,:NE], DeltaZ2[:,NE:] = DeltaZ2E, DeltaZ2I

### E population and I population separately
Z2E_E, Z2I_E = N*J**2*hat_sigmae**2*tau_series*ALPHAE, -N*g*J**2*hat_sigmae*hat_sigmai*tau_series*ALPHAI
Z2E_I, Z2I_I = -N*g*J**2*hat_sigmae*hat_sigmai*tau_series*ALPHAE, N*g**2*J**2*hat_sigmai**2*tau_series*ALPHAI
DeltaZ2E_E, DeltaZ2I_E = Z2E_E[1]-Z2E_E[0],Z2I_E[1]-Z2I_E[0] ### for practical use
DeltaZ2E_I, DeltaZ2I_I = Z2E_I[1]-Z2E_I[0],Z2I_I[1]-Z2I_I[0] ### for practical use
DeltaZ2_E, DeltaZ2_I = np.zeros((N,N)),np.zeros((N,N))
DeltaZ2_E[:,:NE], DeltaZ2_E[:,NE:] = DeltaZ2E_E, DeltaZ2I_E

eigvJ0: (-1.5491933384830117+0j)  theory: -1.5491933384829668
DeltaZ2E,DeltaZ2I: -0.0004000000000000003 0.0024000000000000002


In [4]:
import scipy
from functools import partial
def odeIntegral(x,t,J,I=0):
    x = np.squeeze(x)
    x = np.reshape(x,(len(x),1))
    # dxdt = -x+J@np.tanh(x)#+I[0]
    dxdt = -x+J@(x)+I[0]
    return np.squeeze(dxdt)
def odesimulation(t,xinit,Jpt,I):
	return scipy.integrate.odeint(partial(odeIntegral,J=Jpt,I=I),xinit,t)

Simulations

In [5]:
ce, ci = c,c
radius_theo_map = np.zeros(ntau)
for it, tau in enumerate(tau_series):
    ### mean connectivity 
    je, ji = J, g*J
    hat_sigmae, hat_sigmai = np.sqrt(ce*(1-ce)),np.sqrt(ci*(1-ci))
    sigmae, sigmai = np.sqrt(je**2*ce*(1-ce)*N), np.sqrt(ji**2*ci*(1-ci)*N) 
    ### generate tau_div_vec, tau_con_vec, tau_rec_vec, tau_chn_vec
    tau_div_vec, tau_con_vec, tau_rec_vec, tau_chn_vec = np.zeros(2),np.zeros((2,2)),np.zeros((2,2)),np.zeros((2,2))
    ### YS: alltaus do not have E-I polarity
    tau_div_vec = np.array([tau,tau])
    tau_con_vec = np.array([[tau,tau],[tau,tau]])
    # tau_rec_vec = np.array([[0,0],[0,0]])
    tau_rec_vec = np.array([[2*tau,2*tau],[2*tau,2*tau]])
    tau_chn_vec = np.array([[tau,tau],[tau,tau]])
    # gamma = 1.0 ## NI/NE
    sigma = np.sqrt(sigmae**2/(1+gamma)+sigmai**2*gamma/(1+gamma))
    ### compute the radius 
    # compute a single number for multiple populations
    tau_div_num = hat_sigmae**2*tau_div_vec[0]+gamma*g**2*hat_sigmai**2*tau_div_vec[1]
    tau_div_den = hat_sigmae**2+gamma*g**2*hat_sigmai**2
    tau_div = tau_div_num/tau_div_den

    tau_con_num = hat_sigmae**2*tau_con_vec[0,0]+gamma*g**2*hat_sigmai**2*tau_con_vec[1,1]
    tau_con_den = hat_sigmae**2+gamma*g**2*hat_sigmai**2
    tau_con  = tau_con_num/tau_con_den

    tau_rec_num = hat_sigmae**2*tau_rec_vec[0,0] - (1+gamma)*g*hat_sigmae*hat_sigmai*tau_rec_vec[0,1]+gamma*g**2*hat_sigmai**2*tau_rec_vec[1,1]
    tau_rec_den = hat_sigmae**2+gamma*g**2*hat_sigmai**2
    tau_rec = tau_rec_num/tau_rec_den/2.0 

    tau_chn_num = hat_sigmae**2*tau_chn_vec[0,0] - 0.5*(1+gamma)*g*hat_sigmae*hat_sigmai*(tau_chn_vec[0,1]+tau_chn_vec[1,0])+gamma*g**2*hat_sigmai**2*tau_chn_vec[1,1]
    tau_chn_den = hat_sigmae**2+gamma*g**2*hat_sigmai**2
    tau_chn = tau_chn_num/tau_chn_den/2.0

    radius_theo = (1.0-tau_con-tau_div+tau_rec-2*tau_chn)/np.sqrt(1-tau_con-tau_div)*sigma
    
    radius_theo_map[it] = radius_theo
            
### theoretical prediction of the eigenvalues

org_k = J*N*(ALPHAE-g*ALPHAI)/2.0
eigvchn_theo = np.zeros((ntau,2))
eigvchn_theo[:,0] = org_k*(c+np.sqrt(c**2+4*c*(1-c)*tau_series))
eigvchn_theo[:,1] = org_k*(c-np.sqrt(c**2+4*c*(1-c)*tau_series))

In [6]:
eigvchn_theo,radius_theo_map

(array([[-1.54919334, -0.        ],
        [-1.69111181,  0.14191847],
        [-1.81382715,  0.26463382],
        [-1.9235092 ,  0.37431586],
        [-2.02359627,  0.47440293],
        [-2.11623746,  0.56704412],
        [-2.20288235,  0.65368902],
        [-2.28456356,  0.73537022],
        [-2.36204746,  0.81285412],
        [-2.43592144,  0.8867281 ]]),
 array([0.56568542, 0.55136195, 0.53665631, 0.52153619, 0.50596443,
        0.48989795, 0.47328638, 0.45607017, 0.43817805, 0.41952354]))

In [7]:
#### constant and deterministic input signal
Inp   = np.squeeze(np.ones((N,1)))/np.sqrt(N) 
tt = np.linspace(0,100,1000)
#### random and structural input signal 
### simulation using the mean network connectivity (inhibition-dominated)
Jpt   = Jbar.copy()
xinit = np.squeeze(np.random.normal(0, 1E-2, (1, N)))
xtemporal = odesimulation(tt, xinit, Jpt, Inp)
firing_rate0 = np.reshape(xtemporal[-1,:N],(N,1))

### generate i.i.d. s
def randbin(M,N,P):  
    return np.random.choice([0, 1], size=(M,N), p=[P, 1-P])

sparseiid_dyns = np.zeros((trials,N,1))
for ktrial in range(trials):
    ### generate iid sparse connectivity 
    Ecol = randbin(1,NE*N,1-ce)
    Icol = randbin(1,NI*N,1-ci)
    Ecol = np.reshape(Ecol,(N,NE))
    Icol = np.reshape(Icol,(N,NI))
    iidsparse = np.zeros((N,N))
    iidsparse[:,:NE], iidsparse[:,NE:] = Ecol.copy()*J,Icol.copy()*J*(-g)
    xinit = np.squeeze(np.random.normal(0, 1E-2, (1, N)))
    xtemporal = odesimulation(tt, xinit, iidsparse, Inp)
    firing_rate0 = np.reshape(xtemporal[-1,:N],(N,1))
    sparseiid_dyns[ktrial,:,:] = firing_rate0.copy()
    
sparseiid_dyns = np.squeeze(sparseiid_dyns)
mean_sparseiid_dyns = np.zeros((trials,2))
for ktrial in range(trials):
    mean_sparseiid_dyns[ktrial,0],mean_sparseiid_dyns[ktrial,1]= np.mean(sparseiid_dyns[ktrial,:NE]),np.mean(sparseiid_dyns[ktrial,NE:])
trialavg_sparseiid_dyns = np.mean(mean_sparseiid_dyns,axis=0)
trialstd_sparseiid_dyns = np.std(mean_sparseiid_dyns,axis=0)
print(trialavg_sparseiid_dyns)

[0.01041059 0.01025805]


load in the sparse network data

In [ ]:
# #### mean of the elements on the eigenvectors corresponding to the eigenvalues in the bulk
# data_name = "/Users/shaoyuxiu/Library/CloudStorage/Dropbox/DailyWork/Allen_project/preparation4paper_Data/sonets/ConnStats_WD_Adjacency_4April_PRX_60N1500_normlr_whilecode_used_vecrec.npz"
# # np.savez(data_name, **data)
# ### load the stored data, in particular, extracting variable: eigvchn_series 
# data = np.load(data_name,allow_pickle=True)

# ### also loading other variables 
# eigvchn_series_sp_bulk  = data['eigvchn_series']
# eigrvec_series_sp_bulk  = data['eigrvec_series']
# eiglvec_series_sp_bulk  = data['eiglvec_series']
# eiglvec0_series_sp_bulk = data['eiglvec0_series']#'eiglvec0norm_series']#eiglvec0_series']
# params = data['params']
# firing_rateeq = data['firing_rateeq']

# ### numerical
# mean_reigvec_num_series_sp_bulk  = np.zeros((trials,ntau,2,2))
# mean_leigvec_num_series_sp_bulk  = np.zeros((trials,ntau,2,2))
# mean_leig0vec_num_series_sp_bulk = np.zeros((trials,ntau,2,2))

# thl = 1.5
# ths = 1.5

# ### for the numerical 
# for ktrial in range(trials):
#     for it in range(ntau):
#         if it<2:
#             threshold = ths
#         else:
#             threshold = thl
#         for ir in range(2,N):
#             mean_reigvec_num_series_sp_bulk[ktrial,it,ir,0] = np.mean(eigrvec_series_sp[ktrial,it,:NE,ir])
#             mean_reigvec_num_series_sp_bulk[ktrial,it,ir,1] = np.mean(eigrvec_series_sp[ktrial,it,NE:,ir])
#             mean_leigvec_num_series_sp_bulk[ktrial,it,ir,0] = np.mean(eiglvec_series_sp[ktrial,it,:NE,ir])
#             mean_leigvec_num_series_sp_bulk[ktrial,it,ir,1] = np.mean(eiglvec_series_sp[ktrial,it,NE:,ir])
#             mean_leig0vec_num_series_sp_bulk[ktrial,it,ir,0] = np.mean(eiglvec0_series_sp[ktrial,it,:NE,ir])
#             mean_leig0vec_num_series_sp_bulk[ktrial,it,ir,1] = np.mean(eiglvec0_series_sp[ktrial,it,NE:,ir])
            
# # ### select the middle 30 values 
# # kktrial = np.arange(trials)
# # cuts = 6#3
# # for it in range(ntau):
# #     for ir in range(2,N):        
# #         idxsort = np.argsort(mean_reigvec_num_series_sp_bulk[:,it,ir,0].real)
# #         idxnan = np.append(idxsort[:cuts],idxsort[-cuts:]) 
# #         mean_reigvec_num_series_sp_bulk[idxnan,it,ir,0] = np.nan 
# #         idxnan = np.where(np.abs(mean_reigvec_num_series_sp_bulk[:,it,ir,0].real)>threshold)[0]
# #         mean_reigvec_num_series_sp_bulk[idxnan,it,ir,0] = np.nan
        
# #         idxsort = np.argsort(mean_reigvec_num_series_sp_bulk[:,it,ir,1].real)
# #         idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
# #         mean_reigvec_num_series_sp_bulk[idxnan,it,ir,1] = np.nan
# #         idxnan = np.where(np.abs(mean_reigvec_num_series_sp_bulk[:,it,ir,1].real)>threshold)[0]
# #         mean_reigvec_num_series_sp_bulk[idxnan,it,ir,1] = np.nan
        
# #         idxsort = np.argsort(mean_leigvec_num_series_sp_bulk[:,it,ir,0].real)
# #         idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
# #         mean_leigvec_num_series_sp_bulk[idxnan,it,ir,0] = np.nan
# #         idxnan = np.where(np.abs(mean_leigvec_num_series_sp_bulk[:,it,ir,0].real)>threshold)[0]
# #         mean_leigvec_num_series_sp_bulk[idxnan,it,ir,0] = np.nan
        
# #         idxsort = np.argsort(mean_leigvec_num_series_sp_bulk[:,it,ir,1].real)
# #         idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
# #         mean_leigvec_num_series_sp_bulk[idxnan,it,ir,1] = np.nan
# #         idxnan = np.where(np.abs(mean_leigvec_num_series_sp_bulk[:,it,ir,1].real)>threshold)[0]
# #         mean_leigvec_num_series_sp_bulk[idxnan,it,ir,1] = np.nan
        
# #         idxsort = np.argsort(mean_leig0vec_num_series_sp_bulk[:,it,ir,0].real)
# #         idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
# #         mean_leig0vec_num_series_sp_bulk[idxnan,it,ir,0] = np.nan
# #         idxnan = np.where(np.abs(mean_leig0vec_num_series_sp_bulk[:,it,ir,0].real)>threshold)[0]
# #         mean_leig0vec_num_series_sp_bulk[idxnan,it,ir,0] = np.nan
        
# #         idxsort = np.argsort(mean_leig0vec_num_series_sp_bulk[:,it,ir,1].real)
# #         idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
# #         mean_leig0vec_num_series_sp_bulk[idxnan,it,ir,1] = np.nan
# #         idxnan = np.where(np.abs(mean_leig0vec_num_series_sp_bulk[:,it,ir,1].real)>threshold)[0]
# #         mean_leig0vec_num_series_sp_bulk[idxnan,it,ir,1] = np.nan


In [9]:
# data_name = "/Users/shaoyuxiu/Library/CloudStorage/Dropbox/DailyWork/Allen_project/preparation4paper_Data/sonets/ConnStats_WD_Adjacency_4April_PRX_60N1500_normlr_whilecode_used_vecrec.npz"
data_name = "E:/Dropbox/DailyWork/Allen_project/preparation4paper_Data/sonets/ConnStats_WD_Adjacency_4April_PRX_60N1500_normlr_whilecode_used_vecrec.npz"
# data_name = "/Users/shaoyuxiu/Library/CloudStorage/Dropbox/DailyWork/Allen_project/preparation4paper_Data/Numerics/Biological_sonet_transpose_lvec_PRX_60N1500_normlr_compare.npz"
### load the stored data, in particular, extracting variable: eigvchn_series 
data = np.load(data_name,allow_pickle=True)

### also loading other variables 
eigvchn_series_sp  = data['eigvchn_series']
eigrvec_series_sp  = data['eigrvec_series']
eiglvec_series_sp  = data['eiglvec_series']
eiglvec0_series_sp = data['eiglvec0_series']#'eiglvec0norm_series']#eiglvec0_series']
params = data['params']
firing_rateeq_sp = data['firing_rateeq']

### numerical
mean_reigvec_num_series_sp  = np.zeros((trials,ntau,2,2))
mean_leigvec_num_series_sp  = np.zeros((trials,ntau,2,2))
mean_leig0vec_num_series_sp = np.zeros((trials,ntau,2,2))

thl = 1.5
ths = 1.5
threshold = thl
### for the numerical 
for ktrial in range(trials):
    for it in range(ntau):
        for ir in range(2):
            mean_reigvec_num_series_sp[ktrial,it,ir,0] = np.mean(eigrvec_series_sp[ktrial,it,:NE,ir])
            mean_reigvec_num_series_sp[ktrial,it,ir,1] = np.mean(eigrvec_series_sp[ktrial,it,NE:,ir])
            mean_leigvec_num_series_sp[ktrial,it,ir,0] = np.mean(eiglvec_series_sp[ktrial,it,:NE,ir])*eigvchn_series_sp[ktrial,it,ir].real
            # print('....',mean_leigvec_num_series_sp[ktrial,it,ir,0])
            mean_leigvec_num_series_sp[ktrial,it,ir,1] = np.mean(eiglvec_series_sp[ktrial,it,NE:,ir])*eigvchn_series_sp[ktrial,it,ir].real
            mean_leig0vec_num_series_sp[ktrial,it,ir,0] = np.mean(eiglvec0_series_sp[ktrial,it,:NE,ir])
            mean_leig0vec_num_series_sp[ktrial,it,ir,1] = np.mean(eiglvec0_series_sp[ktrial,it,NE:,ir])
            
### select the middle 30 values 
kktrial = np.arange(trials)
cuts = 3#6#
for it in range(ntau):
    for ir in range(2):        
        idxsort = np.argsort(mean_reigvec_num_series_sp[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:]) 
        mean_reigvec_num_series_sp[idxnan,it,ir,0] = np.nan 
        idxnan = np.where(np.abs(mean_reigvec_num_series_sp[:,it,ir,0].real)>threshold)[0]
        mean_reigvec_num_series_sp[idxnan,it,ir,0] = np.nan
        
        idxsort = np.argsort(mean_reigvec_num_series_sp[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_reigvec_num_series_sp[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_reigvec_num_series_sp[:,it,ir,1].real)>threshold)[0]
        mean_reigvec_num_series_sp[idxnan,it,ir,1] = np.nan
        
        idxsort = np.argsort(mean_leigvec_num_series_sp[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leigvec_num_series_sp[idxnan,it,ir,0] = np.nan
        # idxnan = np.where(np.abs(mean_leigvec_num_series_sp[:,it,ir,0].real)>threshold)[0]
        # mean_leigvec_num_series_sp[idxnan,it,ir,0] = np.nan
        
        idxsort = np.argsort(mean_leigvec_num_series_sp[:,it,ir,1].real)
        idxnan  = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leigvec_num_series_sp[idxnan,it,ir,1] = np.nan
        # idxnan = np.where(np.abs(mean_leigvec_num_series_sp[:,it,ir,1].real)>threshold)[0]
        # mean_leigvec_num_series_sp[idxnan,it,ir,1] = np.nan
        
        # idxsort = np.argsort(mean_leig0vec_num_series_sp[:,it,ir,0].real)
        # idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        # mean_leig0vec_num_series_sp[idxnan,it,ir,0] = np.nan
        # idxnan = np.where(np.abs(mean_leig0vec_num_series_sp[:,it,ir,0].real)>threshold)[0]
        # mean_leig0vec_num_series_sp[idxnan,it,ir,0] = np.nan
        
        # idxsort = np.argsort(mean_leig0vec_num_series_sp[:,it,ir,1].real)
        # idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        # mean_leig0vec_num_series_sp[idxnan,it,ir,1] = np.nan
        # idxnan = np.where(np.abs(mean_leig0vec_num_series_sp[:,it,ir,1].real)>threshold)[0]
        # mean_leig0vec_num_series_sp[idxnan,it,ir,1] = np.nan
        


In [10]:
### compute the numerical radius
radius_num_map = np.zeros((trials,ntau))
for ktrial in range(trials):
    for it in range(ntau): 
        eigvchn_sp = eigvchn_series_sp[ktrial,it,:].copy()

        eigvchn_real_sp = np.real(eigvchn_sp)
        eigvchn_real_sp = np.sort(eigvchn_real_sp)
        eigvchn_real_sp = eigvchn_real_sp[::-1]

        radius = (eigvchn_real_sp[2])
        radius_num_map[ktrial,it] = radius

In [11]:
#### print trial by trial the first two eigenvallues
for ktrial in range(trials):
    eigvchn_sp = eigvchn_series_sp[ktrial,:,:].copy()
    eigvchn_real_sp = (eigvchn_sp)
# ### delete the 17th trial
# eigvchn_series = np.delete(eigvchn_series,17,axis=0) #for ConnStats_Adjacency_12OctDense_.npz
### sorting the 3 maximum eigvchn_series[:,-1,0].real and 3 minimum
eigvchn_real_sp = eigvchn_series_sp[:,-1,0].real.copy()
### sorting eigvchn_real 
idx = np.argsort(eigvchn_real_sp)
idx_eff = idx[3:-3]
print('effective trials:',idx_eff,len(idx_eff))

effective trials: [ 8 20 21 29 24  9 15 17  4  0 30 35  3 31 14  5 13 25 28 18 10 33 34 12
 26 16  1 23  7 22] 30


Fig 6 inset

In [38]:
### plot the numerical and theoretical eigenvalues outliers
fig,ax = plt.subplots(figsize=(5,3))
### label is r_{bulk}^{num}
ax.plot(tau_series,radius_theo_map,'gray',label=r'$r_{bulk}^{theo}$')
ax.plot(tau_series,-radius_theo_map,'gray')
ax.fill_between(tau_series,np.mean(radius_num_map,axis=0),-np.mean(radius_num_map,axis=0),color='gray',alpha=0.15,label=r'$r_{bulk}^{num}$')
ax.set_xlabel(r'$\tau$')
ax.set_ylabel('Radius')
ax.legend()
plt.show()

alphass= 1

ax.plot(tau_series,eigvchn_theo[:,0],'tab:red',label='theoretical',alpha=alphass)
ax.plot(tau_series,eigvchn_theo[:,1],'tab:green',alpha=alphass)

### plot error bar  
ax.errorbar(tau_series,np.mean(eigvchn_series_sp[idx_eff,:,0].real,axis=0),yerr=np.std(eigvchn_series_sp[idx_eff,:,0].real,axis=0),fmt='x',color='tab:red',ecolor='tab:red',alpha=alphass,ls='',elinewidth=1.5)

ax.errorbar(tau_series,np.mean(eigvchn_series_sp[idx_eff,:,1].real,axis=0),yerr=np.std(eigvchn_series_sp[idx_eff,:,1].real,axis=0),fmt='x',color='tab:green',ecolor='tab:green',alpha=alphass,ls='',elinewidth=1.5)
    
ax.set_xlabel(r'$\tau$')
ax.set_ylabel('Outliers')
### set x and y lims as well as ticks
dtau = tau_series[1]-tau_series[0] 
ax.set_xlim([tau_series[0]-dtau/8,tau_series[-1]+dtau/8])
ax.set_xticks([tau_series[0],tau_series[-1]])
ax.set_ylim([-3,1.5])
ax.set_yticks([-3,0,1])
ax.plot([0,tau_series[-1]],[1,1],color='gray',linestyle='--',linewidth=1.5)
ax.legend()
### move the x and y axis to the center
ax.spines['bottom'].set_position(('data',0))
ax.spines['left'].set_position(('data',tau_series[0]))
### set other x-axis invisible
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')
plt.show()

In [ ]:
### only the first pair of connectivity vectors
### if you want to re-run the simulation
# trials = 6
eiglvec0norm_series = np.zeros((trials,ntau,N,2))
leig0mean_series    = np.zeros((trials,ntau,N,2))
leig0pre_series     = np.zeros((trials,ntau,N,2))
### simulation using the low-rank framework
firing_rateeq = np.zeros((trials,ntau,N))
firing_rateeq_bulk = np.zeros((trials,ntau,N))
z_pre = np.zeros((N,N))
### recording dynamics
lowrank_eq, lowrank_eq_num = np.zeros((trials,ntau,2)), np.zeros((trials,ntau,N))
ovs_inplr, ovs_inplr_num = np.zeros((trials,ntau,2)), np.zeros((trials,ntau,2))
ovs_inplr_div, ovs_inplr_div_num = np.zeros((trials,ntau,2)), np.zeros((trials,ntau,2))
contributions_lr, contributions_lr_num = np.zeros((trials,ntau,2,2)), np.zeros((trials,ntau,2,2))### rank, population

switch,factor = 4,1

norm_4rvec_series, norm_4lvec_series = np.zeros((trials,ntau,2)),np.zeros((trials,ntau,2))
norm_4lvec_series_ = np.zeros((trials,ntau,2))

intg_ml0 = np.zeros((ntau,2,N))
intg_mr  = np.zeros((ntau,2,N))
intg_ml  = np.zeros((ntau,2,N))

intg_norml0 = np.zeros((ntau,2,2))### rank pop
intg_normr  = np.zeros((ntau,2,2))

### trials random networks and vectors 
xrs   = np.zeros((trials,N,N))
xrecs = np.zeros((trials,N,N))
chnetas = np.zeros((trials,N,6))

flag_run = True
for ktrial in range(trials):
    while (flag_run):
        xr      = iidGaussian([0,1/np.sqrt(N)],[N,N])
        chneta  = iidGaussian([0,1/np.sqrt(N)],[N,6])
        xrec    = iidGaussian([0,1/np.sqrt(N)],[N,N])
        ### zscore
        xr   = stats.zscore(xr.flatten())
        xr   = xr*1/np.sqrt(N)
        xr   = np.reshape(xr,(N,N))
        ### zscore
        xrec = stats.zscore(xrec.flatten())
        xrec = xrec*1/np.sqrt(N)
        xrec = np.reshape(xrec,(N,N))
        # ### zscore
        # nsample = 20
        chneta  = iidGaussian([0,1/np.sqrt(N)],[N,6])
        chneta[:,0] = stats.zscore(chneta[:,0])
        chneta[:,0] *=(1/np.sqrt(N))
        
        tau = tau_series[-1]
        a    = np.sqrt(tau)
        zrow = a*np.repeat(np.reshape(chneta[:,0],(1,-1)),N,axis=0)
        zcol = a*np.repeat(np.reshape(chneta[:,0],(-1,1)),N,axis=1)
        gammarec = a*xrec-a*xrec.copy().T
        ### adjacency matrix
        zr   = zrow.copy()+zcol.copy()+np.sqrt(1-2*tau)*xr### without considering the reciprocal term
        ### E-I matrix 
        zr[:,:NE],zr[:,NE:] = zr[:,:NE]*ge,zr[:,NE:]*(-gi)        
        ### generate J connectivity matrix
        Jchn = Jbar.copy()+zr.copy()           
        eigvchn, eigrvec = la.eig(Jchn)
        FIRSTV = np.abs(eigvchn[0]-eigvchn_theo[-1,0])/np.abs(eigvchn_theo[-1,0])
        SECONDV = np.abs(eigvchn[1]-eigvchn_theo[-1,1])/np.abs(eigvchn_theo[-1,1])
        if FIRSTV<0.1 and SECONDV<0.1 and eigvchn[0].imag==0 and eigvchn[1].imag==0:
            flag_run = False
            print('eigvchn:',eigvchn[:2],'theory:',eigvchn_theo[-1,:])
    ### ---------------------
    print('Go run...........................')
    
    xrs[ktrial,:,:]      = xr.copy()
    chnetas[ktrial,:,:]  = chneta.copy()
    xrecs[ktrial,:,:]    = xrec.copy()
    flag_run = True

### if you want to re-run the simulations for dynamics, set rerun_dyns to be True 
htau = tau_series[-1]-tau_series[-2]
for it, tau in enumerate(tau_series):
    a    = np.sqrt(tau)
    z_forder_square_me = np.zeros((N,N))
    z_sorder_square    = np.zeros((N,N))
    z_forder_square_mi = np.zeros((N,N))
    z_forder_square_ne = np.zeros((N,N))
    z_forder_square_ni = np.zeros((N,N))
    norm_lvec = np.zeros(2)
    for ktrial in range(trials):
        xr      = xrs[ktrial,:,:]
        chneta  = chnetas[ktrial,:,:]
        xrec    = xrecs[ktrial,:,:]
        
        zrow = a*np.repeat(np.reshape(chneta[:,0],(1,-1)),N,axis=0)
        zcol = a*np.repeat(np.reshape(chneta[:,0],(-1,1)),N,axis=1)
        gammarec = a*xrec-a*xrec.copy().T
        # zr   = zrow.copy()+zcol.copy()+np.sqrt(1-4*tau)*xr+gammarec
        zr   = zrow.copy()+zcol.copy()+np.sqrt(1-2*tau)*xr
        zr[:,:NE],zr[:,NE:] = zr[:,:NE]*ge,zr[:,NE:]*(-gi)
        if it==0:
            zr_pre = np.zeros_like(zr)
        else:
            a_pre = np.sqrt(tau_series[it-1])
            zrow_pre = a_pre*np.repeat(np.reshape(chneta[:,0],(1,-1)),N,axis=0)
            zcol_pre = a_pre*np.repeat(np.reshape(chneta[:,0],(-1,1)),N,axis=1)
            gammarec_pre = a_pre*xrec-a_pre*xrec.copy().T
            zr_pre   = zrow_pre.copy()+zcol_pre.copy()+np.sqrt(1-2*tau_series[it-1])*xr
            zr_pre[:,:NE],zr_pre[:,NE:] = zr_pre[:,:NE]*ge,zr_pre[:,NE:]*(-gi)

        ### generate J connectivity matrix
        Jchn = Jbar.copy()+zr.copy()
        ### full rank simulation
        xinit = np.squeeze(np.random.normal(0, 1E-2, (1, N)))
        xc_temporal = odesimulation(tt, xinit, Jchn, Inp)
        firing_rateeq[ktrial,it,:] = xc_temporal[-1,:].copy()
        
        eigvchn, eigrvec = la.eig(Jchn)
        eigvchn_,eiglvec = la.eig(Jchn.copy().T)
        assert(np.sum(np.abs(eigvchn[0]-eigvchn_[0]))<1e-9)## check the consistency of left and right eigenvectors
        ### normalization
        reig  = np.squeeze(eigrvec[:,:].copy())            
        ### inverse left eigenvector
        inveigrvec       = la.inv(eigrvec)
        leig = np.squeeze(inveigrvec[:,:].copy()) # inverse
        leig = leig.copy().T
        ### leig0 is normalized 
        norm_lvec += la.norm(leig[:,:2],axis=0)
        leig0 = leig/la.norm(leig,axis=0)

        if np.mean(reig[:NE,0])<0:
            reig[:,0]*=-1
            leig[:,0]*=-1
            leig0[:,0]*=-1
        if np.mean(reig[:NE,1])<0: ### the second rank-1 component is negative
            reig[:,1]*=-1
            leig[:,1]*=-1
            leig0[:,1]*=-1
        
        eigvchn_series[ktrial,it,:]    = eigvchn.copy()#eigvw_norm.copy()#
        eigrvec_series[ktrial,it,:,:]  = reig[:,:2].copy()#eigvecw_norm.copy()#
        eiglvec_series[ktrial,it,:,:]  = leig[:,:2].copy()#eigvect_norm.copy()#
        eiglvec0_series[ktrial,it,:,:] = leig0[:,:2].copy()#eigvect_norm.copy()#
    
    norm_lvec = norm_lvec/trials ### normalization is used for computing the left connectivity vectors
    
    ### analytical solution for the small perturbation 
    if it == 0:
        htau = 0
        z_forder_square_ne, z_forder_square_ni = np.zeros((N,N)),np.zeros((N,N))
        z_forder_square_me, z_forder_square_mi = np.zeros((N,N)),np.zeros((N,N))
        z_sorder_square = np.zeros((N,N))
        
        z_forder_square_ne = np.eye(N)*ge**2/N*NE
        z_forder_square_ni = np.eye(N)*gi**2/N*NI 
        
        z_forder_square_me = np.eye(N)
        z_forder_square_mi = np.eye(N)
        z_forder_square_me[:NE,:NE] *= ge**2*NE/N 
        z_forder_square_me[NE:,NE:] *= gi**2*NE/N 
        z_forder_square_mi[:NE,:NE] *= ge**2*NI/N 
        z_forder_square_mi[NE:,NE:] *= gi**2*NI/N
        
        z_sorder_square = np.zeros((N,N))
    else:
        htau = (np.sqrt(tau)-np.sqrt(tau_series[it-1]))**2
        # htau = (np.sqrt(tau)-np.sqrt(tau_series[it-1]))**2
        z_forder_square_ne, z_forder_square_ni = np.zeros((N,N)),np.zeros((N,N))
        z_forder_square_me, z_forder_square_mi = np.zeros((N,N)),np.zeros((N,N))
        z_sorder_square = np.zeros((N,N))
        
        #### variance of the vector n 
        z_forder_square_ne[:NE,:NE],z_forder_square_ne[:NE,NE:] = htau*ge**2*NE/N,htau*ge**2*NE/N 
        z_forder_square_ne[NE:,:NE],z_forder_square_ne[NE:,NE:] = htau*ge**2*NE/N,htau*ge**2*NE/N 
        
        z_forder_square_ni[:NE,:NE],z_forder_square_ni[:NE,NE:] = htau*gi**2*NI/N, htau*gi**2*NI/N 
        z_forder_square_ni[NE:,:NE],z_forder_square_ni[NE:,NE:] = htau*gi**2*NI/N, htau*gi**2*NI/N 
        
        #### variance of the vector m
        z_forder_square_me[:NE,:NE],z_forder_square_me[:NE,NE:] = htau*ge**2*NE/N, -htau*ge*gi*NE/N
        z_forder_square_me[NE:,:NE],z_forder_square_me[NE:,NE:] = -htau*ge*gi*NE/N,htau*gi**2*NE/N
        
        z_forder_square_mi[:NE,:NE],z_forder_square_mi[:NE,NE:] = htau*ge**2*NI/N, -htau*ge*gi*NI/N
        z_forder_square_mi[NE:,:NE],z_forder_square_mi[NE:,NE:] = -htau*ge*gi*NI/N, htau*gi**2*NI/N
        
        #### covariance between m and n 
        z_sorder_square[:,:NE] = htau*ge**2*NE/N-htau*ge*gi*NI/N
        z_sorder_square[:,NE:] = -htau*ge*gi*NE/N+htau*gi**2*NI/N 
    
    lvec_m, rvec_m   = np.zeros((N,2)),np.zeros((N,2))
    lvec_mean, rvec_mean   = np.zeros((N,2)),np.zeros((N,2))
    noise_r, noise_l = np.zeros((N,2)),np.zeros((N,2))
    norm_r, norm_l   = np.zeros((2,2)),np.zeros((2,2))       
    
    if it==0: ### initial
        lvec_mean[:,0], rvec_mean[:,0] = -np.squeeze(nvec/la.norm(nvec)),np.squeeze(mvec/la.norm(mvec))
        lvec_norm, rvec_norm = np.zeros((2,2)), np.zeros((2,2))
        eigv_pre = (JE-JI)*np.ones(2)            
        norm_for_rvec, norm_for_lvec = np.zeros((2,2)),np.zeros((2,2)) ### rank pop
    elif it<=switch+1:
        lvec_mean[:,0], rvec_mean[:,0] = np.squeeze(intg_ml0[it-1,0,:]),np.squeeze(intg_mr[it-1,0,:]) ## N, rank
        lvec_norm[0,:], rvec_norm[0,:] = np.squeeze(intg_norml0[it-1,0,:]),np.squeeze(intg_normr[it-1,0,:]) ## rank, pop
        eigv_pre = eigvchn_theo[it-1,:]#np.mean(eigvchn_series[:,it-1,:2].real,axis=0)#eigvchn_theo[it,:]#
        for ktrial in range(trials):
            ir = 1
            lvec_m[:NE,ir],rvec_m[:NE,ir] = np.mean(eiglvec0_series[ktrial,it,:NE,ir]),np.mean(eigrvec_series[ktrial,it,:NE,ir])
            # print('lvec mean???:',lvec_m[1,ir])
            lvec_m[NE:,ir],rvec_m[NE:,ir] = np.mean(eiglvec0_series[ktrial,it,NE:,ir]),np.mean(eigrvec_series[ktrial,it,NE:,ir])
            noise_r[:,ir],noise_l[:,ir] = eigrvec_series[ktrial,it,:,ir]-rvec_m[:,ir],eiglvec0_series[ktrial,it,:,ir]-lvec_m[:,ir]
            ipp = 0
            norm_r[ir,ipp],norm_l[ir,ipp] = la.norm(noise_r[:NE,ir]),la.norm(noise_l[:NE,ir])
            ipp = 1
            norm_r[ir,ipp],norm_l[ir,ipp] = la.norm(noise_r[NE:,ir]),la.norm(noise_l[NE:,ir])
                
            lvec_mean[:,ir] += lvec_m[:,ir].copy()
            rvec_mean[:,ir] += rvec_m[:,ir].copy()
            lvec_norm[ir,:] += norm_l[ir,:].copy()
            rvec_norm[ir,:] += norm_r[ir,:].copy()
        lvec_mean[:,1],rvec_mean[:,1] = lvec_mean[:,1]/trials,rvec_mean[:,1]/trials
        lvec_norm[1,:],rvec_norm[1,:] = lvec_norm[1,:]/trials,rvec_norm[1,:]/trials 
            
        for ir in range(1,2):
            intg_ml[it,ir,:NE] = np.mean(np.mean(eiglvec_series[:,it,:NE,ir],axis=1))*eigvchn_theo[it,ir]
            intg_ml[it,ir,NE:] = np.mean(np.mean(eiglvec_series[:,it,NE:,ir],axis=1))*eigvchn_theo[it,ir]
    
        intg_mr[it,1,:]  = np.squeeze(rvec_mean[:,1])
        intg_ml0[it,1,:]  = np.squeeze(lvec_mean[:,1])
        
        intg_norml0[it,1,:] = lvec_norm[1,:]
        intg_normr[it,1,:]  = rvec_norm[1,:]
        eigv_pre = eigvchn_theo[it-1,:]#np.mean(eigvchn_series[:,it-1,:2].real,axis=0)
        for itt in range(it):
            if itt == 0:
                eigv_pre = (JE-JI)**2*np.ones(2)
                eigv_pre[1] = 0
            else:
                eigv_pre = eigv_pre+eigvchn_theo[itt,:2].real**2
        eigv_pre = np.sqrt(eigv_pre/it)
    else:
        lvec_mean, rvec_mean = intg_ml0[it-1,:,:],intg_mr[it-1,:,:]
        lvec_mean, rvec_mean = lvec_mean.T, rvec_mean.T
        lvec_norm, rvec_norm = intg_norml0[it-1,:,:],intg_normr[it-1,:,:] # rank pop
        # eigv_pre = eigvchn_theo[it-1,:]#np.mean(eigvchn_series[:,it-1,:2].real,axis=0)#
        for itt in range(it):
            if itt == 0:
                eigv_pre = (JE-JI)**2*np.ones(2)
                eigv_pre[1] = 0
            else:
                eigv_pre = eigv_pre+eigvchn_theo[itt,:2].real**2
        eigv_pre = np.sqrt(eigv_pre/it)
    
    norm_for_rvec, norm_for_lvec   = np.zeros((2,2)),np.zeros((2,2)) ### rank pop
    sigma_for_rvec, sigma_for_lvec = np.zeros((2,2)),np.zeros((2,2))
    if it<=switch+1:
        ranks_use = 1
    else:
        ranks_use = 2
    for i in range(ranks_use):
        rnorm_tmp,lnorm_tmp = np.zeros(2),np.zeros(2)
        rnorm_tmp_rescale,lnorm_tmp_rescale = np.zeros(2),np.zeros(2)
        ### norm_for_rvec and norm_for_lvec 
        ### E pop
        norm_for_rvec[i,0] = la.norm(np.reshape(rvec_mean[:NE,i],(NE,1))+np.reshape((z_sorder_square[:NE,:]@np.reshape(rvec_mean[:,i],(-1,1)))/eigv_pre[i]**2,(NE,1)))**2 
        norm_for_rvec[i,0] += rvec_norm[i,0]**2 
        norm_for_rvec[i,0] += np.squeeze(np.reshape(rvec_mean[:,i],(1,-1))@z_forder_square_me@np.reshape(rvec_mean[:,i],(-1,1)))/eigv_pre[i]**2 
        norm_for_rvec[i,0] = np.sqrt(norm_for_rvec[i,0])
        ### I pop
        norm_for_rvec[i,1] = la.norm(np.reshape(rvec_mean[NE:,i],(NI,1))+np.reshape((z_sorder_square[NE:,:]@np.reshape(rvec_mean[:,i],(-1,1)))/eigv_pre[i]**2,(NI,1)))**2 
        norm_for_rvec[i,1] += rvec_norm[i,1]**2 
        norm_for_rvec[i,1] += np.squeeze(np.reshape(rvec_mean[:,i],(1,-1))@z_forder_square_mi@np.reshape(rvec_mean[:,i],(-1,1)))/eigv_pre[i]**2 
        norm_for_rvec[i,1] = np.sqrt(norm_for_rvec[i,1])
        ### normx = sigmax *np.sqrt(N) --> derive sigma x 
        sigma_for_rvec[i,0] = norm_for_rvec[i,0]/np.sqrt(NE)
        sigma_for_rvec[i,1] = norm_for_rvec[i,1]/np.sqrt(NI)
        
        ### compute mean 
        rmean_tmp = np.reshape(rvec_mean[:,i].copy(),(-1,1)) + np.reshape((z_sorder_square@np.reshape(rvec_mean[:,i],(-1,1)))/eigv_pre[i]**2,(N,1)) ### mean-shifting 
        rmean_tmp_rescale = rmean_tmp.copy()/np.sqrt(norm_for_rvec[i,0]**2+norm_for_rvec[i,1]**2)  
        rnorm_tmp[0] = np.sqrt(1*rvec_norm[i,0]**2 + np.squeeze(np.reshape(rvec_mean[:,i],(1,-1))@z_forder_square_me@np.reshape(rvec_mean[:,i],(-1,1)))/eigv_pre[i]**2)
        rnorm_tmp[1] = np.sqrt(1*rvec_norm[i,1]**2 + np.squeeze(np.reshape(rvec_mean[:,i],(1,-1))@z_forder_square_mi@np.reshape(rvec_mean[:,i],(-1,1)))/eigv_pre[i]**2)
        rnorm_tmp_rescale = rnorm_tmp/np.sqrt(norm_for_rvec[i,0]**2+norm_for_rvec[i,1]**2)  
        
        ### for E pop
        norm_for_lvec[i,0] = la.norm(np.reshape(lvec_mean[:NE,i],(NE,1))+np.reshape(np.reshape(lvec_mean[:,i],(1,-1))@z_sorder_square[:,:NE]/eigv_pre[i]**2,(NE,1)))**2 
        norm_for_lvec[i,0] += lvec_norm[i,0]**2 
        norm_for_lvec[i,0] += np.squeeze(np.reshape(lvec_mean[:,i],(1,-1))@z_forder_square_ne@np.reshape(lvec_mean[:,i],(-1,1)))/eigv_pre[i]**2
        norm_for_lvec[i,0]  = np.sqrt(norm_for_lvec[i,0])
        ### for I pop
        norm_for_lvec[i,1] = la.norm(np.reshape(lvec_mean[NE:,i],(NI,1))+np.reshape(np.reshape(lvec_mean[:,i],(1,-1))@z_sorder_square[:,NE:]/eigv_pre[i]**2,(NI,1)))**2 
        norm_for_lvec[i,1] += lvec_norm[i,1]**2 
        norm_for_lvec[i,1] += 1*np.squeeze(np.reshape(lvec_mean[:,i],(1,-1))@z_forder_square_ni@np.reshape(lvec_mean[:,i],(-1,1)))/eigv_pre[i]**2
            
        norm_for_lvec[i,1]  = np.sqrt(norm_for_lvec[i,1])
        
        ### compute mean 
        lmean_tmp = np.reshape(lvec_mean[:,i].copy(),(-1,1)) + np.reshape(np.reshape(lvec_mean[:,i],(1,-1))@z_sorder_square/eigv_pre[i]**2,(N,1))
        lmean_tmp_rescale = lmean_tmp.copy()/np.sqrt(norm_for_lvec[i,0]**2+norm_for_lvec[i,1]**2)
        lnorm_tmp[0] = np.sqrt(1*lvec_norm[i,0]**2 + np.squeeze(np.reshape(lvec_mean[:,i],(1,-1))@z_forder_square_ne@np.reshape(lvec_mean[:,i],(-1,1)))/eigv_pre[i]**2)
        lnorm_tmp[1] = np.sqrt(1*lvec_norm[i,1]**2 + np.squeeze(np.reshape(lvec_mean[:,i],(1,-1))@z_forder_square_ni@np.reshape(lvec_mean[:,i],(-1,1)))/eigv_pre[i]**2)
        lnorm_tmp_rescale = lnorm_tmp/np.sqrt(norm_for_lvec[i,0]**2+norm_for_lvec[i,1]**2)
        # norm_f = np.squeeze(np.reshape(lmean_tmp,(1,-1))@np.reshape(rmean_tmp,(-1,1)))+np.squeeze(np.reshape(lvec_mean[:,i],(1,-1))@z_sorder_square@np.reshape(rvec_mean[:,i],(-1,1))/eigv_pre[i]**2)/(norm_for_lvec[i,0]**2+norm_for_lvec[i,1]**2)
        
        ### compute ml   
        lmean_eigv_rescale = lmean_tmp_rescale*norm_lvec[i]*eigvchn_theo[it,i]
        ### more simplified version using lvec_mean and rvec_mean
        intg_ml0[it,i,:] = np.squeeze(lmean_tmp_rescale)
        intg_mr[it,i,:]  = np.squeeze(rmean_tmp_rescale)
        intg_ml[it,i,:]  = np.squeeze(lmean_eigv_rescale)
        
        intg_norml0[it,i,:] = lnorm_tmp_rescale
        intg_normr[it,i,:]  = rnorm_tmp_rescale
        print('excitatory')
        print(it,'..',i,'mean:',np.mean(np.mean(eiglvec0_series[:,it,:NE,i],axis=1),axis=0),np.mean(intg_ml0[it,i,:NE]))
        print(it,'..',i,'mean:',np.mean(np.mean(eiglvec_series[:,it,:NE,i],axis=1),axis=0)*eigvchn_theo[it,i],np.mean(intg_ml[it,i,:NE]))
        print(it,'..',i,'mean:',np.mean(np.mean(eiglvec0_series[:,it,NE:,i],axis=1),axis=0),np.mean(intg_ml0[it,i,NE:]))
        print(it,'..',i,'mean:',np.mean(np.mean(eiglvec_series[:,it,NE:,i],axis=1),axis=0)*eigvchn_theo[it,i],np.mean(intg_ml[it,i,NE:]))

Save the data 

In [12]:

# params = {'g':g,
#           'gamma':gamma,
#           'NE':NE,
#           'NI':NI,
#           'J':J,
#           'ce':ce,
#           'ci':ci,
#           'tau_series':tau_series,
#           }
# lst = [eigvchn_series, eigrvec_series, eiglvec_series,
#        eigrvec_series_rec, eiglvec_series_rec,
#         eiglvec0_series, norml0_series, params,
#         intg_ml,intg_ml0,intg_mr,
#         intg_norml0,intg_normr,
#         leig0mean_series,
#         firing_rateeq,
#         norm_4lvec_series,norm_4rvec_series,norm_4lvec_series_]
# stg = ["eigvchn_series, eigrvec_series, eiglvec_series,"
#        "eigrvec_series_rec, eiglvec_series_rec,"
#         "eiglvec0_series, norml0_series, params,"
#         "intg_ml,intg_ml0,intg_mr,"
#         "intg_norml0,intg_normr,"
#         "leig0mean_series,"
#         "firing_rateeq,"
#         "norm_4lvec_series,norm_4rvec_series,norm_4lvec_series_"]
# data = list_to_dict(lst=lst, string=stg)
switch = 4
# data_name = "/Users/shaoyuxiu/Library/CloudStorage/Dropbox/DailyWork/Allen_project/preparation4paper_Data/Numerics_remote/Biological_PRX_60N1500_normlr_theory_compare_update_inttheo_"+str(switch)+"_.npz"
data_name = "E:/Dropbox/DailyWork/Allen_project/preparation4paper_Data/Numerics/Biological_PRX_60N1500_normlr_theory_compare_update_inttheo_"+str(switch)+"_.npz"
# data_name = "/Users/shaoyuxiu/Library/CloudStorage/Dropbox/DailyWork/Allen_project/preparation4paper_Data/Numerics/Biological_PRX_60N1500_normlr_theory_compare.npz"
# # data_name = "your data folder/ConnStats_Adjacency_06Dec_16PRX.npz"
# np.savez(data_name, **data)
data = np.load(data_name,allow_pickle=True)
### also loading other variables 
eigvchn_series  = data['eigvchn_series']
eigrvec_series  = data['eigrvec_series']
eiglvec_series  = data['eiglvec_series']
eiglvec0_series = data['eiglvec0_series']
params   = data['params']
firing_rateeq = data['firing_rateeq']
intg_ml0 = data['intg_ml0']
intg_mr  = data['intg_mr']
intg_ml  = data['intg_ml']


the Gaussian network model

In [13]:
### compute the trial averaged mean 
# eigvchn_series = data['eigvchn_series']
# ntau = np.shape(eigvchn_series)[1]
# trials = np.shape(eigvchn_series)[0]
# NE, NI = params['NE'],params['NI']
### numerical  
mean_reigvec_num_series = np.zeros((trials,ntau,2,2))
mean_leigvec_num_series = np.zeros((trials,ntau,2,2))
mean_leig0vec_num_series = np.zeros((trials,ntau,2,2))
thl = 1.5
ths = 1.5
threshold = thl       
### for the numerical 
for ktrial in range(trials):
    for it in range(ntau):
        for ir in range(2):
            mean_reigvec_num_series[ktrial,it,ir,0] = np.mean(eigrvec_series[ktrial,it,:NE,ir])
            mean_reigvec_num_series[ktrial,it,ir,1] = np.mean(eigrvec_series[ktrial,it,NE:,ir])
            mean_leigvec_num_series[ktrial,it,ir,0] = np.mean(eiglvec_series[ktrial,it,:NE,ir])*eigvchn_series[ktrial,it,ir].real
            mean_leigvec_num_series[ktrial,it,ir,1] = np.mean(eiglvec_series[ktrial,it,NE:,ir])*eigvchn_series[ktrial,it,ir].real
            mean_leig0vec_num_series[ktrial,it,ir,0] = np.mean(eiglvec0_series[ktrial,it,:NE,ir])
            mean_leig0vec_num_series[ktrial,it,ir,1] = np.mean(eiglvec0_series[ktrial,it,NE:,ir])
            
### select the middle 30 values 
kktrial = np.arange(trials)
cuts = 3#6#
for it in range(ntau):
    for ir in range(2):        
        idxsort = np.argsort(mean_reigvec_num_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:]) 
        mean_reigvec_num_series[idxnan,it,ir,0] = np.nan 
        idxnan = np.where(np.abs(mean_reigvec_num_series[:,it,ir,0].real)>threshold)[0]
        mean_reigvec_num_series[idxnan,it,ir,0] = np.nan
        
        idxsort = np.argsort(mean_reigvec_num_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_reigvec_num_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_reigvec_num_series[:,it,ir,1].real)>threshold)[0]
        mean_reigvec_num_series[idxnan,it,ir,1] = np.nan
        
        idxsort = np.argsort(mean_leigvec_num_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leigvec_num_series[idxnan,it,ir,0] = np.nan
        
        idxsort = np.argsort(mean_leigvec_num_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leigvec_num_series[idxnan,it,ir,1] = np.nan



In [12]:
## multiply the numerical simulation by np.sqrt(N)
mean_reigvec_num_series  *= np.sqrt(N)
mean_leigvec_num_series  *= np.sqrt(N)
mean_leig0vec_num_series *= np.sqrt(N)

mean_reigvec_num_series_sp  *= np.sqrt(N)
mean_leigvec_num_series_sp  *= np.sqrt(N)
mean_leig0vec_num_series_sp *= np.sqrt(N)

intg_ml  *= np.sqrt(N)
intg_ml0 *= np.sqrt(N)
intg_mr  *= np.sqrt(N)

In [13]:
### plot the mean of the left and right eigenvectors
dtau = tau_series[1]-tau_series[0]
tau_max = tau_series[-1]
fig,ax=plt.subplots(1,2,figsize=(8,3),sharex=True,sharey=True)
ax[0].fill_between(tau_series,np.nanmean(mean_reigvec_num_series[:,:,0,0],axis=0)-np.nanstd(mean_reigvec_num_series[:,:,0,0],axis=0),np.nanmean(mean_reigvec_num_series[:,:,0,0],axis=0)+np.nanstd(mean_reigvec_num_series[:,:,0,0],axis=0),facecolor='orange',alpha=0.6)

ax[0].fill_between(tau_series,np.nanmean(mean_reigvec_num_series_sp[:,:,0,0],axis=0)-np.nanstd(mean_reigvec_num_series_sp[:,:,0,0],axis=0),np.nanmean(mean_reigvec_num_series_sp[:,:,0,0],axis=0)+np.nanstd(mean_reigvec_num_series_sp[:,:,0,0],axis=0),facecolor='tab:red',alpha=0.6)
# for i in range(trials):
#     ax[0].scatter(tau_series,mean_reigvec_num_series[i,:,0,0])
#     print(mean_reigvec_num_series[i,:,0,0])
ax[0].plot(tau_series,np.nanmean(intg_mr[:,0,:NE],axis=1),marker='o',color='orange',lw=1.5)
## second rank
ax[0].plot(tau_series,np.nanmean(intg_mr[:,1,:NE],axis=1),marker='o',color='green',lw=1.5)
ax[0].fill_between(tau_series,np.nanmean(mean_reigvec_num_series[:,:,1,0],axis=0)-np.nanstd(mean_reigvec_num_series[:,:,1,0],axis=0),np.nanmean(mean_reigvec_num_series[:,:,1,0],axis=0)+np.nanstd(mean_reigvec_num_series[:,:,1,0],axis=0),facecolor='green',alpha=0.6)
ax[0].fill_between(tau_series,np.nanmean(mean_reigvec_num_series_sp[:,:,1,0],axis=0)-np.nanstd(mean_reigvec_num_series_sp[:,:,1,0],axis=0),np.nanmean(mean_reigvec_num_series_sp[:,:,1,0],axis=0)+np.nanstd(mean_reigvec_num_series_sp[:,:,1,0],axis=0),facecolor='tab:blue',alpha=0.6)

### excitatory population
ax[0].set_title('Excitatory population',fontsize=12)
## move the x-axis to  the cecnter
ax[0].spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax[0].spines['top'].set_color('none')
ax[0].spines['right'].set_color('none')
ax[0].spines['left'].set_position(('data',0))
ax[0].set_xticks([0,0.1,0.2])
ax[0].set_xlim(0,tau_max+0.02/8)
ax[0].set_ylim(-0.01,0.04)
ax[0].set_yticks([0,0.04])
ax[0].set_ylim(-0.,1.5)
ax[0].set_yticks([0,1.5])
ax[0].set_xlabel('Chain-motif statistics \n'+r'$\tau$',fontsize=12)
ax[0].set_ylabel('Mean of right \n eigenvector(E)',fontsize=12)

### start the inhibitory population
ax[1].fill_between(tau_series,np.nanmean(mean_reigvec_num_series[:,:,0,1],axis=0)-np.nanstd(mean_reigvec_num_series[:,:,0,1],axis=0),np.nanmean(mean_reigvec_num_series[:,:,0,1],axis=0)+np.nanstd(mean_reigvec_num_series[:,:,0,1],axis=0),facecolor='orange',alpha=0.6)

ax[1].fill_between(tau_series,np.nanmean(mean_reigvec_num_series_sp[:,:,0,1],axis=0)-np.nanstd(mean_reigvec_num_series_sp[:,:,0,1],axis=0),np.nanmean(mean_reigvec_num_series_sp[:,:,0,1],axis=0)+np.nanstd(mean_reigvec_num_series_sp[:,:,0,1],axis=0),facecolor='tab:red',alpha=0.6)

ax[1].plot(tau_series,np.nanmean(intg_mr[:,0,NE:],axis=1),marker='o',color='orange',lw=1.5)
## second rank
ax[1].plot(tau_series,np.nanmean(intg_mr[:,1,NE:],axis=1),marker='o',color='green',lw=1.5)
ax[1].fill_between(tau_series,np.nanmean(mean_reigvec_num_series[:,:,1,1],axis=0)-np.nanstd(mean_reigvec_num_series[:,:,1,1],axis=0),np.nanmean(mean_reigvec_num_series[:,:,1,1],axis=0)+np.nanstd(mean_reigvec_num_series[:,:,1,1],axis=0),facecolor='green',alpha=0.6)

ax[1].fill_between(tau_series,np.nanmean(mean_reigvec_num_series_sp[:,:,1,1],axis=0)-np.nanstd(mean_reigvec_num_series_sp[:,:,1,1],axis=0),np.nanmean(mean_reigvec_num_series_sp[:,:,1,1],axis=0)+np.nanstd(mean_reigvec_num_series_sp[:,:,1,1],axis=0),facecolor='tab:blue',alpha=0.6)

### move the x-axis to  the cecnter
ax[1].spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax[1].spines['top'].set_color('none')
ax[1].spines['right'].set_color('none')
ax[1].spines['left'].set_position(('data',0))
ax[1].set_title('Inhibitory population',fontsize=12)
ax[1].set_xticks([0,0.1,0.2])
ax[1].set_xlim(0-dtau/8.0,tau_max+dtau/8)
# ax[1].set_ylim(-0.01,0.04)
ax[1].set_ylim(-0.0,1.5)
ax[1].set_xlabel('Chain-motif statistics \n'+r'$\tau$',fontsize=12)
ax[1].set_ylabel('Mean of right \n eigenvector(I)',fontsize=12)
fig.tight_layout()

In [14]:
### plot the mean of the left and right eigenvectors
dtau = tau_series[1]-tau_series[0]
tau_max = tau_series[-1]
fig,ax=plt.subplots(1,2,figsize=(8,3),sharex=True)
ax[0].fill_between(tau_series,np.nanmean(mean_leig0vec_num_series[:,:,0,0],axis=0)-np.nanstd(mean_leig0vec_num_series[:,:,0,0],axis=0),np.nanmean(mean_leig0vec_num_series[:,:,0,0],axis=0)+np.nanstd(mean_leig0vec_num_series[:,:,0,0],axis=0),facecolor='tab:orange',alpha=0.6)

ax[0].fill_between(tau_series,np.nanmean(mean_leig0vec_num_series_sp[:,:,0,0],axis=0)-np.nanstd(mean_leig0vec_num_series_sp[:,:,0,0],axis=0),np.nanmean(mean_leig0vec_num_series_sp[:,:,0,0],axis=0)+np.nanstd(mean_leig0vec_num_series_sp[:,:,0,0],axis=0),facecolor='tab:red',alpha=0.6)

ax[0].plot(tau_series,np.nanmean(intg_ml0[:,0,:NE],axis=1),marker='o',color='orange',lw=1.5)
## second rank
ax[0].plot(tau_series,np.nanmean(intg_ml0[:,1,:NE],axis=1),marker='o',color='green',lw=1.5)
ax[0].fill_between(tau_series,np.nanmean(mean_leig0vec_num_series[:,:,1,0],axis=0)-np.nanstd(mean_leig0vec_num_series[:,:,1,0],axis=0),np.nanmean(mean_leig0vec_num_series[:,:,1,0],axis=0)+np.nanstd(mean_leig0vec_num_series[:,:,1,0],axis=0),facecolor='tab:green',alpha=0.6)
ax[0].fill_between(tau_series,np.nanmean(mean_leig0vec_num_series_sp[:,:,1,0],axis=0)-np.nanstd(mean_leig0vec_num_series_sp[:,:,1,0],axis=0),np.nanmean(mean_leig0vec_num_series_sp[:,:,1,0],axis=0)+np.nanstd(mean_leig0vec_num_series_sp[:,:,1,0],axis=0),facecolor='tab:blue',alpha=0.6)

### excitatory population
ax[0].set_title('Excitatory population',fontsize=12)
## move the x-axis to  the cecnter
ax[0].spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax[0].spines['top'].set_color('none')
ax[0].spines['right'].set_color('none')
ax[0].spines['left'].set_position(('data',0))
ax[0].set_xticks([0,0.1,0.2])
ax[0].set_xlim(0,tau_max+0.02/8)
# ax[0].set_ylim(-0.01,0.04)
# ax[0].set_yticks([0,0.04])
# ax[0].set_ylim(-0.,1.5)
# ax[0].set_yticks([0,1.5])
ax[0].set_xlabel('Chain-motif statistics \n'+r'$\tau$',fontsize=12)
ax[0].set_ylabel('Mean of right \n eigenvector(E)',fontsize=12)

### start the inhibitory population
ax[1].fill_between(tau_series,np.nanmean(mean_leig0vec_num_series[:,:,0,1],axis=0)-np.nanstd(mean_leig0vec_num_series[:,:,0,1],axis=0),np.nanmean(mean_leig0vec_num_series[:,:,0,1],axis=0)+np.nanstd(mean_leig0vec_num_series[:,:,0,1],axis=0),facecolor='tab:orange',alpha=0.3)

ax[1].fill_between(tau_series,np.nanmean(mean_leig0vec_num_series_sp[:,:,0,1],axis=0)-np.nanstd(mean_leig0vec_num_series_sp[:,:,0,1],axis=0),np.nanmean(mean_leig0vec_num_series_sp[:,:,0,1],axis=0)+np.nanstd(mean_leig0vec_num_series_sp[:,:,0,1],axis=0),facecolor='tab:red',alpha=0.3)

ax[1].plot(tau_series,np.nanmean(intg_ml0[:,0,NE:],axis=1),marker='o',color='orange',lw=1.5)
## second rank
ax[1].plot(tau_series,np.nanmean(intg_ml0[:,1,NE:],axis=1),marker='o',color='green',lw=1.5)
ax[1].fill_between(tau_series,np.nanmean(mean_leig0vec_num_series[:,:,1,1],axis=0)-np.nanstd(mean_leig0vec_num_series[:,:,1,1],axis=0),np.nanmean(mean_leig0vec_num_series[:,:,1,1],axis=0)+np.nanstd(mean_leig0vec_num_series[:,:,1,1],axis=0),facecolor='tab:green',alpha=0.3)

ax[1].fill_between(tau_series,np.nanmean(mean_leig0vec_num_series_sp[:,:,1,1],axis=0)-np.nanstd(mean_leig0vec_num_series_sp[:,:,1,1],axis=0),np.nanmean(mean_leig0vec_num_series_sp[:,:,1,1],axis=0)+np.nanstd(mean_leig0vec_num_series_sp[:,:,1,1],axis=0),facecolor='tab:blue',alpha=0.3)

### move the x-axis to  the cecnter
ax[1].spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax[1].spines['top'].set_color('none')
ax[1].spines['right'].set_color('none')
ax[1].spines['left'].set_position(('data',0))
ax[1].set_title('Inhibitory population',fontsize=12)
ax[1].set_xticks([0,0.1,0.2])
ax[1].set_xlim(0-dtau/8.0,tau_max+dtau/8)
# ax[1].set_ylim(-0.1,0.14)
# ax[1].set_ylim(-0.0,1.5)
ax[1].set_xlabel('Chain-motif statistics \n'+r'$\tau$',fontsize=12)
ax[1].set_ylabel('Mean of right \n eigenvector(I)',fontsize=12)
fig.tight_layout()

In [15]:
### plot the mean of the left and right eigenvectors
dtau = tau_series[1]-tau_series[0]
tau_max = tau_series[-1]
fig,ax=plt.subplots(1,2,figsize=(8,3),sharex=True)
ax[0].fill_between(tau_series,np.nanmean(mean_leigvec_num_series[:,:,0,0],axis=0)-np.nanstd(mean_leigvec_num_series[:,:,0,0],axis=0),np.nanmean(mean_leigvec_num_series[:,:,0,0],axis=0)+np.nanstd(mean_leigvec_num_series[:,:,0,0],axis=0),facecolor='orange',alpha=0.6)

# ax[0].fill_between(tau_series,np.nanmean(mean_leigvec_num_series_sp[:,:,0,0]*eigvchn_series_sp[:,:,0],axis=0)-np.nanstd(mean_leigvec_num_series_sp[:,:,0,0]*eigvchn_series_sp[:,:,0],axis=0),np.nanmean(mean_leigvec_num_series_sp[:,:,0,0]*eigvchn_series_sp[:,:,0],axis=0)+np.nanstd(mean_leigvec_num_series_sp[:,:,0,0]*eigvchn_series_sp[:,:,0],axis=0),facecolor='tab:red',alpha=0.6)

ax[0].fill_between(tau_series,np.nanmean(mean_leigvec_num_series_sp[:,:,0,0],axis=0)-np.nanstd(mean_leigvec_num_series_sp[:,:,0,0],axis=0),np.nanmean(mean_leigvec_num_series_sp[:,:,0,0],axis=0)+np.nanstd(mean_leigvec_num_series_sp[:,:,0,0],axis=0),facecolor='tab:red',alpha=0.6)

ax[0].plot(tau_series,np.nanmean(intg_ml[:,0,:NE],axis=1),marker='o',color='orange',lw=1.5)
## second rank
ax[0].plot(tau_series,np.nanmean(intg_ml[:,1,:NE],axis=1),marker='o',color='green',lw=1.5)
ax[0].fill_between(tau_series,np.nanmean(mean_leigvec_num_series[:,:,1,0],axis=0)-np.nanstd(mean_leigvec_num_series[:,:,1,0],axis=0),np.nanmean(mean_leigvec_num_series[:,:,1,0],axis=0)+np.nanstd(mean_leigvec_num_series[:,:,1,0],axis=0),facecolor='green',alpha=0.6)

# ax[0].fill_between(tau_series,np.nanmean(mean_leigvec_num_series_sp[:,:,1,0]*eigvchn_series_sp[:,:,1],axis=0)-np.nanstd(mean_leigvec_num_series_sp[:,:,1,0]*eigvchn_series_sp[:,:,1],axis=0),np.nanmean(mean_leigvec_num_series_sp[:,:,1,0]*eigvchn_series_sp[:,:,1],axis=0)+np.nanstd(mean_leigvec_num_series_sp[:,:,1,0]*eigvchn_series_sp[:,:,1],axis=0),facecolor='tab:blue',alpha=0.6)
ax[0].fill_between(tau_series,np.nanmean(mean_leigvec_num_series_sp[:,:,1,0],axis=0)-np.nanstd(mean_leigvec_num_series_sp[:,:,1,0],axis=0),np.nanmean(mean_leigvec_num_series_sp[:,:,1,0],axis=0)+np.nanstd(mean_leigvec_num_series_sp[:,:,1,0],axis=0),facecolor='tab:blue',alpha=0.6)

### excitatory population
ax[0].set_title('Excitatory population',fontsize=12)
## move the x-axis to  the cecnter
ax[0].spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax[0].spines['top'].set_color('none')
ax[0].spines['right'].set_color('none')
ax[0].spines['left'].set_position(('data',0))
ax[0].set_xticks([0,0.1,0.2])
ax[0].set_xlim(0,tau_max+0.02/8)
# ax[0].set_ylim(-0.01,0.04)
# ax[0].set_yticks([0,0.04])
ax[0].set_ylim(-2.0,8)
ax[0].set_yticks([-2.0,8])
ax[0].set_xlabel('Chain-motif statistics \n'+r'$\tau$',fontsize=12)
ax[0].set_ylabel('Mean of right \n eigenvector(E)',fontsize=12)

### start the inhibitory population
ax[1].fill_between(tau_series,np.nanmean(mean_leigvec_num_series[:,:,0,1],axis=0)-np.nanstd(mean_leigvec_num_series[:,:,0,1],axis=0),np.nanmean(mean_leigvec_num_series[:,:,0,1],axis=0)+np.nanstd(mean_leigvec_num_series[:,:,0,1],axis=0),facecolor='orange',alpha=0.6)

ax[1].plot(tau_series,np.nanmean(intg_ml[:,0,NE:],axis=1),marker='o',color='orange',lw=1.5)
## second rank
ax[1].plot(tau_series,np.nanmean(intg_ml[:,1,NE:],axis=1),marker='o',color='green',lw=1.5)
ax[1].fill_between(tau_series,np.nanmean(mean_leigvec_num_series[:,:,1,1],axis=0)-np.nanstd(mean_leigvec_num_series[:,:,1,1],axis=0),np.nanmean(mean_leigvec_num_series[:,:,1,1],axis=0)+np.nanstd(mean_leigvec_num_series[:,:,1,1],axis=0),facecolor='green',alpha=0.6)

# ax[1].fill_between(tau_series,np.nanmean(mean_leigvec_num_series_sp[:,:,0,1]*eigvchn_series_sp[:,:,0],axis=0)-np.nanstd(mean_leigvec_num_series_sp[:,:,0,1]*eigvchn_series_sp[:,:,0],axis=0),np.nanmean(mean_leigvec_num_series_sp[:,:,0,1]*eigvchn_series_sp[:,:,0],axis=0)+np.nanstd(mean_leigvec_num_series_sp[:,:,0,1]*eigvchn_series_sp[:,:,0],axis=0),facecolor='tab:red',alpha=0.6)

ax[1].fill_between(tau_series,np.nanmean(mean_leigvec_num_series_sp[:,:,0,1],axis=0)-np.nanstd(mean_leigvec_num_series_sp[:,:,0,1],axis=0),np.nanmean(mean_leigvec_num_series_sp[:,:,0,1],axis=0)+np.nanstd(mean_leigvec_num_series_sp[:,:,0,1],axis=0),facecolor='tab:red',alpha=0.6)

# ax[1].fill_between(tau_series,np.nanmean(mean_leigvec_num_series_sp[:,:,1,1]*eigvchn_series_sp[:,:,1],axis=0)-np.nanstd(mean_leigvec_num_series_sp[:,:,1,1]*eigvchn_series_sp[:,:,1],axis=0),np.nanmean(mean_leigvec_num_series_sp[:,:,1,1]*eigvchn_series_sp[:,:,1],axis=0)+np.nanstd(mean_leigvec_num_series_sp[:,:,1,1]*eigvchn_series_sp[:,:,1],axis=0),facecolor='tab:blue',alpha=0.6)

ax[1].fill_between(tau_series,np.nanmean(mean_leigvec_num_series_sp[:,:,1,1],axis=0)-np.nanstd(mean_leigvec_num_series_sp[:,:,1,1],axis=0),np.nanmean(mean_leigvec_num_series_sp[:,:,1,1],axis=0)+np.nanstd(mean_leigvec_num_series_sp[:,:,1,1],axis=0),facecolor='tab:blue',alpha=0.6)
### move the x-axis to  the cecnter
ax[1].spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax[1].spines['top'].set_color('none')
ax[1].spines['right'].set_color('none')
ax[1].spines['left'].set_position(('data',0))
ax[1].set_title('Inhibitory population',fontsize=12)
ax[1].set_xticks([0,0.1,0.2])
ax[1].set_xlim(0-dtau/8.0,tau_max+dtau/8)
ax[1].set_ylim(-40,20)
ax[1].set_yticks([-40,0,20])
ax[1].set_xlabel('Chain-motif statistics \n'+r'$\tau$',fontsize=12)
ax[1].set_ylabel('Mean of right \n eigenvector(I)',fontsize=12)
fig.tight_layout()

Dynamics, compute the mean 

In [16]:
### multiply the numerical simulation by np.sqrt(N)
mean_reigvec_num_series /= np.sqrt(N)
mean_leigvec_num_series /= np.sqrt(N)

mean_reigvec_num_series_sp /= np.sqrt(N)
mean_leigvec_num_series_sp /= np.sqrt(N)

intg_ml /= np.sqrt(N)
intg_mr /= np.sqrt(N)

In [ ]:
# ### compute the population mean and variance
# # ntau = 6
# firing_rateeq_mean_sp,firing_rateeq_var_sp = np.zeros((trials,ntau,2)),np.zeros((trials,ntau,2))
# firing_rateeq_mean_sp[:,:,0],firing_rateeq_mean_sp[:,:,1] = np.mean(firing_rateeq_sp[:,:,:NE],axis=2),np.mean(firing_rateeq_sp[:,:,NE:],axis=2)

# firing_rateeq_mean,firing_rateeq_var = np.zeros((trials,ntau,2)),np.zeros((trials,ntau,2))
# firing_rateeq_mean[:,:,0],firing_rateeq_mean[:,:,1] = np.mean(firing_rateeq[:,:,:NE],axis=2),np.mean(firing_rateeq[:,:,NE:],axis=2)

In [ ]:
# ### sort firing_rate_lw_fr_sp[:,it,0]
# firing_rateeq_mean_sp[np.where(firing_rateeq_mean_sp<0)]=np.nan
# firing_rateeq_mean_sp[np.where(firing_rateeq_mean_sp<0)]=np.nan

# firing_rateeq_mean_sp[np.where(firing_rateeq_mean_sp>0.6)]=np.nan
# firing_rateeq_mean_sp[np.where(firing_rateeq_mean_sp>0.6)]=np.nan

# firing_rateeq_mean[np.where(firing_rateeq_mean<0)]=np.nan
# firing_rateeq_mean[np.where(firing_rateeq_mean<0)]=np.nan

# firing_rateeq_mean[np.where(firing_rateeq_mean>0.6)]=np.nan
# firing_rateeq_mean[np.where(firing_rateeq_mean>0.6)]=np.nan

In [ ]:
# frate_real_sp = firing_rateeq_mean_sp[:,-1,0].copy()
# ### sorting eigvchn_real 
# idx = np.argsort(frate_real_sp)
# idx_eff = idx[3:-3]
# print('effective trials:',idx_eff,len(idx_eff))

In [14]:
### compute low-rank theoretical
# contribution_fr = np.zeros((trials,ntau,2,2))
# firing_rate_lwfr = np.zeros((trials,ntau,2))
# for ktrial in range(trials):
#     for it in range(ntau):
#             for ir in range(2):
#                 contribution_fr[ktrial,it,ir,0] = np.squeeze(((mean_leigvec_num_series[ktrial,it,ir,0])*Inp[0]*NE+(mean_leigvec_num_series[ktrial,it,ir,1])*Inp[0]*NI)/(1-eigvchn_series[ktrial,it,ir].real)*(mean_reigvec_num_series[ktrial,it,ir,0]))
#                 contribution_fr[ktrial,it,ir,1] = np.squeeze(((mean_leigvec_num_series[ktrial,it,ir,0])*Inp[0]*NE+(mean_leigvec_num_series[ktrial,it,ir,1])*Inp[0]*NI)/(1-eigvchn_series[ktrial,it,ir].real)*(mean_reigvec_num_series[ktrial,it,ir,1]))
#     firing_rate_lwfr[ktrial,:,:] = np.ones((ntau,2))*Inp[0]
#     firing_rate_lwfr[ktrial,:,:] +=np.sum(contribution_fr[ktrial,:,:,:].copy(),axis=1)

### compute low-rank theoretical
contribution_fr = np.zeros((trials,ntau,2,2))
firing_rate_lwfr = np.zeros((trials,ntau,2))
for ktrial in range(trials):
    for it in range(ntau):
            for ir in range(2):
                contribution_fr[ktrial,it,ir,0] = np.squeeze(((mean_leigvec_num_series[ktrial,it,ir,0])*Inp[0]*NE+(mean_leigvec_num_series[ktrial,it,ir,1])*Inp[0]*NI)/(1-eigvchn_theo[it,ir].real)*(mean_reigvec_num_series[ktrial,it,ir,0]))
                contribution_fr[ktrial,it,ir,1] = np.squeeze(((mean_leigvec_num_series[ktrial,it,ir,0])*Inp[0]*NE+(mean_leigvec_num_series[ktrial,it,ir,1])*Inp[0]*NI)/(1-eigvchn_theo[it,ir].real)*(mean_reigvec_num_series[ktrial,it,ir,1]))
    firing_rate_lwfr[ktrial,:,:] = np.ones((ntau,2))*Inp[0]
    firing_rate_lwfr[ktrial,:,:] +=np.sum(contribution_fr[ktrial,:,:,:].copy(),axis=1)


In [15]:
# ## compute low-rank theoretical
# trials = 36
# contribution_fr_sp = np.zeros((trials,ntau,2,2))
# firing_rate_lwfr_sp = np.zeros((trials,ntau,2))
# for ktrial in range(trials):
#     for it in range(ntau):
#             for ir in range(2):
#                 contribution_fr_sp[ktrial,it,ir,0] = np.squeeze(((mean_leigvec_num_series_sp[ktrial,it,ir,0])*Inp[0]*NE+(mean_leigvec_num_series_sp[ktrial,it,ir,1])*Inp[0]*NI)/(1-eigvchn_series_sp[ktrial,it,ir].real)*(mean_reigvec_num_series_sp[ktrial,it,ir,0]))
#                 contribution_fr_sp[ktrial,it,ir,1] = np.squeeze(((mean_leigvec_num_series_sp[ktrial,it,ir,0])*Inp[0]*NE+(mean_leigvec_num_series_sp[ktrial,it,ir,1])*Inp[0]*NI)/(1-eigvchn_series_sp[ktrial,it,ir].real)*(mean_reigvec_num_series_sp[ktrial,it,ir,1]))
#     firing_rate_lwfr_sp[ktrial,:,:] = np.ones((ntau,2))*Inp[0]
#     firing_rate_lwfr_sp[ktrial,:,:] +=np.sum(contribution_fr_sp[ktrial,:,:,:].copy(),axis=1)


contribution_fr_sp = np.zeros((trials,ntau,2,2))
firing_rate_lwfr_sp = np.zeros((trials,ntau,2))
for ktrial in range(trials):
    for it in range(ntau):
            for ir in range(2):
                contribution_fr_sp[ktrial,it,ir,0] = np.squeeze(((mean_leigvec_num_series_sp[ktrial,it,ir,0])*Inp[0]*NE+(mean_leigvec_num_series_sp[ktrial,it,ir,1])*Inp[0]*NI)/(1-eigvchn_theo[it,ir].real)*(mean_reigvec_num_series_sp[ktrial,it,ir,0]))
                contribution_fr_sp[ktrial,it,ir,1] = np.squeeze(((mean_leigvec_num_series_sp[ktrial,it,ir,0])*Inp[0]*NE+(mean_leigvec_num_series_sp[ktrial,it,ir,1])*Inp[0]*NI)/(1-eigvchn_theo[it,ir].real)*(mean_reigvec_num_series_sp[ktrial,it,ir,1]))
    firing_rate_lwfr_sp[ktrial,:,:] = np.ones((ntau,2))*Inp[0]
    firing_rate_lwfr_sp[ktrial,:,:] +=np.sum(contribution_fr_sp[ktrial,:,:,:].copy(),axis=1)


In [ ]:
# ### compute the population mean and variance
# # ntau = 6
# firing_rateeq_mean_sp,firing_rateeq_var_sp = np.zeros((trials,ntau,2)),np.zeros((trials,ntau,2))
# firing_rateeq_mean_sp[:,:,0],firing_rateeq_mean_sp[:,:,1] = np.mean(firing_rateeq_sp[:,:,:NE],axis=2),np.mean(firing_rateeq_sp[:,:,NE:],axis=2)

# firing_rateeq_mean,firing_rateeq_var = np.zeros((trials,ntau,2)),np.zeros((trials,ntau,2))
# firing_rateeq_mean[:,:,0],firing_rateeq_mean[:,:,1] = np.mean(firing_rateeq[:,:,:NE],axis=2),np.mean(firing_rateeq[:,:,NE:],axis=2)

In [16]:
### compute the population mean and variance
# ntau = 6
firing_rateeq_mean_sp,firing_rateeq_var_sp = np.zeros((trials,ntau,2)),np.zeros((trials,ntau,2))
firing_rateeq_mean_sp[:,:,0],firing_rateeq_mean_sp[:,:,1] = np.mean(firing_rateeq_sp[:,:,:NE],axis=2),np.mean(firing_rateeq_sp[:,:,NE:],axis=2)

firing_rateeq_mean,firing_rateeq_var = np.zeros((trials,ntau,2)),np.zeros((trials,ntau,2))
firing_rateeq_mean[:,:,0],firing_rateeq_mean[:,:,1] = np.mean(firing_rateeq[:,:,:NE],axis=2),np.mean(firing_rateeq[:,:,NE:],axis=2)


cuts = 6
for it in range(ntau):    
    idxsort = np.argsort(firing_rateeq_mean_sp[:,it,0])
    idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
    firing_rateeq_mean_sp[idxnan,it,0] = np.nan 
    
    idxsort = np.argsort(firing_rateeq_mean_sp[:,it,1])
    idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
    firing_rateeq_mean_sp[idxnan,it,1] = np.nan 
    
    idxsort = np.argsort(firing_rateeq_mean[:,it,0])
    idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
    firing_rateeq_mean[idxnan,it,0] = np.nan 
    
    idxsort = np.argsort(firing_rateeq_mean[:,it,1])
    idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
    firing_rateeq_mean[idxnan,it,1] = np.nan 


### ensuring there is no extreme value
firing_rate_lwfr_sp[np.where(firing_rate_lwfr_sp<0)]=np.nan
firing_rate_lwfr[np.where(firing_rate_lwfr<0)]=np.nan

firing_rate_lwfr_sp[np.where(firing_rate_lwfr_sp>0.6)]=np.nan
firing_rate_lwfr[np.where(firing_rate_lwfr>0.6)]=np.nan

firing_rateeq_mean_sp[np.where(firing_rateeq_mean_sp<0)]=np.nan
firing_rateeq_mean[np.where(firing_rateeq_mean<0)]=np.nan

# firing_rateeq_mean_sp[np.where(firing_rateeq_mean_sp>0.6)]=np.nan
# firing_rateeq_mean[np.where(firing_rateeq_mean>0.6)]=np.nan
firing_rateeq_mean_sp[np.where(firing_rateeq_mean_sp>0.1)]=np.nan
firing_rateeq_mean[np.where(firing_rateeq_mean>0.1)]=np.nan

contribution_fr_sp[np.where(contribution_fr_sp<-0.1)]=np.nan
contribution_fr[np.where(contribution_fr<-0.1)]=np.nan

contribution_fr_sp[np.where(contribution_fr_sp>0.6)]=np.nan
contribution_fr[np.where(contribution_fr>0.6)]=np.nan
    
    
    

In [14]:
firing_rateeq_mean_sp[:,-1,0]

array([       nan,        nan,        nan,        nan,        nan,
              nan,        nan, 0.03022249,        nan, 0.06252094,
       0.03853489,        nan, 0.01816894, 0.02756891,        nan,
              nan, 0.06179391,        nan, 0.02630309,        nan,
              nan,        nan, 0.02429303,        nan,        nan,
              nan, 0.07095663,        nan,        nan, 0.04341639,
       0.09034802,        nan, 0.02752146, 0.03210229,        nan,
       0.05000267])

In [101]:
firing_rateeq_mean[:,-1,0]

array([0.09376232,        nan,        nan, 0.09486923, 0.05408936,
              nan,        nan, 0.05429838,        nan, 0.09295482,
              nan,        nan, 0.05965143, 0.08358152, 0.09992417,
       0.07698454,        nan, 0.05064881, 0.0738072 , 0.07961225,
       0.05669213, 0.05967071, 0.06515342,        nan, 0.04899413,
              nan, 0.08486104,        nan,        nan, 0.06515867,
       0.0586201 , 0.07609774, 0.08302774, 0.0675625 ,        nan,
       0.05100645])

In [84]:
(contribution_fr_sp[:,-1,1,0])

array([0.06358331,        nan, 0.0554049 , 0.05954807, 0.04125454,
              nan,        nan, 0.02155897,        nan,        nan,
       0.03630701,        nan, 0.00981939, 0.02339927, 0.04920797,
              nan, 0.04338026, 0.0265005 , 0.02871496, 0.05862191,
       0.05657845,        nan, 0.02812314, 0.02194056, 0.04294196,
              nan, 0.04419572, 0.0432468 , 0.05682763, 0.04273871,
       0.03656195,        nan, 0.03781607,        nan, 0.05627083,
       0.03712008])

In [85]:
(contribution_fr_sp[:,-1,0,0])

array([-0.01492369, -0.01200926,         nan,         nan, -0.01400793,
               nan,         nan,         nan,         nan, -0.01565592,
       -0.01311802,         nan, -0.01084018, -0.01302533, -0.01450504,
               nan,         nan, -0.01425147, -0.01317561,         nan,
       -0.01661233, -0.01641874,         nan, -0.01191823, -0.01497962,
               nan, -0.01288319, -0.01651859, -0.01427694, -0.01506263,
       -0.01374942, -0.01186371, -0.01178559, -0.01252692, -0.01321982,
       -0.01322849])

In [24]:
#### constant and deterministic input signal
Inp   = np.squeeze(np.ones((N,1)))/np.sqrt(N) 
tt = np.linspace(0,100,1000)
#### random and structural input signal 
### simulation using the mean network connectivity (inhibition-dominated)
Jpt   = Jbar.copy()
xinit = np.squeeze(np.random.normal(0, 1E-2, (1, N)))
xtemporal = odesimulation(tt, xinit, Jpt, Inp)
firing_rate0 = np.reshape(xtemporal[-1,:N],(N,1))

### generate i.i.d. s
def randbin(M,N,P):  
    return np.random.choice([0, 1], size=(M,N), p=[P, 1-P])

sparseiid_dyns = np.zeros((trials,N,1))
for ktrial in range(trials):
    ### generate iid sparse connectivity 
    Ecol = randbin(1,NE*N,1-ce)
    Icol = randbin(1,NI*N,1-ci)
    Ecol = np.reshape(Ecol,(N,NE))
    Icol = np.reshape(Icol,(N,NI))
    iidsparse = np.zeros((N,N))
    iidsparse[:,:NE], iidsparse[:,NE:] = Ecol.copy()*J,Icol.copy()*J*(-g)
    xinit = np.squeeze(np.random.normal(0, 1E-2, (1, N)))
    xtemporal = odesimulation(tt, xinit, iidsparse, Inp)
    firing_rate0 = np.reshape(xtemporal[-1,:N],(N,1))
    sparseiid_dyns[ktrial,:,:] = firing_rate0.copy()
    
sparseiid_dyns /= Inp[0] ### devided by ... give the response function
    
sparseiid_dyns = np.squeeze(sparseiid_dyns)
mean_sparseiid_dyns = np.zeros((trials,2))
for ktrial in range(trials):
    mean_sparseiid_dyns[ktrial,0],mean_sparseiid_dyns[ktrial,1]= np.mean(sparseiid_dyns[ktrial,:NE]),np.mean(sparseiid_dyns[ktrial,NE:])
trialavg_sparseiid_dyns = np.mean(mean_sparseiid_dyns,axis=0)
trialstd_sparseiid_dyns = np.std(mean_sparseiid_dyns,axis=0)
print(trialavg_sparseiid_dyns)

[0.39572611 0.39366107]


Figure Finale

In [18]:
''' Theoretical prediction of the intersection between radius and eigenvalue outliers'''

ce, ci = c,c

def cal_radius_eigv_intersection(x, J,g,gamma,ce,ci,N,ALPHAE,ALPHAI):
    tau = x[0]
    ### mean connectivity 
    je, ji = J, g*J
    hat_sigmae, hat_sigmai = np.sqrt(ce*(1-ce)),np.sqrt(ci*(1-ci))
    sigmae, sigmai = np.sqrt(je**2*ce*(1-ce)*N), np.sqrt(ji**2*ci*(1-ci)*N) 
    ### generate tau_div_vec, tau_con_vec, tau_rec_vec, tau_chn_vec
    tau_div_vec, tau_con_vec, tau_rec_vec, tau_chn_vec = np.zeros(2),np.zeros((2,2)),np.zeros((2,2)),np.zeros((2,2))
    ### YS: alltaus do not have E-I polarity
    tau_div_vec = np.array([tau,tau])
    tau_con_vec = np.array([[tau,tau],[tau,tau]])
    # tau_rec_vec = np.array([[0,0],[0,0]])
    tau_rec_vec = np.array([[2*tau,2*tau],[2*tau,2*tau]])
    tau_chn_vec = np.array([[tau,tau],[tau,tau]])
    # gamma = 1.0 ## NI/NE
    sigma = np.sqrt(sigmae**2/(1+gamma)+sigmai**2*gamma/(1+gamma))
    ### compute the radius 
    # compute a single number for multiple populations
    tau_div_num = hat_sigmae**2*tau_div_vec[0]+gamma*g**2*hat_sigmai**2*tau_div_vec[1]
    tau_div_den = hat_sigmae**2+gamma*g**2*hat_sigmai**2
    tau_div = tau_div_num/tau_div_den

    tau_con_num = hat_sigmae**2*tau_con_vec[0,0]+gamma*g**2*hat_sigmai**2*tau_con_vec[1,1]
    tau_con_den = hat_sigmae**2+gamma*g**2*hat_sigmai**2
    tau_con  = tau_con_num/tau_con_den

    tau_rec_num = hat_sigmae**2*tau_rec_vec[0,0] - (1+gamma)*g*hat_sigmae*hat_sigmai*tau_rec_vec[0,1]+gamma*g**2*hat_sigmai**2*tau_rec_vec[1,1]
    tau_rec_den = hat_sigmae**2+gamma*g**2*hat_sigmai**2
    tau_rec = tau_rec_num/tau_rec_den/2.0 

    tau_chn_num = hat_sigmae**2*tau_chn_vec[0,0] - 0.5*(1+gamma)*g*hat_sigmae*hat_sigmai*(tau_chn_vec[0,1]+tau_chn_vec[1,0])+gamma*g**2*hat_sigmai**2*tau_chn_vec[1,1]
    tau_chn_den = hat_sigmae**2+gamma*g**2*hat_sigmai**2
    tau_chn = tau_chn_num/tau_chn_den/2.0

    radius_theo = (1.0-tau_con-tau_div+tau_rec-2*tau_chn)/np.sqrt(1-tau_con-tau_div)*sigma
    
            
    ### theoretical prediction of the eigenvalues

    org_k = J*N*(ALPHAE-g*ALPHAI)/2.0

    eigvchn_theo0 = org_k*(c+np.sqrt(c**2+4*c*(1-c)*tau))
    eigvchn_theo1 = org_k*(c-np.sqrt(c**2+4*c*(1-c)*tau))
    
    return eigvchn_theo1-radius_theo

### copmpute the intersection 
from scipy.optimize import fsolve
tau_inter = fsolve(cal_radius_eigv_intersection,0.15,args=(J,g,gamma,ce,ci,N,ALPHAE,ALPHAI))
print('the intersection between radius and eigenvalue outliers:',tau_inter)
print('tau series:',tau_series)

### compute the eigvchn_theo[:,1] and radius_theo_map intersection
idx = np.argmin(np.abs(tau_series-tau_inter))
intersect = np.where(eigvchn_theo[:,1]<=radius_theo_map)[0]
print(intersect[-1])

the intersection between radius and eigenvalue outliers: [0.10711292]
tau series: [0.    0.025 0.05  0.075 0.1   0.125 0.15  0.175 0.2   0.225]
4


In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

idxpop = 0 ### excitatory population 

datasp = firing_rateeq_mean_sp[:,:,idxpop]/Inp[0]
data   = firing_rateeq_mean[:,:,idxpop]/Inp[0]

fig, ax = plt.subplots(figsize=(4,3))
# Plotting
htau = tau_series[1]-tau_series[0]
### error bar with standard deviation of firing_rate_lwfr the axis 0 
std_fr_sp = np.nanstd(datasp,axis=0)
ax.errorbar(tau_series,np.nanmean(datasp.real,axis=0),yerr=std_fr_sp,fmt='x',color='gray',ecolor='gray',alpha=1,ls='',elinewidth=1.5)
std_fr = np.nanstd(data,axis=0)
ax.errorbar(tau_series+htau/8,np.nanmean(data.real,axis=0),yerr=std_fr,fmt='^',color='gray',ecolor='gray',alpha=1,ls='',elinewidth=1.5)
ax.plot(tau_series,np.nanmean(firing_rate_lwfr_sp[:,:,0].real,axis=0)/Inp[0],color='gray',alpha=1,lw=1.5)

### set y-axis    

# ax.set_title('Side-by-Side Boxplots of Data1 and Data2 with Face Color Alpha')

### right and top axes visible = 'false'
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

ax.set_ylim(-1,3.5)
ax.set_yticks([0,3])

# ax.fill_between(tau_series, trialavg_sparseiid_dyns[0]*np.ones((ntau,))-trialstd_sparseiid_dyns[0]*np.ones((ntau,)),trialavg_sparseiid_dyns[0]*np.ones((ntau,))+trialstd_sparseiid_dyns[0]*np.ones((ntau,)),facecolor='gray',alpha=0.3)


ax.plot(tau_series, trialavg_sparseiid_dyns[0]*np.ones((ntau,)),color='gray',ls='--',alpha=1,lw=1.5)
ax.plot(tau_inter*np.ones(2),[-1,3.5],linestyle='-.',color='gray',linewidth=1.5)

### contribution_fr
### Plot the contributions from individual ranks
htau = tau_series[1]-tau_series[0]
### start with the excitatory neuron population
### contribution from the first rank-1 component Excitatory population
ax.plot(tau_series,np.nanmean(contribution_fr_sp[:,:,0,0].real,axis=0)/Inp[0],color='tab:orange',alpha=0.36,lw=1.5)
ax.plot(tau_series, np.nanmean(contribution_fr_sp[:,:,1,0].real,axis=0)/Inp[0],color='tab:green',alpha=0.36,lw=1.5)

### design the axes
ax.set_xlim(tau_series[0]-htau/8.0,tau_series[-1]+htau/4.0)
ax.set_xticks(tau_series[::2])
### move the axes to the center
ax.spines['left'].set_position(('data', 0))
ax.spines['bottom'].set_position(('data', 0))
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.set_xlabel(r'$\tau$',fontsize=14)
ax.set_ylabel(r'$\nu_{I}$',fontsize=14)
ax.tick_params(labelsize=12)
ax.set_title('Excitatory population',fontsize=12)


Text(0.5, 1.0, 'Excitatory population')

In [20]:
intersect[-1]

4

In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

idxpop = 0 ### excitatory population 

datasp = firing_rateeq_mean_sp[:,:,idxpop]
data   = firing_rateeq_mean[:,:,idxpop]
# datasp[np.where(np.abs(datasp)>0.2)] = np.nan   
# data[np.where(np.abs(data)>0.2)]     = np.nan
# Sample data
# Replace these with your actual m*n arrays
data1 = datasp#np.random.rand(10, 5)
data2 = data#np.random.rand(10, 5)

# Convert to DataFrame
df1 = pd.DataFrame(data1, columns=[f'Feature {i}' for i in range(data1.shape[1])])
df1['Dataset'] = 'Sparse'
df2 = pd.DataFrame(data2, columns=[f'Feature {i}' for i in range(data2.shape[1])])
df2['Dataset'] = 'Gauss. Approx.'

# Combine the two DataFrames
df = pd.concat([df1, df2])

# Melt the DataFrame for seaborn
df_melted = df.melt(id_vars='Dataset', var_name='Feature', value_name='Value')

fig, ax = plt.subplots(figsize=(4,3))
# Plotting
boxplot = sns.boxplot(ax=ax,x='Feature', y='Value', hue='Dataset', data=df_melted, palette=['tab:red','tab:orange'],boxprops=dict(alpha=.6,lw=0), medianprops=dict(lw=0), whiskerprops=dict(lw=0.3), capprops=dict(lw=0.3), showfliers=False)

### set y-axis    

# ax.set_title('Side-by-Side Boxplots of Data1 and Data2 with Face Color Alpha')

### right and top axes visible = 'false'
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

ax.set_ylim(-0.01,0.15)
ax.set_yticks([0,0.15])

ax.fill_between(np.arange(ntau), trialavg_sparseiid_dyns[0]*np.ones((ntau,))-trialstd_sparseiid_dyns[0]*np.ones((ntau,)),trialavg_sparseiid_dyns[0]*np.ones((ntau,))+trialstd_sparseiid_dyns[0]*np.ones((ntau,)),facecolor='gray',alpha=0.3)


In [25]:
# fig,ax=plt.subplots(figsize=(4,3))
htau = tau_series[1]-tau_series[0]
### error bar with standard deviation of firing_rate_lwfr the axis 0 
std_lwfr_sp = np.nanstd(firing_rate_lwfr_sp[:,:,0],axis=0)
ax.errorbar(np.arange(ntau),np.nanmean(firing_rate_lwfr_sp[:,:,0].real,axis=0),yerr=std_lwfr_sp,fmt='x',color='tab:red',ecolor='tab:red',alpha=1,ls='-',elinewidth=1.5)
std_lwfr = np.nanstd(firing_rate_lwfr[:,:,0],axis=0)
ax.errorbar(np.arange(ntau),np.nanmean(firing_rate_lwfr[:,:,0].real,axis=0),yerr=std_lwfr,fmt='x',color='tab:orange',ecolor='tab:orange',alpha=1,ls='-',elinewidth=1.5)
# ax.plot(np.arange(ntau), np.nanmean(firing_rate_lwfr_sp[:,:,idxpop].real,axis=0),color='tab:red',alpha=1,lw=1.5)

# ax.plot(np.arange(ntau), np.nanmean(firing_rate_lwfr[:,:,idxpop].real,axis=0),color='tab:orange',alpha=1,lw=1.5)

<ErrorbarContainer object of 3 artists>

Response function of excitatory neurons

In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

idxpop = 0 ### excitatory population 

datasp = firing_rateeq_mean_sp[:,:,idxpop]/Inp[0]
data   = firing_rateeq_mean[:,:,idxpop]/Inp[0]
# datasp[np.where(np.abs(datasp)>0.2)] = np.nan   
# data[np.where(np.abs(data)>0.2)]     = np.nan
# Sample data
# Replace these with your actual m*n arrays
data1 = datasp#np.random.rand(10, 5)
data2 = data#np.random.rand(10, 5)

# Convert to DataFrame
df1 = pd.DataFrame(data1, columns=[f'Feature {i}' for i in range(data1.shape[1])])
df1['Dataset'] = 'Sparse'
df2 = pd.DataFrame(data2, columns=[f'Feature {i}' for i in range(data2.shape[1])])
df2['Dataset'] = 'Gauss. Approx.'

# Combine the two DataFrames
df = pd.concat([df1, df2])

# Melt the DataFrame for seaborn
df_melted = df.melt(id_vars='Dataset', var_name='Feature', value_name='Value')

fig, ax = plt.subplots(figsize=(4,3))
# Plotting
boxplot = sns.boxplot(ax=ax,x='Feature', y='Value', hue='Dataset', data=df_melted, palette=['tab:red','tab:orange'],boxprops=dict(alpha=.6,lw=0), medianprops=dict(lw=0), whiskerprops=dict(lw=0.3), capprops=dict(lw=0.3), showfliers=False)

### set y-axis    

# ax.set_title('Side-by-Side Boxplots of Data1 and Data2 with Face Color Alpha')

### right and top axes visible = 'false'
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

ax.set_ylim(0,5)
ax.set_yticks([0,5])

ax.fill_between(np.arange(ntau), trialavg_sparseiid_dyns[0]*np.ones((ntau,))-trialstd_sparseiid_dyns[0]*np.ones((ntau,)),trialavg_sparseiid_dyns[0]*np.ones((ntau,))+trialstd_sparseiid_dyns[0]*np.ones((ntau,)),facecolor='gray',alpha=0.3)

# fig,ax=plt.subplots(figsize=(4,3))
htau = tau_series[1]-tau_series[0]
### error bar with standard deviation of firing_rate_lwfr the axis 0 
# std_lwfr = np.nanstd(firing_rate_lwfr[:,:,idxpop]/Inp[0],axis=0)
# ax.plot(np.arange(ntau), np.nanmean(firing_rate_lwfr_sp[:,:,idxpop].real/Inp[0],axis=0),color='tab:red',alpha=1,lw=1.5)

# ax.plot(np.arange(ntau), np.nanmean(firing_rate_lwfr[:,:,idxpop].real/Inp[0],axis=0),color='tab:orange',alpha=1,lw=1.5)

std_lwfr_sp = np.nanstd(firing_rate_lwfr_sp[:,:,0]/Inp[0],axis=0)
ax.errorbar(np.arange(ntau),np.nanmean(firing_rate_lwfr_sp[:,:,0].real,axis=0)/Inp[0],yerr=std_lwfr_sp,fmt='x',color='tab:red',ecolor='tab:red',alpha=1,ls='-',elinewidth=1.5)
std_lwfr = np.nanstd(firing_rate_lwfr[:,:,0]/Inp[0],axis=0)
ax.errorbar(np.arange(ntau),np.nanmean(firing_rate_lwfr[:,:,0].real,axis=0)/Inp[0],yerr=std_lwfr,fmt='x',color='tab:orange',ecolor='tab:orange',alpha=1,ls='-',elinewidth=1.5)


<ErrorbarContainer object of 3 artists>

In [73]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

idxpop = 1 ### inhibitory population
datasp = firing_rateeq_mean_sp[:,:,idxpop]
data   = firing_rateeq_mean[:,:,idxpop]
# datasp[np.where(np.abs(datasp)>1e2)] = np.nan   
# data[np.where(np.abs(data)>1e2)]     = np.nan
# Sample data
# Replace these with your actual m*n arrays
data1 = datasp#np.random.rand(10, 5)
data2 = data#np.random.rand(10, 5)

# Convert to DataFrame
df1 = pd.DataFrame(data1, columns=[f'Feature {i}' for i in range(data1.shape[1])])
df1['Dataset'] = 'Sparse'
df2 = pd.DataFrame(data2, columns=[f'Feature {i}' for i in range(data2.shape[1])])
df2['Dataset'] = 'Gauss. Approx.'

# Combine the two DataFrames
df = pd.concat([df1, df2])

# Melt the DataFrame for seaborn
df_melted = df.melt(id_vars='Dataset', var_name='Feature', value_name='Value')

fig, ax = plt.subplots(figsize=(4,3))
# Plotting
# boxplot = sns.boxplot(ax=ax,x='Feature', y='Value', hue='Dataset', data=df_melted, palette=['tab:blue','tab:cyan'],boxprops=dict(alpha=.3))
# ### set y-axis    

boxplot = sns.boxplot(ax=ax,x='Feature', y='Value', hue='Dataset', data=df_melted, palette=['tab:blue','tab:cyan'],boxprops=dict(alpha=.6,lw=0), medianprops=dict(lw=0), whiskerprops=dict(lw=0.3), capprops=dict(lw=0.3), showfliers=False)

# ax.set_title('Side-by-Side Boxplots of Data1 and Data2 with Face Color Alpha')

### right and top axes visible = 'false'
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

ax.set_ylim(-0.01,0.15)
ax.set_yticks([0,0.15])


In [74]:
# fig,ax=plt.subplots(figsize=(4,3))
htau = tau_series[1]-tau_series[0]
### error bar with standard deviation of firing_rate_lwfr the axis 0 
ax.fill_between(np.arange(ntau), trialavg_sparseiid_dyns[0]*np.ones((ntau,))-trialstd_sparseiid_dyns[0]*np.ones((ntau,)),trialavg_sparseiid_dyns[0]*np.ones((ntau,))+trialstd_sparseiid_dyns[0]*np.ones((ntau,)),facecolor='gray',alpha=0.3)
# ax.errorbar(tau_series,np.nanmean(firing_rate_lwfr[idx_eff,:,0].real,axis=0),yerr=std_lwfr,fmt='x',color='tab:red',ecolor='tab:red',alpha=1,ls='',elinewidth=1.5)
ax.plot(np.arange(ntau), np.nanmean(firing_rate_lwfr_sp[:,:,idxpop].real,axis=0),color='tab:blue',alpha=1,lw=1.5)

ax.plot(np.arange(ntau), np.nanmean(firing_rate_lwfr[:,:,idxpop].real,axis=0),color='tab:cyan',alpha=1,lw=1.5)

2024-07-02 16:41:16.825 python[63083:2398226] Warning: Window move completed without beginning


Response function of inhibitory neurons

In [120]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

idxpop = 1 ### inhibitory population
datasp = firing_rateeq_mean_sp[:,:,idxpop]/Inp[-1]
data   = firing_rateeq_mean[:,:,idxpop]/Inp[-1]
# datasp[np.where(np.abs(datasp)>1e2)] = np.nan   
# data[np.where(np.abs(data)>1e2)]     = np.nan
# Sample data
# Replace these with your actual m*n arrays
data1 = datasp#np.random.rand(10, 5)
data2 = data#np.random.rand(10, 5)

# Convert to DataFrame
df1 = pd.DataFrame(data1, columns=[f'Feature {i}' for i in range(data1.shape[1])])
df1['Dataset'] = 'Sparse'
df2 = pd.DataFrame(data2, columns=[f'Feature {i}' for i in range(data2.shape[1])])
df2['Dataset'] = 'Gauss. Approx.'

# Combine the two DataFrames
df = pd.concat([df1, df2])

# Melt the DataFrame for seaborn
df_melted = df.melt(id_vars='Dataset', var_name='Feature', value_name='Value')

fig, ax = plt.subplots(figsize=(4,3))
# Plotting
# boxplot = sns.boxplot(ax=ax,x='Feature', y='Value', hue='Dataset', data=df_melted, palette=['tab:blue','tab:cyan'],boxprops=dict(alpha=.3))
# ### set y-axis    

boxplot = sns.boxplot(ax=ax,x='Feature', y='Value', hue='Dataset', data=df_melted, palette=['tab:blue','tab:cyan'],boxprops=dict(alpha=.6,lw=0), medianprops=dict(lw=0), whiskerprops=dict(lw=0.3), capprops=dict(lw=0.3), showfliers=False)

# ax.set_title('Side-by-Side Boxplots of Data1 and Data2 with Face Color Alpha')

### right and top axes visible = 'false'
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

ax.set_ylim(0,5)
ax.set_yticks([0,5])

# fig,ax=plt.subplots(figsize=(4,3))
htau = tau_series[1]-tau_series[0]
### error bar with standard deviation of firing_rate_lwfr the axis 0 
ax.fill_between(np.arange(ntau), trialavg_sparseiid_dyns[0]*np.ones((ntau,))-trialstd_sparseiid_dyns[0]*np.ones((ntau,)),trialavg_sparseiid_dyns[0]*np.ones((ntau,))+trialstd_sparseiid_dyns[0]*np.ones((ntau,)),facecolor='gray',alpha=0.3)
# ax.errorbar(tau_series,np.nanmean(firing_rate_lwfr[idx_eff,:,0].real,axis=0),yerr=std_lwfr,fmt='x',color='tab:red',ecolor='tab:red',alpha=1,ls='',elinewidth=1.5)
ax.plot(np.arange(ntau), np.nanmean(firing_rate_lwfr_sp[:,:,idxpop].real/Inp[-1],axis=0),color='tab:blue',alpha=1,lw=1.5)

ax.plot(np.arange(ntau), np.nanmean(firing_rate_lwfr[:,:,idxpop].real/Inp[-1],axis=0),color='tab:cyan',alpha=1,lw=1.5)

Contribution from different ranks

In [121]:
### Plot the contributions from individual ranks
fig,ax=plt.subplots(figsize=(4,3))
htau = tau_series[1]-tau_series[0]
### start with the excitatory neuron population
ax.errorbar(tau_series, np.nanmean(firing_rate_lwfr_sp[:,:,0],axis=0)/Inp[0],yerr=np.nanstd(firing_rate_lwfr_sp[:,:,0]/Inp[0],axis=0),fmt='x',color='tab:red',ecolor='tab:red',alpha=1,ls='',elinewidth=1.5)
### contribution from the first rank-1 component Excitatory population
ax.errorbar(tau_series, np.nanmean(contribution_fr_sp[:,:,0,0].real/Inp[0],axis=0),yerr=np.nanstd(contribution_fr_sp[:,:,0,0]/Inp[0],axis=0),fmt='x',color='tab:orange',ecolor='tab:orange',alpha=1,ls='',elinewidth=1.5)
ax.errorbar(tau_series, np.nanmean(contribution_fr_sp[:,:,1,0].real/Inp[0],axis=0),yerr=np.nanstd(contribution_fr_sp[:,:,1,0].real/Inp[0],axis=0),fmt='x',color='tab:green',ecolor='tab:green',alpha=1,ls='',elinewidth=1.5)

### design the axes
ax.set_xlim(tau_series[0]-htau/8.0,tau_series[-1]+htau/8.0)
ax.set_xticks([tau_series[0],tau_series[-1]])
ax.set_ylim(-1,3)
ax.set_yticks([-1,0,2])
### move the axes to the center
ax.spines['left'].set_position(('data', 0))
ax.spines['bottom'].set_position(('data', 0))
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.set_xlabel(r'$\tau$',fontsize=14)
ax.set_ylabel(r'$\nu_{I}$',fontsize=14)
ax.tick_params(labelsize=12)
ax.set_title('Excitatory population',fontsize=12)

### Plot the contributions from individual ranks
fig,ax=plt.subplots(figsize=(4,3))
htau = tau_series[1]-tau_series[0]
### start with the excitatory neuron population
ax.errorbar(tau_series, np.nanmean(firing_rate_lwfr_sp[:,:,1]/Inp[0],axis=0),yerr=np.nanstd(firing_rate_lwfr_sp[:,:,1]/Inp[0],axis=0),fmt='x',color='tab:blue',ecolor='tab:blue',alpha=1,ls='',elinewidth=1.5)
### contribution from the first rank-1 component Excitatory population
ax.errorbar(tau_series, np.nanmean(contribution_fr_sp[:,:,0,1].real/Inp[0],axis=0),yerr=np.nanstd(contribution_fr_sp[:,:,0,1]/Inp[0],axis=0),fmt='x',color='tab:orange',ecolor='tab:orange',alpha=1,ls='',elinewidth=1.5)
ax.errorbar(tau_series, np.nanmean(contribution_fr_sp[:,:,1,1].real/Inp[0],axis=0),yerr=np.nanstd(contribution_fr_sp[:,:,1,1].real/Inp[0],axis=0),fmt='x',color='tab:green',ecolor='tab:green',alpha=1,ls='',elinewidth=1.5)

ax.set_title('Inhibitory population',fontsize=12)

### design the axes
ax.set_xlim(tau_series[0]-htau/8.0,tau_series[-1]+htau/8.0)
ax.set_xticks([tau_series[0],tau_series[-1]])
ax.set_ylim(-1,3)
ax.set_yticks([-1,0,2])
### move the axes to the center
ax.spines['left'].set_position(('data', 0))
ax.spines['bottom'].set_position(('data', 0))
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.set_xlabel(r'$\tau$',fontsize=14)
ax.set_ylabel(r'$\nu_{I}$',fontsize=14)
ax.tick_params(labelsize=12)

2024-07-04 14:14:06.682 python[63083:2398226] Warning: Window move completed without beginning


In [ ]:
### Plot the contributions from individual ranks
fig,ax=plt.subplots(figsize=(4,3))
htau = tau_series[1]-tau_series[0]
### start with the excitatory neuron population
ax.errorbar(tau_series, np.nanmean(firing_rate_lwfr[:,:,0],axis=0),yerr=np.nanstd(firing_rate_lwfr[:,:,0],axis=0),fmt='x',color='tab:red',ecolor='tab:red',alpha=1,ls='-',elinewidth=1.5)
### contribution from the first rank-1 component Excitatory population
ax.errorbar(tau_series, np.nanmean(contribution_fr[:,:,0,0].real,axis=0),yerr=np.nanstd(contribution_fr[:,:,0,0],axis=0),fmt='x',color='tab:orange',ecolor='tab:orange',alpha=1,ls='-',elinewidth=1.5)
ax.errorbar(tau_series, np.nanmean(contribution_fr[:,:,1,0].real,axis=0),yerr=np.nanstd(contribution_fr[:,:,1,0].real,axis=0),fmt='x',color='tab:green',ecolor='tab:green',alpha=1,ls='-',elinewidth=1.5)

### design the axes
ax.set_xlim(tau_series[0]-htau/8.0,tau_series[-1]+htau/8.0)
ax.set_xticks([tau_series[0],tau_series[-1]])
ax.set_ylim(-0.02,0.05)
ax.set_yticks([0,0.05])
### move the axes to the center
ax.spines['left'].set_position(('data', 0))
ax.spines['bottom'].set_position(('data', 0))
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.set_xlabel(r'$\tau$',fontsize=14)
ax.set_ylabel(r'$\nu_{I}$',fontsize=14)
ax.tick_params(labelsize=12)
ax.set_title('Excitatory population',fontsize=12)

### Plot the contributions from individual ranks
fig,ax=plt.subplots(figsize=(4,3))
htau = tau_series[1]-tau_series[0]
### start with the excitatory neuron population
ax.errorbar(tau_series, np.nanmean(firing_rate_lwfr[:,:,1],axis=0),yerr=np.std(firing_rate_lwfr[:,:,1],axis=0),fmt='x',color='tab:blue',ecolor='tab:blue',alpha=1,ls='-',elinewidth=1.5)
### contribution from the first rank-1 component Excitatory population
ax.errorbar(tau_series, np.nanmean(contribution_fr[:,:,0,1].real,axis=0),yerr=np.nanstd(contribution_fr[:,:,0,1],axis=0),fmt='x',color='tab:orange',ecolor='tab:orange',alpha=1,ls='-',elinewidth=1.5)
ax.errorbar(tau_series, np.nanmean(contribution_fr[:,:,1,1].real,axis=0),yerr=np.nanstd(contribution_fr[:,:,1,1].real,axis=0),fmt='x',color='tab:green',ecolor='tab:green',alpha=1,ls='-',elinewidth=1.5)

ax.set_title('Inhibitory population',fontsize=12)

### design the axes
ax.set_xlim(tau_series[0]-htau/8.0,tau_series[-1]+htau/8.0)
ax.set_xticks([tau_series[0],tau_series[-1]])
ax.set_ylim(-0.02,0.05)
ax.set_yticks([0,0.05])
### move the axes to the center
ax.spines['left'].set_position(('data', 0))
ax.spines['bottom'].set_position(('data', 0))
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.set_xlabel(r'$\tau$',fontsize=14)
ax.set_ylabel(r'$\nu_{I}$',fontsize=14)
ax.tick_params(labelsize=12)

In [ ]:
#### constant and deterministic input signal
Inp   = np.squeeze(np.ones((N,1)))/np.sqrt(N) 
tt = np.linspace(0,100,1000)
#### random and structural input signal 
### simulation using the mean network connectivity (inhibition-dominated)
Jpt   = Jbar.copy()
xinit = np.squeeze(np.random.normal(0, 1E-2, (1, N)))
xtemporal = odesimulation(tt, xinit, Jpt, Inp)
firing_rate0 = np.reshape(xtemporal[-1,:N],(N,1))

ce=ci=c
### generate i.i.d. s
def randbin(M,N,P):  
    return np.random.choice([0, 1], size=(M,N), p=[P, 1-P])

sparseiid_dyns = np.zeros((trials,N,1))
for ktrial in range(trials):
    ### generate iid sparse connectivity 
    Ecol = randbin(1,NE*N,1-ce)
    Icol = randbin(1,NI*N,1-ci)
    Ecol = np.reshape(Ecol,(N,NE))
    Icol = np.reshape(Icol,(N,NI))
    iidsparse = np.zeros((N,N))
    iidsparse[:,:NE], iidsparse[:,NE:] = Ecol.copy()*J,Icol.copy()*J*(-g)
    xinit = np.squeeze(np.random.normal(0, 1E-2, (1, N)))
    xtemporal = odesimulation(tt, xinit, iidsparse, Inp)
    firing_rate0 = np.reshape(xtemporal[-1,:N],(N,1))
    sparseiid_dyns[ktrial,:,:] = firing_rate0.copy()
    
sparseiid_dyns = np.squeeze(sparseiid_dyns)
mean_sparseiid_dyns = np.zeros((trials,2))
for ktrial in range(trials):
    mean_sparseiid_dyns[ktrial,0],mean_sparseiid_dyns[ktrial,1]= np.mean(sparseiid_dyns[ktrial,:NE]),np.mean(sparseiid_dyns[ktrial,NE:])
trialavg_sparseiid_dyns = np.mean(mean_sparseiid_dyns,axis=0)
trialstd_sparseiid_dyns = np.std(mean_sparseiid_dyns,axis=0)
print(trialavg_sparseiid_dyns)

Compute the theoretical eigenvalue outliers and the spectral radius

In [ ]:
### compute the theoretical outliers and the spectral radius
### recording
eigvchn_theo   = np.zeros((ntau,2))
radius_theo_map = np.zeros(ntau)
### print variables and parameters of the adjacency matrix 
print('-------------------')
print('N:',N)
print('g:',g)
print('gamma:',gamma)
print('J:',J)
print('JE,JI:',JE,JI)
print('ge,gi:',ge,gi)
print('c:',c)
print('je,ji:',je,ji)
print('-------------------')


In [ ]:
ce, ci = c,c
for it, tau in enumerate(tau_series):
    ### mean connectivity 
    je, ji = J, g*J
    hat_sigmae, hat_sigmai = np.sqrt(ce*(1-ce)),np.sqrt(ci*(1-ci))
    sigmae, sigmai = np.sqrt(je**2*ce*(1-ce)*N), np.sqrt(ji**2*ci*(1-ci)*N) 
    ### generate tau_div_vec, tau_con_vec, tau_rec_vec, tau_chn_vec
    tau_div_vec, tau_con_vec, tau_rec_vec, tau_chn_vec = np.zeros(2),np.zeros((2,2)),np.zeros((2,2)),np.zeros((2,2))
    ### YS: alltaus do not have E-I polarity
    tau_div_vec = np.array([tau,tau])
    tau_con_vec = np.array([[tau,tau],[tau,tau]])
    # tau_rec_vec = np.array([[0,0],[0,0]])
    tau_rec_vec = np.array([[2*tau,2*tau],[2*tau,2*tau]])
    tau_chn_vec = np.array([[tau,tau],[tau,tau]])
    # gamma = 1.0 ## NI/NE
    sigma = np.sqrt(sigmae**2/(1+gamma)+sigmai**2*gamma/(1+gamma))
    ### compute the radius 
    # compute a single number for multiple populations
    tau_div_num = hat_sigmae**2*tau_div_vec[0]+gamma*g**2*hat_sigmai**2*tau_div_vec[1]
    tau_div_den = hat_sigmae**2+gamma*g**2*hat_sigmai**2
    tau_div = tau_div_num/tau_div_den

    tau_con_num = hat_sigmae**2*tau_con_vec[0,0]+gamma*g**2*hat_sigmai**2*tau_con_vec[1,1]
    tau_con_den = hat_sigmae**2+gamma*g**2*hat_sigmai**2
    tau_con  = tau_con_num/tau_con_den

    tau_rec_num = hat_sigmae**2*tau_rec_vec[0,0] - (1+gamma)*g*hat_sigmae*hat_sigmai*tau_rec_vec[0,1]+gamma*g**2*hat_sigmai**2*tau_rec_vec[1,1]
    tau_rec_den = hat_sigmae**2+gamma*g**2*hat_sigmai**2
    tau_rec = tau_rec_num/tau_rec_den/2.0 

    tau_chn_num = hat_sigmae**2*tau_chn_vec[0,0] - 0.5*(1+gamma)*g*hat_sigmae*hat_sigmai*(tau_chn_vec[0,1]+tau_chn_vec[1,0])+gamma*g**2*hat_sigmai**2*tau_chn_vec[1,1]
    tau_chn_den = hat_sigmae**2+gamma*g**2*hat_sigmai**2
    tau_chn = tau_chn_num/tau_chn_den/2.0

    radius_theo = (1.0-tau_con-tau_div+tau_rec-2*tau_chn)/np.sqrt(1-tau_con-tau_div)*sigma
    
    radius_theo_map[it] = radius_theo
            
### theoretical prediction of the eigenvalues

org_k = J*N*(ALPHAE-g*ALPHAI)/2.0

eigvchn_theo[:,0] = org_k*(c+np.sqrt(c**2+4*c*(1-c)*tau_series))
eigvchn_theo[:,1] = org_k*(c-np.sqrt(c**2+4*c*(1-c)*tau_series))

In [ ]:
# params = {'g':g,
#           'gamma':gamma,
#           'NE':NE,
#           'NI':NI,
#           'J':J,
#           'ce':ce,
#           'ci':ci,
#           'tau_series':tau_series,
#           }
# lst = [eigvchn_series, eigrvec_series, eiglvec_series,
#        eigrvec_series_rec, eiglvec_series_rec,
#         eiglvec0_series, norml0_series, params,
#         intg_ov_series,first_perturb_ov,first_perturb_ovP,
#         intg_mean_series,intg_std_series,intg_std_num_series,
#         intg_crossov_series,intg_crossovPop_series,leig0mean_series,
#         norm_4lvec_series,norm_4rvec_series,norm_4lvec_series_]
# stg = ["eigvchn_series, eigrvec_series, eiglvec_series,"
#        "eigrvec_series_rec, eiglvec_series_rec,"
#         "eiglvec0_series, norml0_series, params,"
#         "intg_ov_series,first_perturb_ov,first_perturb_ovP,"
#         "intg_mean_series,intg_std_series,intg_std_num_series,"
#         "intg_crossov_series,intg_crossovPop_series,leig0mean_series,"
#         "norm_4lvec_series,norm_4rvec_series,norm_4lvec_series_"]
# data = list_to_dict(lst=lst, string=stg)
data_name = "E:/Dropbox/DailyWork/Allen_project/preparation4paper_Data/sonets/equivalentGaussian/ConnStats_AdjGau_4April_PRX_68N1500_normlr_whilecode_used_vecrec.npz"
data_name = "/Users/shaoyuxiu/Library/CloudStorage/Dropbox/DailyWork/Allen_project/preparation4paper_Data/sonets/equivalentGaussian/ConnStats_AdjGau_4April_PRX_68N1500_normlr_whilecode_used_vecrec.npz"
# # data_name = "your data folder/ConnStats_Adjacency_06Dec_16PRX.npz"
# np.savez(data_name, **data)

In [ ]:
''' generate fig3 b'''

In [ ]:
# lst = [eigvchn_series, eigrvec_series, eiglvec_series,
#        eigrvec_series_rec, eiglvec_series_rec,
#         eiglvec0_series, norml0_series, params,
#         intg_ov_series,first_perturb_ov,first_perturb_ovP,
#         intg_mean_series,intg_std_series,intg_std_num_series,mean_shift_ov,
#         intg_crossov_series,intg_crossovPop_series]
# stg = ["eigvchn_series, eigrvec_series, eiglvec_series,"
#        "eigrvec_series_rec, eiglvec_series_rec,"
#         "eiglvec0_series, norml0_series, params,"
#         "intg_ov_series,first_perturb_ov,first_perturb_ovP,"
#         "intg_mean_series,intg_std_series,intg_std_num_series,mean_shift_ov,"
#         "intg_crossov_series,intg_crossovPop_series"]
# data = list_to_dict(lst=lst, string=stg)
# data_name = "E:/Dropbox/DailyWork/Allen_project/preparation4paper_Data/sonets/equivalentGaussian/ConnStats_AdjGau_4April_PRX_68N1500_normlr_whilecode_used_vecrec.npz"
# data_name = "E:/Dropbox/DailyWork/Allen_project/preparation4paper_Data/sonets/ConnStats_WD_Adjacency_4April_PRX_68N1500_normlr_whilecode_used_vecrec.npz"
# data_name = "your data folder/ConnStats_Adjacency_06Dec_16PRX.npz"
# np.savez(data_name, **data)
### load the stored data, in particular, extracting variable: eigvchn_series 
data = np.load(data_name,allow_pickle=True)

### also loading other variables 
eigvchn_series = data['eigvchn_series']
eigrvec_series = data['eigrvec_series']
eiglvec_series = data['eiglvec_series']
eigrvec_series_rec = data['eigrvec_series_rec']
eiglvec_series_rec = data['eiglvec_series_rec']
eiglvec0_series = data['eiglvec0_series']
norml0_series = data['norml0_series']
params = data['params']
intg_mean_series = data['intg_mean_series']
leig0mean_series = data['leig0mean_series']


In [ ]:
params[()]['NE']
trials=36
ntau=10

Plot the eigenvalue outlier (numerical, eigvchn_series), as well as the theoretically computed eigenvalue outliers (eigvchn_theo)

In [ ]:
### compute the numerical radius
radius_num_map = np.zeros((trials,ntau))
for ktrial in range(trials):
    for it in range(ntau): 
        eigvchn = eigvchn_series[ktrial,it,:].copy()

        eigvchn_real = np.real(eigvchn)
        eigvchn_real = np.sort(eigvchn_real)
        eigvchn_real = eigvchn_real[::-1]

        radius = (eigvchn_real[2])
        radius_num_map[ktrial,it] = radius

In [ ]:
#### print trial by trial the first two eigenvallues
for ktrial in range(trials):
    eigvchn = eigvchn_series[ktrial,:,:].copy()
    eigvchn_real = (eigvchn)
# ### delete the 17th trial
# eigvchn_series = np.delete(eigvchn_series,17,axis=0) #for ConnStats_Adjacency_12OctDense_.npz
### sorting the 3 maximum eigvchn_series[:,-1,0].real and 3 minimum
eigvchn_real = eigvchn_series[:,-1,0].real.copy()
### sorting eigvchn_real 
idx = np.argsort(eigvchn_real)
idx_eff = idx[3:-3]
print('effective trials:',idx_eff,len(idx_eff))

In [ ]:
### plot the numerical and theoretical eigenvalues outliers
fig,ax = plt.subplots(figsize=(6,4))
### label is r_{bulk}^{num}
ax.plot(tau_series,radius_theo_map,'gray',linestyle='--',label=r'$r_{bulk}^{theo}$')
ax.plot(tau_series,-radius_theo_map,'gray',linestyle='--')
# ax.plot(tau_series,np.mean(radius_num_map,axis=0),'k',label=r'$r_{bulk}^{num}$')
ax.fill_between(tau_series,np.mean(radius_num_map,axis=0),-np.mean(radius_num_map,axis=0),color='gray',alpha=0.15,label=r'$r_{bulk}^{num}$')
ax.set_xlabel(r'$\tau$')
ax.set_ylabel('Radius')
ax.legend()
plt.show()

ax.plot(tau_series,eigvchn_theo[:,0],'tab:red',label='theoretical')
ax.plot(tau_series,eigvchn_theo[:,1],'tab:green')

### plot error bar  
ax.errorbar(tau_series,np.mean(eigvchn_series[idx_eff,:,0].real,axis=0),yerr=np.std(eigvchn_series[idx_eff,:,0].real,axis=0),fmt='x',color='tab:red',ecolor='tab:red',alpha=1,ls='',elinewidth=1.5)

ax.errorbar(tau_series,np.mean(eigvchn_series[idx_eff,:,1].real,axis=0),yerr=np.std(eigvchn_series[idx_eff,:,1].real,axis=0),fmt='x',color='tab:green',ecolor='tab:green',alpha=1,ls='',elinewidth=1.5)
    
ax.set_xlabel(r'$\tau$')
ax.set_ylabel('Outliers')
### set x and y lims as well as ticks
dtau = tau_series[1]-tau_series[0] 
ax.set_xlim([tau_series[0]-dtau/8,tau_series[-1]+dtau/8])
ax.set_xticks([0,tau_series[-1]//2,tau_series[-1]])
ax.set_ylim([-4.5,2.5])
ax.set_yticks([-4,0,2])
ax.legend()
### move the x and y axis to the center
ax.spines['bottom'].set_position(('data',0))
ax.spines['left'].set_position(('data',0))
### set other x-axis invisible
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')
plt.show()

In [ ]:
''' Theoretical prediction of the intersection between radius and eigenvalue outliers'''

ce, ci = c,c

def cal_radius_eigv_intersection(x, J,g,gamma,ce,ci,N,ALPHAE,ALPHAI):
    tau = x[0]
    ### mean connectivity 
    je, ji = J, g*J
    hat_sigmae, hat_sigmai = np.sqrt(ce*(1-ce)),np.sqrt(ci*(1-ci))
    sigmae, sigmai = np.sqrt(je**2*ce*(1-ce)*N), np.sqrt(ji**2*ci*(1-ci)*N) 
    ### generate tau_div_vec, tau_con_vec, tau_rec_vec, tau_chn_vec
    tau_div_vec, tau_con_vec, tau_rec_vec, tau_chn_vec = np.zeros(2),np.zeros((2,2)),np.zeros((2,2)),np.zeros((2,2))
    ### YS: alltaus do not have E-I polarity
    tau_div_vec = np.array([tau,tau])
    tau_con_vec = np.array([[tau,tau],[tau,tau]])
    # tau_rec_vec = np.array([[0,0],[0,0]])
    tau_rec_vec = np.array([[2*tau,2*tau],[2*tau,2*tau]])
    tau_chn_vec = np.array([[tau,tau],[tau,tau]])
    # gamma = 1.0 ## NI/NE
    sigma = np.sqrt(sigmae**2/(1+gamma)+sigmai**2*gamma/(1+gamma))
    ### compute the radius 
    # compute a single number for multiple populations
    tau_div_num = hat_sigmae**2*tau_div_vec[0]+gamma*g**2*hat_sigmai**2*tau_div_vec[1]
    tau_div_den = hat_sigmae**2+gamma*g**2*hat_sigmai**2
    tau_div = tau_div_num/tau_div_den

    tau_con_num = hat_sigmae**2*tau_con_vec[0,0]+gamma*g**2*hat_sigmai**2*tau_con_vec[1,1]
    tau_con_den = hat_sigmae**2+gamma*g**2*hat_sigmai**2
    tau_con  = tau_con_num/tau_con_den

    tau_rec_num = hat_sigmae**2*tau_rec_vec[0,0] - (1+gamma)*g*hat_sigmae*hat_sigmai*tau_rec_vec[0,1]+gamma*g**2*hat_sigmai**2*tau_rec_vec[1,1]
    tau_rec_den = hat_sigmae**2+gamma*g**2*hat_sigmai**2
    tau_rec = tau_rec_num/tau_rec_den/2.0 

    tau_chn_num = hat_sigmae**2*tau_chn_vec[0,0] - 0.5*(1+gamma)*g*hat_sigmae*hat_sigmai*(tau_chn_vec[0,1]+tau_chn_vec[1,0])+gamma*g**2*hat_sigmai**2*tau_chn_vec[1,1]
    tau_chn_den = hat_sigmae**2+gamma*g**2*hat_sigmai**2
    tau_chn = tau_chn_num/tau_chn_den/2.0

    radius_theo = (1.0-tau_con-tau_div+tau_rec-2*tau_chn)/np.sqrt(1-tau_con-tau_div)*sigma
    
            
    ### theoretical prediction of the eigenvalues

    org_k = J*N*(ALPHAE-g*ALPHAI)/2.0

    eigvchn_theo0 = org_k*(c+np.sqrt(c**2+4*c*(1-c)*tau))
    eigvchn_theo1 = org_k*(c-np.sqrt(c**2+4*c*(1-c)*tau))
    
    return eigvchn_theo1-radius_theo

### copmpute the intersection 
from scipy.optimize import fsolve
tau_inter = fsolve(cal_radius_eigv_intersection,0.15,args=(J,g,gamma,ce,ci,N,ALPHAE,ALPHAI))
print('the intersection between radius and eigenvalue outliers:',tau_inter)
print('tau series:',tau_series)

### compute the eigvchn_theo[:,1] and radius_theo_map intersection
idx = np.argmin(np.abs(tau_series-tau_inter))
intersect = np.where(eigvchn_theo[:,1]<=radius_theo_map)[0]
print(intersect[-1])

In [ ]:
### plot error bar  
ax.errorbar(tau_series+0.00,np.mean(eigvchn_series[idx_eff,:,0].real,axis=0),yerr=np.std(eigvchn_series[idx_eff,:,0].real,axis=0),fmt='^',color='gray',ecolor='gray',alpha=1,ls='',elinewidth=1.5)

ax.errorbar(tau_series+0.00,np.mean(eigvchn_series[idx_eff,:,1].real,axis=0),yerr=np.std(eigvchn_series[idx_eff,:,1].real,axis=0),fmt='^',color='gray',ecolor='gray',alpha=1,ls='',elinewidth=1.5)
    
ax.set_xlabel(r'$\tau$')
ax.set_ylabel('Outliers')
### set x and y lims as well as ticks
dtau = tau_series[1]-tau_series[0] 
ax.set_xlim([tau_series[0]-dtau/8,tau_series[-1]+dtau/8])
ax.set_xticks([0,tau_series[-1]//2,tau_series[-1]])
ax.set_ylim([-4.5,2.5])
ax.set_yticks([-4,0,2])
### move the x and y axis to the center
ax.spines['bottom'].set_position(('data',0))
ax.spines['left'].set_position(('data',0))
### set other x-axis invisible
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')
plt.show()

In [ ]:
### sorting the 3 maximum eigvchn_series[:,-1,0].real and 3 minimum
eigvchn_real = eigvchn_series[:,-1,0].real.copy()
### sorting eigvchn_real 
idx = np.argsort(eigvchn_real)
idx_eff = idx[3:-3]
print('effective trials:',idx_eff,len(idx_eff))

In [ ]:
### plot the numerical and theoretical eigenvalues outliers
fig,ax = plt.subplots(figsize=(4,2))
### label is r_{bulk}^{num}
ax.plot(tau_series,radius_theo_map,'gray',linestyle='--',label=r'$r_{bulk}^{theo}$')
ax.plot(tau_series,-radius_theo_map,'gray',linestyle='--')
# ax.plot(tau_series,np.mean(radius_num_map,axis=0),'k',label=r'$r_{bulk}^{num}$')
ax.fill_between(tau_series,np.mean(radius_num_map,axis=0),-np.mean(radius_num_map,axis=0),color='gray',alpha=0.15,label=r'$r_{bulk}^{num}$')
ax.set_xlabel(r'$\tau$')
ax.set_ylabel('Radius')
ax.legend()
plt.show()

### I mean... eigvchn_series[:,:,0].real and eigvchn_series[:,:,1].real compared with eigvchn_theo[:,0] and eigvchn_theo[:,1]
# fig,ax = plt.subplots(figsize=(6,6))
ax.plot(tau_series,eigvchn_theo[:,0],'tab:red',marker='o',label='theoretical')
ax.plot(tau_series,eigvchn_theo[:,1],'tab:green',marker='o')
# ax.plot(tau_series,np.mean(eigvchn_series[:,:,0].real,axis=0),'k',label='numerical')
# ax.plot(tau_series,np.mean(eigvchn_series[:,:,1].real,axis=0),'k')
ax.fill_between(tau_series,np.mean(eigvchn_series[idx_eff,:,0].real,axis=0)-np.std(eigvchn_series[idx_eff,:,0].real,axis=0),np.mean(eigvchn_series[:,:,0].real,axis=0)+np.std(eigvchn_series[idx_eff,:,0].real,axis=0),color='tab:red',alpha=0.15)
ax.fill_between(tau_series,np.mean(eigvchn_series[idx_eff,:,1].real,axis=0)-np.std(eigvchn_series[idx_eff,:,1].real,axis=0),np.mean(eigvchn_series[idx_eff,:,1].real,axis=0)+np.std(eigvchn_series[idx_eff,:,1].real,axis=0),color='tab:green',alpha=0.15)
ax.set_xlabel(r'$\tau$')
ax.set_ylabel('Outliers')
### set x and y lims as well as ticks
dtau = tau_series[1]-tau_series[0] 
ax.set_xlim([tau_series[0]-dtau/8,tau_series[-1]+dtau/8])
ax.set_xticks([0,tau_series[-1]//2,tau_series[-1]])
ax.set_ylim([-4.0,2.5])
ax.set_yticks([-3,0,2])
ax.legend()
### move the x and y axis to the center
ax.spines['bottom'].set_position(('data',0))
ax.spines['left'].set_position(('data',0))
### set other x-axis invisible
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')
plt.show()

In [ ]:
### plot a shifting histogram for the first eigenvalue
fig,ax=plt.subplots(figsize=(6,3))
### for each tau, fit a Gaussian distribution to the first eigenvalue, and plot the Gaussian distribution with the color from light to dark
for it in range(ntau):
    ### fit a Gaussian distribution to the first eigenvalue
    mu, std = norm.fit(eigvchn_series[idx_eff,it,0].real)
    ### plot the Gaussian distribution with the color from light to dark
    # ax.hist(eigvchn_series[:,it,0].real,bins=20,color=plt.cm.YlOrRd(it/ntau),alpha=0.5,density=True,edgecolor=plt.cm.YlOrRd(it/ntau),linewidth=1.2)
    ax.plot(np.linspace(-3,-0.5,100),-norm.pdf(np.linspace(-3,-0.5,100),mu,std)/4.5-0.1,color=plt.cm.YlOrRd(it/ntau),linewidth=1.5,alpha=0.6)
    ### fit a Gaussian distributon to the second eigenvalue
    mu, std = norm.fit(eigvchn_series[idx_eff,it,1].real)
    ### plot the Gaussian distribution with the color from light to dark
    # ax.hist(eigvchn_series[:,it,1].real,bins=20,color=plt.cm.YlGn(it/ntau),alpha=0.5,density=True,edgecolor=plt.cm.YlGn(it/ntau),linewidth=1.2)
    ax.plot(np.linspace(-0.5,2,100),norm.pdf(np.linspace(-0.5,2,100),mu,std)/4.5+0.1,color=plt.cm.YlGn(it/ntau),linewidth=1.5,alpha=0.6)   
ax.set_xlabel(r'$\Re(\lambda)$',fontsize=14)
### set double ylabels
ax.set_ylabel(r'$\Im(\lambda)$',fontsize=14)
ax2 = ax.twinx() 
 
color = 'tab:gray'
ax2.set_ylabel('probability density\n (numerical)', color = color,fontsize=14)
ax2.tick_params(axis ='y', labelcolor = color) 
ax2.set_yticks([])
ax2.spines['bottom'].set_color('none')
### set other x-axis invisible
ax2.spines['top'].set_color('none')
ax2.spines['right'].set_color('none')
ax2.spines['left'].set_color('none')
# ax.set_title('Eigenvalue spectrum',fontsize=14,fontweight='bold')
### refine the plot 
ax.set_xlim([-3.0,2.])
ax.set_ylim([-0.8,0.8])
ax.set_aspect('equal')
ax.spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')
ax.spines['left'].set_position(('data',0))
### set the x and y ticks
ax.set_xticks([-3,0,2])
ax.set_yticks([-0.5,0,0.5])

### plot a circle with the radius of the last radius_num_map
ax.add_patch(plt.Circle((0,0),radius_theo_map[-1],color='tab:blue',fill=False,linewidth=1.5,alpha=1))
### plot a circle with the radius of the first radius_num_map
ax.add_patch(plt.Circle((0,0),radius_theo_map[0],color='tab:purple',fill=False,linewidth=1.5,alpha=0.5,linestyle='--'))

ktrial_index = 15
### scatter plot the eigenvalue bulk of this trial
### randomly select 500 points
idx_random = np.random.choice(np.arange(2,N),size=500,replace=False)
ax.scatter(eigvchn_series[ktrial_index,0,idx_random].real,eigvchn_series[ktrial_index,0,idx_random].imag,color='tab:purple', s=5,alpha=0.15)
ax.scatter(eigvchn_series[ktrial_index,-1,idx_random].real,eigvchn_series[ktrial_index,-1,idx_random].imag,color='tab:blue', s=5,alpha=0.2)
# ### plot the mean of radius_map_num the last one 
# ax.plot(-np.mean(radius_num_map[:-1,-1])*np.ones(2),np.array([-0.5,0.5]),'--',color='black',linewidth=1.5)
# ax.plot(np.mean(radius_num_map[:-1,-1])*np.ones(2),np.array([-0.5,0.5]),'--',color='black',linewidth=1.5)
# ### plot the mean of radius_map_num the first one
# ax.plot(-np.mean(radius_num_map[:-1,0])*np.ones(2),np.array([-0.5,0.5]),'--',color='gray',linewidth=1.5)
# ax.plot(np.mean(radius_num_map[:-1,0])*np.ones(2),np.array([-0.5,0.5]),'--',color='gray',linewidth=1.5)

### also plot the original eigenvalue outlier: JE-JI, using red color and X marker
ax.scatter(JE-JI,0,color='red',marker='X',s=80,alpha=1)


In [ ]:
### scatter plot the predicted eigenvalues eigvchn_theo against the numerical mean np.mean(eigvchn_series[:,:,0],axis=0)
fig, ax = plt.subplots(figsize=(4,4))
### theoretical eigenvlaues eigvchn_theo against the numerical mean np.mean(eigvchn_series,axis=0)
ax.scatter(np.mean(eigvchn_series[idx_eff,:,0],axis=0),eigvchn_theo[:,0],cmap=plt.cm.YlOrRd, c=tau_series, s=50,alpha=1,marker='^')
ax.scatter(np.mean(eigvchn_series[idx_eff,:,1],axis=0),eigvchn_theo[:,1],cmap=plt.cm.YlGn, c=tau_series, s=50,alpha=1,marker='^')
### refine the figure
ax.set_xticks([-3,0,2])
ax.set_yticks([-3,0,2])
ax.set_xlim([-3,2])
ax.set_ylim([-3,2])
### set x-axis and y-axis to zero
ax.set_aspect('equal')
ax.spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')
ax.spines['left'].set_position(('data',0))
ax.plot([-3,3],[-3,3],lw=1.5,color='gray',linestyle='--',alpha=0.5)
ax.set_xlabel('mean of numerical')
ax.set_ylabel('theoretical prediction')

In [ ]:
### validate that eigenvalue_1 - (JE-JI) = -(eigenvalue_2-0)
### using scatter plot, the x-axis is (eigenvalue_1 - (JE-JI)), the y-axis is -(eigenvalue_2-0)
fig,ax=plt.subplots(figsize=(4,4))
for ktrial in range(trials):
    im=ax.scatter(eigvchn_series[ktrial,:,0].real-(JE-JI),-eigvchn_series[ktrial,:,1].real,cmap='viridis',c=tau_series,s=10)
# ,c=tau_series/tau_series.max(),s=10)
ax.plot(np.linspace(-2,2,10),np.linspace(-2,2,10),color='gray',linewidth=1.5,linestyle='--')
ax.set_xlabel(r'$\lambda_1-\lambda_0$',fontsize=14)
ax.set_ylabel(r'$-\lambda_2$',fontsize=14)
ax.set_aspect('equal')
ax.set_xlim(-3,1.5)
ax.set_ylim(-3,1.5)
ax.set_xticks([-2,0,1])
ax.set_yticks([-2,0,1])
### move the x-axis and y-axis to the center
ax.spines['left'].set_position(('data',0))
ax.spines['bottom'].set_position(('data',0))
### hide the top and right spines
ax.spines['right'].set_color('none')
ax.spines['top'].set_color('none')

### add colorbar 
cbar=plt.colorbar(im,ax=ax)
cbar.set_ticks([0,0.2,0.4])
cbar.set_ticklabels([0,0.2,0.4])
cbar.set_label(r'$\tau$',fontsize=14)
plt.show()

#### Compare the reconstructed one with the numerical one

Scatter ploting the random components on the left and right eigenvectors, showing that the random component are positively/negatively correlated

In [ ]:
### plot random components on left and right eigenvectors when tau == 0.02 and ktrial = 15
ktrial = idx_eff[6]
its = [int((ntau-1)*2/3),ntau-1]
fig,ax=plt.subplots(1,2,figsize=(10,4),sharex=True,sharey=True)
for i,it in enumerate(its):
    ax[i].scatter(eigrvec_series[ktrial,it,:NE,0]*np.sqrt(N),eigrvec_series[ktrial,it,:NE,1]*np.sqrt(N),color='tab:red',s=10,label='right eigenvector '+r'$E$',alpha=0.25)
    ax[i].scatter(eigrvec_series[ktrial,it,NE:,0]*np.sqrt(N),eigrvec_series[ktrial,it,NE:,1]*np.sqrt(N),color='tab:blue',s=10,label='right eigenvector '+r'$I$',alpha=0.25)
    ### equal axis
    ax[i].set_aspect('equal')
    ax[i].set_xlim(-0.15,0.15)
    ax[i].set_ylim(-0.15,0.15)
    ax[i].set_xlim(-2.5,2.5)
    ax[i].set_ylim(-2.5,2.5)
    ax[i].set_xlabel('Random component rank 1',fontsize=14)
    ax[i].set_ylabel('Random component rank 2',fontsize=14)
    ax[i].legend()
    ax[i].set_title('Right eigenvector '+r'$\tau=$'+str(tau_series[it]),fontsize=14)
    ax[i].set_xticks([-0.1,0,0.1])
    ax[i].set_yticks([-0.1,0,0.1])
    ax[i].set_xticks([-2,0,2])
    ax[i].set_yticks([-2,0,2])
    ### remove the top and right axes
    ax[i].spines['top'].set_color('none')
    ax[i].spines['right'].set_color('none')
    ### move the x-axis and y-axis to the center
    ax[i].spines['bottom'].set_position(('data',0))
    ax[i].spines['left'].set_position(('data',0))
fig.tight_layout()


#### Save all variables and parameters

In [ ]:
def list_to_dict(lst, string):
    """
    Transform a list of variables into a dictionary.
    Parameters
    ----------
    lst : list
        list with all variables.
    string : str
        string containing the names, separated by commas.
    Returns
    -------
    d : dict
        dictionary with items in which the keys and the values are specified
        in string and lst values respectively.
    """
    string = string[0]
    string = string.replace(']', '')
    string = string.replace('[', '')
    string = string.replace('\\', '')
    string = string.replace(' ', '')
    string = string.replace('\t', '')
    string = string.replace('\n', '')
    string = string.split(',')
    d = {s: v for s, v in zip(string, lst)}
    return d

In [ ]:
params = {'g':g,
          'gamma':gamma,
          'NE':NE,
          'NI':NI,
          'J':J,
          'ce':ce,
          'ci':ci,
          'tau_series':tau_series,
          }

### Dynamics

#### Helper function 

In [ ]:
import scipy
from functools import partial
def odeIntegral(x,t,J,I=0):
    x = np.squeeze(x)
    x = np.reshape(x,(len(x),1))
    # dxdt = -x+J@np.tanh(x)#+I[0]
    dxdt = -x+J@x+I[0]
    return np.squeeze(dxdt)
def odesimulation(t,xinit,Jpt,I):
	return scipy.integrate.odeint(partial(odeIntegral,J=Jpt,I=I),xinit,t)
shiftx = 1.5
def odeIntegralP(x,t,J,I=0):
	x = np.squeeze(x)
	x = np.reshape(x,(len(x),1))
	# print('size:',np.shape(x),np.shape(J@np.tanh(x)))
	dxdt = -x+J@(1.0+np.tanh(x-shiftx))
	return np.squeeze(dxdt)
def odesimulationP(t,xinit,Jpt,I):
	return scipy.integrate.odeint(partial(odeIntegralP,J=Jpt,I=I),xinit,t)

Uniform input

In [ ]:
### define the network parameters of the adjacency matrix
nn = [50,100,150,250,750,1200]#800]
g, gamma = 6.0, 1/4.0#1.5,1/1.#
NE = nn[-1]
NI = int(gamma*NE)
N  = NE+NI
ALPHAE, ALPHAI = NE/N, NI/N
c = 0.2 ### sparsity, identical for excitatory and inhibitory neuron populations
KE, KI = int(c*NE), int(c*NI) ### fixed out-degree
J = 1/np.sqrt(N)*0.5  ### TODO: make sure this scalar with David&Stefano's paper
print('number of connected E/I neurons:',KE,KI)
print('non-zero J:',J)
ji,je = g*J,J 

### define the network parameters of the diluted Gaussian matrix 
ge, gi = np.sqrt(je**2*c*(1-c)*N), np.sqrt(ji**2*c*(1-c)*N) 
hat_sigmae, hat_sigmai = np.sqrt(c*(1-c)), np.sqrt(c*(1-c))### standard deviation of the adjacency matrix
sigmae,sigmai = np.sqrt(c*(1-c)*J**2*N), np.sqrt(c*(1-c)*(-g*J)**2*N)### with magnitude of the coupling
JE,JI = je*c*NE, ji*c*NI 
lambda0 = JE-JI 
print('lambda0:',lambda0)
print('ge,gi:',ge,gi)
print('JE,JI:',JE,JI)

ntau = 10#21#
trials = 30+6
tau_series = np.linspace(0,0.225,ntau)# np.linspace(0.25,0.4,ntau)#
## arrays to store results
## norml0_series: norm of left eigenvector(deltaliri = 1)
eigvchn_series, eigrvec_series, eiglvec_series = np.zeros((trials,ntau,N),dtype=complex), np.zeros((trials,ntau,N,2)), np.zeros((trials,ntau,N,2))
eiglvec0_series, norml0_series = np.zeros((trials,ntau,N,2)), np.zeros((trials,ntau+1,N)) 
### also have the reconstructed left and right eigenvectors 
eigrvec_series_rec, eiglvec_series_rec = np.zeros((trials,ntau,N,2)), np.zeros((trials,ntau,N,2))
htau = tau_series[1]-tau_series[0]
### simulation using the low-rank framework
firing_rateeq = np.zeros((trials,ntau,N))
### recording dynamics
lowrank_eq, lowrank_eq_num = np.zeros((trials,ntau,2)), np.zeros((trials,ntau,N))
ovs_inplr, ovs_inplr_num   = np.zeros((trials,ntau,2)), np.zeros((trials,ntau,2))
ovs_inplr_div, ovs_inplr_div_num = np.zeros((trials,ntau,2)), np.zeros((trials,ntau,2))
contributions_lr, contributions_lr_num = np.zeros((trials,ntau,2,2)), np.zeros((trials,ntau,2,2))### rank, population

## numerical 
intg_ov_series    = np.zeros((trials,ntau,2))
first_perturb_ov  = np.zeros((trials,ntau,2,2))
first_perturb_ovP = np.zeros((trials,ntau,2,2,2)) ### rank and population

intg_mean_series  = np.zeros((trials,ntau,N,2,2))### rank2 and population2
intg_std_series   = np.zeros((trials,ntau,2,2))### rank, population
intg_std_num_series = np.zeros((trials,ntau,2,2))### rank, population
mean_shift_ov     = np.zeros((trials,ntau,2))

latent_kappa_series = np.zeros((trials,ntau,2,2))
intg_crossov_series = np.zeros((trials,ntau,2,2)) # rank, rank
intg_crossovPop_series = np.zeros((trials,ntau,2,2,2)) # rank, rank, population

NE = int(N/(1+gamma))
NI = N-NE#NE*gamma
N  = NE+NI ### update 
ALPHAE, ALPHAI = NE/N, NI/N

ce, ci = c,c
print('ce and ci:',ce,ci)
### assert that the differences between ce and ci are smaller than epsilon
epsilon = 1E-2
assert np.abs(ce-ci)<epsilon
# assert ce==ci
c = ce

### mean connectivity
nvec, mvec = np.zeros((N,1)), np.ones((N,1))
nvec[:NE,0], nvec[NE:,0] = N*JE/NE, -N*JI/NI
Jbar = mvec@nvec.T/N 
## TEST THE EIGENVALUES OF THE MEAN MATRIX 
eigvJ0, eigvecJ0 = la.eig(Jbar)
print('eigvJ0:',eigvJ0[0],' theory:',JE-JI)
### mean left and right eigenvectors
leigvec0, reigvec0 = np.zeros((N,N)), np.zeros((N,N))
norm_left = np.zeros(2)
## first eigenvector
leigvec0[:,0], reigvec0[:,0] = nvec[:,0]/(JE-JI)/np.sqrt(N), mvec[:,0]/np.sqrt(N)
norm_left[0]  = la.norm(leigvec0[:,0])
leigvec0[:,0] = leigvec0[:,0]/norm_left[0]
norml0_series[:,0,0] = np.sum(leigvec0[:,0]*reigvec0[:,0])
## second eigenvector
kk = np.sqrt(NE*JI**2+NI*JE**2)
reigvec0[:NE,1], reigvec0[NE:,1] = JI/kk,JE/kk 
leigvec0[:NE,1], leigvec0[NE:,1] = -kk/(JE-JI)/NE,kk/(JE-JI)/NI 
norm_left[1]     = la.norm(leigvec0[:,1])
leigvec0[:,1] = leigvec0[:,1]/norm_left[1]
norml0_series[:,0,1] = np.sum(leigvec0[:,1]*reigvec0[:,1])
outerproduct  = np.sum(leigvec0[:,0]*reigvec0[:,1])#*norm_left

### compute the square of the random connectivity 
Z2E = N*J**2*hat_sigmae**2*tau_series*ALPHAE-N*g*J**2*hat_sigmae*hat_sigmai*tau_series*ALPHAI
Z2I = -N*g*J**2*hat_sigmae*hat_sigmai*tau_series*ALPHAE+N*g**2*J**2*hat_sigmai**2*tau_series*ALPHAI
DeltaZ2E, DeltaZ2I = Z2E[1]-Z2E[0],Z2I[1]-Z2I[0] ### for practical use
print('DeltaZ2E,DeltaZ2I:',DeltaZ2E,DeltaZ2I)
DeltaZ2 = np.zeros((N,N))
DeltaZ2[:,:NE], DeltaZ2[:,NE:] = DeltaZ2E, DeltaZ2I

### E population and I population separately
Z2E_E, Z2I_E = N*J**2*hat_sigmae**2*tau_series*ALPHAE, -N*g*J**2*hat_sigmae*hat_sigmai*tau_series*ALPHAI
Z2E_I, Z2I_I = -N*g*J**2*hat_sigmae*hat_sigmai*tau_series*ALPHAE, N*g**2*J**2*hat_sigmai**2*tau_series*ALPHAI
DeltaZ2E_E, DeltaZ2I_E = Z2E_E[1]-Z2E_E[0],Z2I_E[1]-Z2I_E[0] ### for practical use
DeltaZ2E_I, DeltaZ2I_I = Z2E_I[1]-Z2E_I[0],Z2I_I[1]-Z2I_I[0] ### for practical use
DeltaZ2_E, DeltaZ2_I = np.zeros((N,N)),np.zeros((N,N))
DeltaZ2_E[:,:NE], DeltaZ2_E[:,NE:] = DeltaZ2E_E, DeltaZ2I_E

In [ ]:
ce, ci = c,c
for it, tau in enumerate(tau_series):
    ### mean connectivity 
    je, ji = J, g*J
    hat_sigmae, hat_sigmai = np.sqrt(ce*(1-ce)),np.sqrt(ci*(1-ci))
    sigmae, sigmai = np.sqrt(je**2*ce*(1-ce)*N), np.sqrt(ji**2*ci*(1-ci)*N) 
    ### generate tau_div_vec, tau_con_vec, tau_rec_vec, tau_chn_vec
    tau_div_vec, tau_con_vec, tau_rec_vec, tau_chn_vec = np.zeros(2),np.zeros((2,2)),np.zeros((2,2)),np.zeros((2,2))
    ### YS: alltaus do not have E-I polarity
    tau_div_vec = np.array([tau,tau])
    tau_con_vec = np.array([[tau,tau],[tau,tau]])
    # tau_rec_vec = np.array([[0,0],[0,0]])
    tau_rec_vec = np.array([[2*tau,2*tau],[2*tau,2*tau]])
    tau_chn_vec = np.array([[tau,tau],[tau,tau]])
    # gamma = 1.0 ## NI/NE
    sigma = np.sqrt(sigmae**2/(1+gamma)+sigmai**2*gamma/(1+gamma))
    ### compute the radius 
    # compute a single number for multiple populations
    tau_div_num = hat_sigmae**2*tau_div_vec[0]+gamma*g**2*hat_sigmai**2*tau_div_vec[1]
    tau_div_den = hat_sigmae**2+gamma*g**2*hat_sigmai**2
    tau_div = tau_div_num/tau_div_den

    tau_con_num = hat_sigmae**2*tau_con_vec[0,0]+gamma*g**2*hat_sigmai**2*tau_con_vec[1,1]
    tau_con_den = hat_sigmae**2+gamma*g**2*hat_sigmai**2
    tau_con  = tau_con_num/tau_con_den

    tau_rec_num = hat_sigmae**2*tau_rec_vec[0,0] - (1+gamma)*g*hat_sigmae*hat_sigmai*tau_rec_vec[0,1]+gamma*g**2*hat_sigmai**2*tau_rec_vec[1,1]
    tau_rec_den = hat_sigmae**2+gamma*g**2*hat_sigmai**2
    tau_rec = tau_rec_num/tau_rec_den/2.0 

    tau_chn_num = hat_sigmae**2*tau_chn_vec[0,0] - 0.5*(1+gamma)*g*hat_sigmae*hat_sigmai*(tau_chn_vec[0,1]+tau_chn_vec[1,0])+gamma*g**2*hat_sigmai**2*tau_chn_vec[1,1]
    tau_chn_den = hat_sigmae**2+gamma*g**2*hat_sigmai**2
    tau_chn = tau_chn_num/tau_chn_den/2.0

    radius_theo = (1.0-tau_con-tau_div+tau_rec-2*tau_chn)/np.sqrt(1-tau_con-tau_div)*sigma
    
    radius_theo_map[it] = radius_theo
            
### theoretical prediction of the eigenvalues

org_k = J*N*(ALPHAE-g*ALPHAI)/2.0

eigvchn_theo[:,0] = org_k*(c+np.sqrt(c**2+4*c*(1-c)*tau_series))
eigvchn_theo[:,1] = org_k*(c-np.sqrt(c**2+4*c*(1-c)*tau_series))

print('theoretical eigenvalues:\n',eigvchn_theo)

In [ ]:
#### constant and deterministic input signal
Inp   = np.squeeze(np.ones((N,1)))/np.sqrt(N) 
tt = np.linspace(0,100,1000)
#### random and structural input signal 
### simulation using the mean network connectivity (inhibition-dominated)
Jpt   = Jbar.copy()
xinit = np.squeeze(np.random.normal(0, 1E-2, (1, N)))
xtemporal = odesimulation(tt, xinit, Jpt, Inp)
firing_rate0 = np.reshape(xtemporal[-1,:N],(N,1))

In [ ]:
#### constant and deterministic input signal
Inp   = np.squeeze(np.ones((N,1)))/np.sqrt(N) 
tt = np.linspace(0,100,1000)
#### random and structural input signal 
### simulation using the mean network connectivity (inhibition-dominated)
Jpt   = Jbar.copy()
xinit = np.squeeze(np.random.normal(0, 1E-2, (1, N)))
xtemporal = odesimulation(tt, xinit, Jpt, Inp)
firing_rate0 = np.reshape(xtemporal[-1,:N],(N,1))

### generate i.i.d. s
def randbin(M,N,P):  
    return np.random.choice([0, 1], size=(M,N), p=[P, 1-P])

sparseiid_dyns = np.zeros((trials,N,1))
for ktrial in range(trials):
    ### generate iid sparse connectivity 
    Ecol = randbin(1,NE*N,1-ce)
    Icol = randbin(1,NI*N,1-ci)
    Ecol = np.reshape(Ecol,(N,NE))
    Icol = np.reshape(Icol,(N,NI))
    iidsparse = np.zeros((N,N))
    iidsparse[:,:NE], iidsparse[:,NE:] = Ecol.copy()*J,Icol.copy()*J*(-g)
    xinit = np.squeeze(np.random.normal(0, 1E-2, (1, N)))
    xtemporal = odesimulation(tt, xinit, iidsparse, Inp)
    firing_rate0 = np.reshape(xtemporal[-1,:N],(N,1))
    sparseiid_dyns[ktrial,:,:] = firing_rate0.copy()
    
sparseiid_dyns = np.squeeze(sparseiid_dyns)
mean_sparseiid_dyns = np.zeros((trials,2))
for ktrial in range(trials):
    mean_sparseiid_dyns[ktrial,0],mean_sparseiid_dyns[ktrial,1]= np.mean(sparseiid_dyns[ktrial,:NE]),np.mean(sparseiid_dyns[ktrial,NE:])
trialavg_sparseiid_dyns = np.mean(mean_sparseiid_dyns,axis=0)
trialstd_sparseiid_dyns = np.std(mean_sparseiid_dyns,axis=0)
print(trialavg_sparseiid_dyns)

In [ ]:
### if you want to re-run the simulations for dynamics, set rerun_dyns to be True 
rerun_dyns = True
eiglvec0norm_series= np.zeros((trials,ntau,N,2))
leig0mean_series = np.zeros((trials,ntau,N,2))
leig0pre_series = np.zeros((trials,ntau,N,2))

norm_4rvec_series, norm_4lvec_series = np.zeros((trials,ntau,2)),np.zeros((trials,ntau,2))
norm_4lvec_series_ = np.zeros((trials,ntau,2))

if rerun_dyns == True:
    switch = 1
    Inp   = np.squeeze(np.ones((N,1)))/np.sqrt(N) 
    flag_run  = True
    for ktrial in range(trials):
        print('trial>>>>>>>>>>>>>>>>>>>>>>>>>',ktrial)
        while (flag_run):
            xr      = iidGaussian([0,1/np.sqrt(N)],[N,N])
            chneta  = iidGaussian([0,1/np.sqrt(N)],[N,6])
            xrec    = iidGaussian([0,1/np.sqrt(N)],[N,N])
            ### zscore
            xr   = stats.zscore(xr.flatten())
            xr   = xr*1/np.sqrt(N)
            xr   = np.reshape(xr,(N,N))
            ### zscore
            xrec = stats.zscore(xrec.flatten())
            xrec = xrec*1/np.sqrt(N)
            xrec = np.reshape(xrec,(N,N))
            # ### zscore
            # nsample = 20
            chneta  = iidGaussian([0,1/np.sqrt(N)],[N,6])
            chneta[:,0] = stats.zscore(chneta[:,0])
            chneta[:,0] *=(1/np.sqrt(N))
            
            tau = tau_series[-1]
            a    = np.sqrt(tau)
            zrow = a*np.repeat(np.reshape(chneta[:,0],(1,-1)),N,axis=0)
            zcol = a*np.repeat(np.reshape(chneta[:,0],(-1,1)),N,axis=1)
            gammarec = a*xrec-a*xrec.copy().T
            ### adjacency matrix
            zr   = zrow.copy()+zcol.copy()+np.sqrt(1-2*tau)*xr### without considering the reciprocal term
            ### E-I matrix 
            zr[:,:NE],zr[:,NE:] = zr[:,:NE]*ge,zr[:,NE:]*(-gi)        
            ### generate J connectivity matrix
            Jchn = Jbar.copy()+zr.copy()           
            eigvchn, eigrvec = la.eig(Jchn)
            FIRSTV = np.abs(eigvchn[0]-eigvchn_theo[-1,0])/np.abs(eigvchn_theo[-1,0])
            SECONDV = np.abs(eigvchn[1]-eigvchn_theo[-1,1])/np.abs(eigvchn_theo[-1,1])
            if FIRSTV<0.1 and SECONDV<0.1 and eigvchn[0].imag==0 and eigvchn[1].imag==0:
                flag_run = False
                # print('eigvchn:',eigvchn,'theory:',eigvchn_theo[-1,:])
                # print('eigvchn:',eigvchn,'theory:',eigvchn_theo[-1,:])    
            
        ### ---------------------
        print('Go run...........................')
        intg_ov  = np.zeros(2) ### rank
        intg_ovP = np.zeros((2,2,2)) ### rank, rank, population
        intg_ml, intg_mr = leigvec0.copy(), reigvec0.copy()
        mean_pre = np.array([JE-JI,0])
        mean_total_change = np.zeros(2)
        z_pre = np.zeros((N,N))
        for it, tau in enumerate(tau_series):
            a    = np.sqrt(tau)
            zrow = a*np.repeat(np.reshape(chneta[:,0],(1,-1)),N,axis=0)
            zcol = a*np.repeat(np.reshape(chneta[:,0],(-1,1)),N,axis=1)
            gammarec = a*xrec-a*xrec.copy().T
            ### adjacency matrix
            zr   = zrow.copy()+zcol.copy()+np.sqrt(1-2*tau)*xr### without considering the reciprocal term
            ### E-I matrix 
            zr[:,:NE],zr[:,NE:] = zr[:,:NE]*ge,zr[:,NE:]*(-gi)        
            
            ha    = np.sqrt(htau)
            hzrow = ha*np.repeat(np.reshape(chneta[:,0],(1,-1)),N,axis=0)
            hzcol = ha*np.repeat(np.reshape(chneta[:,0],(-1,1)),N,axis=1)
            hgammarec = ha*xrec-ha*xrec.copy().T
            # hzr   = hzrow.copy()+hzcol.copy()+np.sqrt(1-4*htau)*xr+hgammarec
            hzr   = hzrow.copy()+hzcol.copy()+np.sqrt(1-2*htau)*xr### without considering the reciprocal term
            hzr[:,:NE],hzr[:,NE:] = hzr[:,:NE]*ge,hzr[:,NE:]*(-gi)
            DELTA_Z = zr-z_pre
            hzr     = DELTA_Z.copy()

            ### generate J connectivity matrix
            Jchn = Jbar.copy()+zr.copy()
            ### full rank simulation
            xinit = np.squeeze(np.random.normal(0, 1E-2, (1, N)))
            xc_temporal = odesimulation(tt, xinit, Jchn, Inp)
            firing_rateeq[ktrial,it,:] = xc_temporal[-1,:].copy()
            
            eigvchn, eigrvec = la.eig(Jchn)
            eigvchn_,eiglvec = la.eig(Jchn.copy().T)
            ### normalization
            reig  = np.squeeze(eigrvec[:,:].copy())
            leig0 = np.squeeze(eiglvec[:,:].copy()) 
            normval = np.sum(reig.copy()*leig0.copy(),axis=0)
            norml0_series[ktrial,it+1,:] = normval.copy() ### normalization factor shift right 1byte
            normval = np.repeat(np.reshape(normval,(1,N)),N,axis=0)
            leig    = leig0.copy()/normval.copy() ### left eigenvector normalization
            if np.mean(reig[:NE,0])*np.mean(reigvec0[:NE,0])<0:
                reig[:,0]*=-1
                leig[:,0]*=-1
                leig0[:,0]*=-1
            if np.mean(reig[:NE,1])<0: ### the second rank-1 component is negative
                reig[:,1]*=-1
                leig[:,1]*=-1
                leig0[:,1]*=-1
                
            ### numerical low-rank approximation 
            ov_inp_lowrank,ov_inp_lowrank_div = np.zeros(2),np.zeros(2)
            vec_lowrank_contribution = np.zeros((N,2))
            ### linear response theory approximation 
            for i in range(2):
                ov_inp_lowrank[i]=np.sum(leig[:,i]*Inp[:])*eigvchn[i]   
                ov_inp_lowrank_div[i]= ov_inp_lowrank[i]/(1-eigvchn[i])
            Equilibrium_lowrank_outliers = np.reshape(Inp.copy(),(N,1))
            # print('EQ shape:',np.shape(Equilibrium_lowrank_outliers))
            for i in range(2):
                vec_lowrank_contribution[:,i] = ov_inp_lowrank_div[i]*reig[:,i]
                # print('shape :...',np.shape(np.reshape(vec_lowrank_contribution[:,i].copy(),(N,1))))
                Equilibrium_lowrank_outliers += np.reshape(vec_lowrank_contribution[:,i].copy(),(N,1))
                #### REDUCE TO 2 POPULATION, THEREFORE RANK, POPULATION
                contributions_lr_num[ktrial,it,i,0] = np.mean(vec_lowrank_contribution[:NE,i])
                contributions_lr_num[ktrial,it,i,1] = np.mean(vec_lowrank_contribution[NE:,i])
            lowrank_eq_num[ktrial,it,:] = np.squeeze(Equilibrium_lowrank_outliers.copy())
            ovs_inplr_num[ktrial,it,:],ovs_inplr_div_num[ktrial,it,:] = ov_inp_lowrank.copy(),ov_inp_lowrank_div.copy() 
                
            ### numerical std 
            intg_std_num_series[ktrial,it,0,0] = np.std(reig[:NE,0])
            intg_std_num_series[ktrial,it,0,1] = np.std(reig[NE:,0])
            intg_std_num_series[ktrial,it,1,0] = np.std(reig[:NE,1])
            intg_std_num_series[ktrial,it,1,1] = np.std(reig[NE:,1])       
            
            
            eigvchn_series[ktrial,it,:]    = eigvchn.copy()#eigvw_norm.copy()#
            eigrvec_series[ktrial,it,:,:]  = reig[:,:2].copy()#eigvecw_norm.copy()#
            eiglvec_series[ktrial,it,:,:]  = leig[:,:2].copy()#eigvect_norm.copy()#
            eiglvec0_series[ktrial,it,:,:] = leig0[:,:2].copy()#eigvect_norm.copy()#
            for iii in range(2):
                eiglvec0norm_series[ktrial,it,:,iii] = leig0[:,iii].copy()/normval[iii,iii]
            DeltaZ2 = hzr@hzr    ### used to correct      
                
            if it<1:
                lvec, rvec = np.squeeze(leigvec0[:,:2]),np.squeeze(reigvec0[:,:2])
                # eigeng     = np.squeeze(eigvchn_series[ktrial,it,:2].copy())
                eigeng     = np.array([JE-JI,0])
                lvec_mean, rvec_mean = lvec.copy(), rvec.copy()
            else:
                lvec, rvec = np.squeeze(eiglvec0_series[ktrial,it-1,:,:2]),np.squeeze(eigrvec_series[ktrial,it-1,:,:2]) ### use the previous eigenvector as the initial condition   
                eigeng = np.squeeze(eigvchn_series[ktrial,it-1,:2].copy())
                ### conditioned mean  
                lvec_mean, rvec_mean = lvec.copy(), rvec.copy()
                lvec_mean[:NE,:],lvec_mean[NE:,:]=np.mean(lvec_mean[:NE,:],axis=0),np.mean(lvec_mean[NE:,:],axis=0)
                rvec_mean[:NE,:],rvec_mean[NE:,:]=np.mean(rvec_mean[:NE,:],axis=0),np.mean(rvec_mean[NE:,:],axis=0)
                
            ### get the appropriate normalization factor
            norm_for_lvec, norm_for_rvec = np.zeros(2),np.zeros(2)
            if it==0:
                hzr_u = xr.copy()
                with_chn = 0
            else:
                hzr_u = hzr.copy()
                with_chn = 1
            
            intg_ov[0] = first_perturb_ov[ktrial,it-1,0,0]#intg_ov_series[ktrial,it-1,0]#
            intg_ov[1] = first_perturb_ov[ktrial,it-1,1,1] #intg_ov_series[ktrial,it-1,1]#
            intg_ovP   = first_perturb_ovP[ktrial,it-1,:,:,:]#intg_ov_series[ktrial,it-1,1]#
            
            norm_rvec_temp, norm_lvec_temp = np.zeros((N,2)), np.zeros((N,2))
            for i in range(2):
                rvec_n = np.reshape(rvec[:,i],(-1,1)) + (hzr_u)@np.reshape(rvec[:,i].copy(),(-1,1))/np.real(eigeng[i])
                lvec_n = np.reshape(lvec[:,i],(-1,1)) + (hzr_u).T@np.reshape(lvec[:,i].copy(),(-1,1))/np.real(eigeng[i])
                
                ''' Mean connectivity '''                
                current_eigv = eigvchn_series[ktrial,it,i].copy()
                if it==0:
                    intg_ml[:NE,i],intg_ml[NE:,i] = np.mean(leig[:NE,i])*current_eigv, np.mean(leig[NE:,i])*current_eigv
                    intg_mr[:NE,i],intg_mr[NE:,i] = np.mean(reig[:NE,i]), np.mean(reig[NE:,i])
                    leig0mean_series[ktrial,it,:,i] =leig0[:,i]/norml0_series[ktrial,it+1,i]
                    ''' otherwise no correlation can be calculated '''
                    norm_rvec_temp[:,i] = np.squeeze(reig[:,i])
                    norm_lvec_temp[:,i] = np.squeeze(leig[:,i])*current_eigv
                elif it < 7 and i==1:### 7 is 2 blocks after radius<outlier
                    ### original values are obtained numerically
                    intg_ml[:NE,i],intg_ml[NE:,i] = np.mean(leig[:NE,i])*current_eigv, np.mean(leig[NE:,i])*current_eigv
                    intg_mr[:NE,i],intg_mr[NE:,i] = np.mean(reig[:NE,i]), np.mean(reig[NE:,i])
                    leig0mean_series[ktrial,it,:,i] =leig0[:,i]/norml0_series[ktrial,it+1,i]
                    ''' otherwise no correlation can be calculated '''
                    norm_rvec_temp[:,i] = np.squeeze(reig[:,i])
                    norm_lvec_temp[:,i] = np.squeeze(leig[:,i])*current_eigv
                else:
                    eigenvalue_u = np.real(current_eigv)
                    eigenvalue_um = np.real(eigeng[i])
                    
                    ### norm_for_rvec and norm_for_lvec 
                    rmean_tmp =np.reshape(rvec_mean[:,i].copy(),(-1,1)) + np.reshape(with_chn*(DeltaZ2@np.reshape(rvec_mean[:,i],(-1,1)))/np.real(eigenvalue_um)**2,(N,1))### mean-shifting 
                    rvec_n[:NE,0] = rvec_n[:NE,0] - np.mean(rvec_n[:NE,0])+rmean_tmp[:NE,0]
                    rvec_n[NE:,0] = rvec_n[NE:,0] - np.mean(rvec_n[NE:,0])+rmean_tmp[NE:,0]

                    lmean_tmp = np.reshape(lvec_mean[:,i].copy(),(-1,1)) + np.reshape(with_chn*np.reshape(lvec_mean[:,i],(1,-1))@DeltaZ2/np.real(eigeng[i])**2,(N,1))### mean-shifting 
                    lvec_n[:NE,0] = lvec_n[:NE,0] - np.mean(lvec_n[:NE,0])+lmean_tmp[:NE,0]
                    lvec_n[NE:,0] = lvec_n[NE:,0] - np.mean(lvec_n[NE:,0])+lmean_tmp[NE:,0]
                    
                    norm_for_rvec[i] = la.norm(rvec_n) ### normalization factor 
                    norm_rvec_n = np.reshape(rvec_n.copy(),(-1,1))/norm_for_rvec[i] 
                    norm_rvec_temp[:,i]=np.squeeze(norm_rvec_n.copy())
                    
                    norm_for_lvec[i] = np.squeeze(np.reshape(lvec_n,(1,-1))@np.reshape(norm_rvec_temp[:,i],(-1,1)))      
                    norm_lvec_n = np.reshape(lvec_n.copy(),(-1,1))/norm_for_lvec[i] 
                    ### need to be re-normalized
                    norm_lvec_temp[:,i]  = np.squeeze(norm_lvec_n.copy())
                    norm_lvec_temp[:,i] *= eigvchn_series[ktrial,it,i].real
                    
                    ### more simplified version using lvec_mean and rvec_mean
                    intg_ml[:,i] = lmean_tmp[:,0]/norm_for_lvec[i]*eigenvalue_u
                    leig0mean_series[ktrial,it,:,i] =lmean_tmp[:,0]/norm_for_lvec[i]
                    leig0pre_series[ktrial,it,:,i] =np.reshape(with_chn*np.reshape(lvec_mean[:,i],(1,-1))@DeltaZ2/np.real(eigeng[i])**2,(N,1))[:,0]#*eigenvalue_u#
                    intg_mr[:,i] = rmean_tmp[:,0]/norm_for_rvec[i]
                    
                    norm_4lvec_series[ktrial,it,i],norm_4rvec_series[ktrial,it,i] = norm_for_lvec[i],norm_for_rvec[i]
                    tilden = np.reshape(lvec_n.copy(),(-1,1))/norml0_series[ktrial,it,i]
                    norm_4lvec_series_[ktrial,it,i]=np.reshape(tilden,(1,-1))@np.reshape(rvec_n.copy(),(-1,1))/norm_for_rvec[i]
                    if i==0:
                        print(i,'should be the same',norm_4lvec_series[ktrial,it,i]/norml0_series[ktrial,it,i],norm_4lvec_series_[ktrial,it,i])
                    
                
                ### @YS 17 Nov, no matter what the variable it is.
                ### modify the mean of the elements on the left and right eigenvectors
                norm_rvec_temp[:NE,i] -= np.mean(norm_rvec_temp[:NE,i])
                intg_std_series[:NE,i] = np.std(norm_rvec_temp[:NE,i]) ### kinda theoretical solutions based on reconstructed eigenvectors
                norm_rvec_temp[NE:,i] -= np.mean(norm_rvec_temp[NE:,i])
                intg_std_series[NE:,i] = np.std(norm_rvec_temp[NE:,i])
                norm_rvec_temp[:NE,i] += np.mean(intg_mr[:NE,i])
                norm_rvec_temp[NE:,i] += np.mean(intg_mr[NE:,i])
                norm_lvec_temp[:NE,i] -= np.mean(norm_lvec_temp[:NE,i])
                norm_lvec_temp[NE:,i] -= np.mean(norm_lvec_temp[NE:,i])
                norm_lvec_temp[:NE,i] += np.mean(intg_ml[:NE,i])
                norm_lvec_temp[NE:,i] += np.mean(intg_ml[NE:,i])
                    
                ### record the reconstructed eigenvectors
                eigrvec_series_rec[ktrial,it,:,i] = norm_rvec_temp[:,i].copy()
                eiglvec_series_rec[ktrial,it,:,i] = norm_lvec_temp[:,i].copy()
                ### record the mean of the elements on the left and right eigenvectors  
                intg_mean_series[ktrial,it,:,i,0] = np.reshape(intg_ml[:,i],(N,))
                intg_mean_series[ktrial,it,:,i,1] = np.reshape(intg_mr[:,i],(N,))
                
                ''' Overall overlap '''
                if it>0:
                    if it<=switch:
                        eigenvalues_use = eigvchn_series[ktrial,it-1,:2].copy()
                        eigenvalues_usem = eigvchn_series[ktrial,it,:2].copy()
                    else:
                        eigenvalues_use = eigvchn_series[ktrial,it,:2].copy()
                        eigenvalues_usem = eigvchn_series[ktrial,it-1,:2].copy()
                    ### simplified DeltaZ2 and mean 
                    intg_ov[i] = intg_ov[i] + with_chn*lvec_mean[:,i].T@DeltaZ2@rvec_mean[:,i]/np.real(eigeng[i])/np.real(eigenvalues_usem[i])/norm_for_lvec[i]/norm_for_rvec[i]*np.real(eigenvalues_use[i])     
                else:
                    intg_ov[i] = lvec_mean[:,i].T@(hzr_u@hzr_u)@rvec_mean[:,i]/np.real(eigeng[i])**2/norm_for_lvec[i]/norm_for_rvec[i]*eigvchn_series[ktrial,it,i].real
                intg_ov_series[ktrial,it,i] = np.squeeze(intg_ov[i]).real
            '''overlap per population '''
            ### theoretically compute the equilibrium population-averaged firing rate
            an = np.zeros((2,2),dtype=complex)
            am = np.zeros((2,2),dtype=complex) ## population X rank    
            for ir in range(2):
                an[0,ir] = np.mean(intg_ml[:NE,ir])
                an[1,ir] = np.mean(intg_ml[NE:,ir])
                am[0,ir] = np.mean(intg_mr[:NE,ir])
                am[1,ir] = np.mean(intg_mr[NE:,ir])
                
            ### overlap sum
            overlap_inp = np.zeros(2,dtype=complex) 
            for ir in range(2):
                overlap_inp[ir] = (NE*an[0,ir]*Inp[0]+NI*an[1,ir]*Inp[-1])
                ovs_inplr[ktrial,it,ir] = overlap_inp[ir] ### recording_theory
                overlap_inp[ir]/= (1.0-eigvchn[ir])
                ovs_inplr_div[ktrial,it,ir] = overlap_inp[ir] ### recording theory
            eq_fr = np.zeros(2)
            eq_fr[0],eq_fr[1] = Inp[0],Inp[-1]
            for ir in range(2): ### rank two 
                ### record 
                contributions_lr[ktrial,it,ir,0] = overlap_inp[ir]*am[0,ir] ### excitatory population
                contributions_lr[ktrial,it,ir,1] = overlap_inp[ir]*am[1,ir] ### inhibitory population
                eq_fr[0] += am[0,ir]*overlap_inp[ir]
                eq_fr[1] += am[1,ir]*overlap_inp[ir]
            lowrank_eq[ktrial,it,:] = eq_fr.copy() 
            
            ### compute the covariance matrix for the random components on the left and right eigenvectors
            if it>=1: ### @YS the previous one is useless. 17Nov
                intg_ovP   = first_perturb_ovP[ktrial,it-1,:,:,:] ## ir, jl -- m, n
                if it<=switch:
                    eigenvalues_use = eigvchn_series[ktrial,it-1,:2].copy()
                    eigenvalues_usem = eigvchn_series[ktrial,it,:2].copy()
                else:
                    eigenvalues_use = eigvchn_series[ktrial,it,:2].copy()
                    eigenvalues_usem = eigvchn_series[ktrial,it-1,:2].copy()
                for ir in range(2):
                    for jl in range(2):
                        intg_ovP[ir,jl,0] = intg_ovP[ir,jl,0] + lvec_mean[:,jl].T@DeltaZ2_E@rvec_mean[:,ir]/(np.real(eigenvalues_usem[ir])*np.real(eigeng[jl]))/norm_for_lvec[jl]/norm_for_rvec[ir]*eigenvalues_use[jl].real
                        intg_ovP[ir,jl,1] = intg_ovP[ir,jl,1] + lvec_mean[:,jl].T@DeltaZ2_I@rvec_mean[:,ir]/(np.real(eigenvalues_usem[ir])*np.real(eigeng[jl]))/norm_for_lvec[jl]/norm_for_rvec[ir]*eigenvalues_use[jl].real ### current eigenvalues are multiplied
                        intg_crossovPop_series[ktrial,it,ir,jl,0] = np.squeeze(intg_ovP[ir,jl,0]).real
                        intg_crossovPop_series[ktrial,it,ir,jl,1] = np.squeeze(intg_ovP[ir,jl,1]).real      
            
            ## intg_ov_series[:,2,:] starts
            meanr, meanl = np.zeros((2,2)),np.zeros((2,2))## rank pop
            meanr[:,0], meanr[:,1] = np.mean(reig[:NE,:2],axis=0),np.mean(reig[NE:,:2],axis=0)
            meanl[:,0], meanl[:,1] = np.mean(leig[:NE,:2],axis=0),np.mean(leig[NE:,:2],axis=0)
            ##### compute current overlap (based on current mean connectivity)
            noiser, noisel = np.zeros((N,2)),np.zeros((N,2))
            for i in range(2):
                noiser[:NE,i], noiser[NE:,i] = reig[:NE,i] - meanr[i,0], reig[NE:,i]-meanr[i,1]
                noisel[:NE,i], noisel[NE:,i] = leig[:NE,i] - meanl[i,0], leig[NE:,i]-meanl[i,1]
            for ir in range(2):
                for jl in range(2):
                    first_perturb_ov[ktrial,it,ir,jl] = np.squeeze(np.reshape(noiser[:,ir],(1,-1))@np.reshape(noisel[:,jl],(-1,1)))*np.real(eigvchn_series[ktrial,it,jl])
                    ### separate E and I population
                    first_perturb_ovP[ktrial,it,ir,jl,0]= np.squeeze(np.reshape(noiser[:NE,ir],(1,-1))@np.reshape(noisel[:NE,jl],(-1,1)))*np.real(eigvchn_series[ktrial,it,jl])
                    first_perturb_ovP[ktrial,it,ir,jl,1]= np.squeeze(np.reshape(noiser[NE:,ir],(1,-1))@np.reshape(noisel[NE:,jl],(-1,1)))*np.real(eigvchn_series[ktrial,it,jl]) ### current eigenvalues are multiplied
            z_pre = zr.copy()
            flag_run = True


#### use

In [ ]:
ce, ci = c,c
eigvchn_theo = np.zeros((len(tau_series),2))
radius_theo_map = np.zeros((len(tau_series),2))
for it, tau in enumerate(tau_series):
    ### mean connectivity 
    je, ji = J, g*J
    hat_sigmae, hat_sigmai = np.sqrt(ce*(1-ce)),np.sqrt(ci*(1-ci))
    sigmae, sigmai = np.sqrt(je**2*ce*(1-ce)*N), np.sqrt(ji**2*ci*(1-ci)*N) 
    ### generate tau_div_vec, tau_con_vec, tau_rec_vec, tau_chn_vec
    tau_div_vec, tau_con_vec, tau_rec_vec, tau_chn_vec = np.zeros(2),np.zeros((2,2)),np.zeros((2,2)),np.zeros((2,2))
    ### YS: alltaus do not have E-I polarity
    tau_div_vec = np.array([tau,tau])
    tau_con_vec = np.array([[tau,tau],[tau,tau]])
    # tau_rec_vec = np.array([[0,0],[0,0]])
    tau_rec_vec = np.array([[2*tau,2*tau],[2*tau,2*tau]])
    tau_chn_vec = np.array([[tau,tau],[tau,tau]])
    # gamma = 1.0 ## NI/NE
    sigma = np.sqrt(sigmae**2/(1+gamma)+sigmai**2*gamma/(1+gamma))
    ### compute the radius 
    # compute a single number for multiple populations
    tau_div_num = hat_sigmae**2*tau_div_vec[0]+gamma*g**2*hat_sigmai**2*tau_div_vec[1]
    tau_div_den = hat_sigmae**2+gamma*g**2*hat_sigmai**2
    tau_div = tau_div_num/tau_div_den

    tau_con_num = hat_sigmae**2*tau_con_vec[0,0]+gamma*g**2*hat_sigmai**2*tau_con_vec[1,1]
    tau_con_den = hat_sigmae**2+gamma*g**2*hat_sigmai**2
    tau_con  = tau_con_num/tau_con_den

    tau_rec_num = hat_sigmae**2*tau_rec_vec[0,0] - (1+gamma)*g*hat_sigmae*hat_sigmai*tau_rec_vec[0,1]+gamma*g**2*hat_sigmai**2*tau_rec_vec[1,1]
    tau_rec_den = hat_sigmae**2+gamma*g**2*hat_sigmai**2
    tau_rec = tau_rec_num/tau_rec_den/2.0 

    tau_chn_num = hat_sigmae**2*tau_chn_vec[0,0] - 0.5*(1+gamma)*g*hat_sigmae*hat_sigmai*(tau_chn_vec[0,1]+tau_chn_vec[1,0])+gamma*g**2*hat_sigmai**2*tau_chn_vec[1,1]
    tau_chn_den = hat_sigmae**2+gamma*g**2*hat_sigmai**2
    tau_chn = tau_chn_num/tau_chn_den/2.0

    radius_theo = (1.0-tau_con-tau_div+tau_rec-2*tau_chn)/np.sqrt(1-tau_con-tau_div)*sigma
    
    radius_theo_map[it] = radius_theo
            
### theoretical prediction of the eigenvalues

org_k = J*N*(ALPHAE-g*ALPHAI)/2.0

eigvchn_theo[:,0] = org_k*(c+np.sqrt(c**2+4*c*(1-c)*tau_series))
eigvchn_theo[:,1] = org_k*(c-np.sqrt(c**2+4*c*(1-c)*tau_series))

In [ ]:
# params = {'g':g,
#           'gamma':gamma,
#           'NE':NE,
#           'NI':NI,
#           'c':c,
#           'J':J,
#           'tau_series':tau_series,
#           }
# lst = [eigvchn_series, eigrvec_series, eiglvec_series,
#        eigrvec_series_rec, eiglvec_series_rec,
#         eiglvec0_series, norml0_series, params,
#         lowrank_eq, firing_rateeq,lowrank_eq_num,
#         ovs_inplr, ovs_inplr_num,ovs_inplr_div, ovs_inplr_div_num,
#         contributions_lr, contributions_lr_num,
#         intg_ov_series,first_perturb_ov,first_perturb_ovP,
#         intg_mean_series,intg_std_series,intg_std_num_series,
#         intg_crossov_series,intg_crossovPop_series,leig0mean_series,
#         norm_4lvec_series,norm_4rvec_series,norm_4lvec_series_]
# stg = ["eigvchn_series, eigrvec_series, eiglvec_series,"
#        "eigrvec_series_rec, eiglvec_series_rec,"
#         "eiglvec0_series, norml0_series, params,"
#         "lowrank_eq, firing_rateeq,lowrank_eq_num,"
#         "ovs_inplr, ovs_inplr_num,ovs_inplr_div, ovs_inplr_div_num,"
#         "contributions_lr, contributions_lr_num,"
#         "intg_ov_series,first_perturb_ov,first_perturb_ovP,"
#         "intg_mean_series,intg_std_series,intg_std_num_series,"
#         "intg_crossov_series,intg_crossovPop_series,leig0mean_series,"
#         "norm_4lvec_series,norm_4rvec_series,norm_4lvec_series_"]
# data = list_to_dict(lst=lst, string=stg)
# # data_name = "E:/Dropbox/DailyWork/Allen_project/preparation4paper_Data/19Mar_eigenvectors_corrected/Sparse_data/DynsStats_Adjacency_20Mar_16PRX_54N1000_normlr_whilecode.npz"
data_name = "E:/Dropbox/DailyWork/Allen_project/preparation4paper_Data/21Mar_eigenvalues_Gaussian/DynsStats_Adjacency_21Mar_16PRX_54N1000_normlr_whilecode_used_vecrec.npz"
# np.savez(data_name, **data)
### load the stored data, in particular, extracting variable: eigvchn_series 
data = np.load(data_name,allow_pickle=True)

### also loading other variables 
eigvchn_series = data['eigvchn_series']
eigrvec_series = data['eigrvec_series']
eiglvec_series = data['eiglvec_series']
eigrvec_series_rec = data['eigrvec_series_rec']
eiglvec_series_rec = data['eiglvec_series_rec']
eiglvec0_series = data['eiglvec0_series']
norml0_series = data['norml0_series']
params = data['params']
intg_mean_series = data['intg_mean_series']
leig0mean_series = data['leig0mean_series']
# ### load data
# data = np.load(data_name,allow_pickle=True)
# firing_rateeq = data['firing_rateeq']
# lowrank_eq    = data['lowrank_eq']
# lowrank_eq_num = data['lowrank_eq_num'] 
# contributions_lr = data['contributions_lr'] ### trial, tau, rank, pop
# contributions_lr_num = data['contributions_lr_num']
# eigvchn_series = data['eigvchn_series']
# eigrvec_series = data['eigrvec_series']
# eiglvec_series = data['eiglvec_series']
# eiglvec0_series = data['eiglvec0_series']
# norml0_series = data['norml0_series']
# params = data['params']
# intg_ov_series = data['intg_ov_series']
# first_perturb_ov = data['first_perturb_ov']
# first_perturb_ovP = data['first_perturb_ovP']
# intg_mean_series = data['intg_mean_series']
# intg_std_series = data['intg_std_series']
# leig0mean_series = data['leig0mean_series']


In [ ]:

# '''Using "E:/Dropbox/DailyWork/Allen_project/preparation4paper_Data/19Mar_eigenvectors_corrected/Sparse_data/DynsStats_Adjacency_20Mar_16PRX_54N1000_normlr_whilecode.npz" code '''
''' E:/Dropbox/DailyWork/Allen_project/preparation4paper_Data/21Mar_eigenvalues_Gaussian/DynsStats_Adjacency_21Mar_16PRX_54N1000_normlr_whilecode_used.npz '''
'''"E:/Dropbox/DailyWork/Allen_project/preparation4paper_Data/21Mar_eigenvalues_Gaussian/DynsStats_Adjacency_21Mar_16PRX_54N1000_normlr_whilecode_used_vecrec.npz"'''

In [ ]:
### compute the numerical radius
radius_num_map = np.zeros((trials,ntau))
for ktrial in range(trials):
    for it in range(ntau): 
        eigvchn = eigvchn_series[ktrial,it,:].copy()

        eigvchn_real = np.real(eigvchn)
        eigvchn_real = np.sort(eigvchn_real)
        eigvchn_real = eigvchn_real[::-1]

        radius = (eigvchn_real[2])
        radius_num_map[ktrial,it] = radius

In [ ]:
#### print trial by trial the first two eigenvallues
for ktrial in range(trials):
    eigvchn = eigvchn_series[ktrial,:,:].copy()
    eigvchn_real = (eigvchn)
# ### delete the 17th trial
# eigvchn_series = np.delete(eigvchn_series,17,axis=0) #for ConnStats_Adjacency_12OctDense_.npz
### sorting the 3 maximum eigvchn_series[:,-1,0].real and 3 minimum
eigvchn_real = eigvchn_series[:,-1,0].real.copy()
### sorting eigvchn_real 
idx = np.argsort(eigvchn_real)
idx_eff = idx[3:-3]
print('effective trials:',idx_eff,len(idx_eff))

In [ ]:
### plot a shifting histogram for the first eigenvalue
fig,ax=plt.subplots(figsize=(6,3))
### plot the mean of the eigenvalues, how it changes with tau_series
### colormap gradiently changes with the tau_series
cm = plt.cm.YlOrRd
### the mean of the first eigenvalue
mean_eigvchn = np.mean(eigvchn_series[idx_eff,:,0].real,axis=0)
sc=ax.scatter(mean_eigvchn,-0.1*np.ones(ntau),c=tau_series,cmap=cm)
### the mean of the second eigenvalue 
cm = plt.cm.YlGn
mean_eigvchn = np.mean(eigvchn_series[idx_eff,:,1].real,axis=0)
sc=ax.scatter(mean_eigvchn,0.1*np.ones(ntau),c=tau_series,cmap=cm)

ax.set_xlabel(r'$\Re(\lambda)$',fontsize=14)
### set double ylabels
ax.set_ylabel(r'$\Im(\lambda)$',fontsize=14)
ax2 = ax.twinx() 
 
color = 'tab:gray'
ax2.tick_params(axis ='y', labelcolor = color) 
ax2.set_yticks([])
ax2.spines['bottom'].set_color('none')
### set other x-axis invisible
ax2.spines['top'].set_color('none')
ax2.spines['right'].set_color('none')
ax2.spines['left'].set_color('none')
# ax.set_title('Eigenvalue spectrum',fontsize=14,fontweight='bold')
### refine the plot 
# ax.set_xlim([-3,2])
# ax.set_ylim([-0.3,0.3])
ax.set_xlim([-2.5,1.5])
ax.set_ylim([-0.3,0.3])

ax.set_aspect('equal')
ax.spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')
ax.spines['left'].set_position(('data',0))
## set the x and y ticks
ax.set_xticks([-2,0,1])
ax.set_yticks([-0.5,0,0.5])
# ### also, plot the theoretical predictions for the first and second eigenvalues
# ax.scatter(eigvchn_theo[:,0].real,np.zeros(ntau),cmap=plt.cm.YlOrRd, c=tau_series, s=50,alpha=1,marker='^')
# ax.scatter(eigvchn_theo[:,1].real,np.zeros(ntau),cmap=plt.cm.YlGn, c=tau_series, s=50,alpha=1,marker='^')

### plot a circle with the radius of the last radius_num_map
ax.add_patch(plt.Circle((0,0),radius_theo_map[-1,0],color='tab:blue',fill=False,linewidth=1.5,alpha=1))
### plot a circle with the radius of the first radius_num_map
ax.add_patch(plt.Circle((0,0),radius_theo_map[0,0],color='tab:purple',fill=False,linewidth=1.5,alpha=0.5,linestyle='--'))

ktrial_index = 0
### scatter plot the eigenvalue bulk of this trial
### randomly select 500 points
idx_random = np.random.choice(np.arange(2,N),size=500,replace=False)
ax.scatter(eigvchn_series[ktrial_index,0,idx_random].real,eigvchn_series[ktrial_index,0,idx_random].imag,color='tab:purple', s=5,alpha=0.15)
ax.scatter(eigvchn_series[ktrial_index,-1,idx_random].real,eigvchn_series[ktrial_index,-1,idx_random].imag,color='tab:blue', s=5,alpha=0.2)
### also plot the original eigenvalue outlier: JE-JI, using red color and X marker
ax.scatter(JE-JI,0,color='black',marker='X',s=80,alpha=1)


In [ ]:
radius_theo_map

In [ ]:
### plot the numerical and theoretical eigenvalues outliers
fig,ax = plt.subplots(figsize=(4,2))
### label is r_{bulk}^{num}
ax.plot(tau_series,radius_theo_map,'gray',linestyle='--',label=r'$r_{bulk}^{theo}$')
ax.plot(tau_series,-radius_theo_map,'gray',linestyle='--')
# ax.plot(tau_series,np.mean(radius_num_map,axis=0),'k',label=r'$r_{bulk}^{num}$')
ax.fill_between(tau_series,np.mean(radius_num_map,axis=0),-np.mean(radius_num_map,axis=0),color='gray',alpha=0.15,label=r'$r_{bulk}^{num}$')
ax.set_xlabel(r'$\tau$')
ax.set_ylabel('Radius')
ax.legend()
plt.show()

### I mean... eigvchn_series[:,:,0].real and eigvchn_series[:,:,1].real compared with eigvchn_theo[:,0] and eigvchn_theo[:,1]
# fig,ax = plt.subplots(figsize=(6,6))
ax.plot(tau_series,eigvchn_theo[:,0],'tab:red',marker='o',label='theoretical')
ax.plot(tau_series,eigvchn_theo[:,1],'tab:green',marker='o')
# ax.plot(tau_series,np.mean(eigvchn_series[:,:,0].real,axis=0),'k',label='numerical')
# ax.plot(tau_series,np.mean(eigvchn_series[:,:,1].real,axis=0),'k')
ax.fill_between(tau_series,np.mean(eigvchn_series[idx_eff,:,0].real,axis=0)-np.std(eigvchn_series[idx_eff,:,0].real,axis=0),np.mean(eigvchn_series[:,:,0].real,axis=0)+np.std(eigvchn_series[idx_eff,:,0].real,axis=0),color='tab:red',alpha=0.15)
ax.fill_between(tau_series,np.mean(eigvchn_series[idx_eff,:,1].real,axis=0)-np.std(eigvchn_series[idx_eff,:,1].real,axis=0),np.mean(eigvchn_series[idx_eff,:,1].real,axis=0)+np.std(eigvchn_series[idx_eff,:,1].real,axis=0),color='tab:green',alpha=0.15)
ax.set_xlabel(r'$\tau$')
ax.set_ylabel('Outliers')
### set x and y lims as well as ticks
dtau = tau_series[1]-tau_series[0] 
ax.set_xlim([tau_series[0]-dtau/8,tau_series[-1]+dtau/8])
ax.set_xticks([0,tau_series[-1]//2,tau_series[-1]])
ax.set_ylim([-4.0,2.5])
ax.set_yticks([-3,0,2])
ax.legend()
### move the x and y axis to the center
ax.spines['bottom'].set_position(('data',0))
ax.spines['left'].set_position(('data',0))
### set other x-axis invisible
ax.spines['top'].set_color('none')
ax.spines['right'].set_color('none')
plt.show()

In [ ]:
### compute the population mean and variance
firing_rateeq_mean,firing_rateeq_var = np.zeros((trials,ntau,2)),np.zeros((trials,ntau,2))
firing_rateeq_mean[:,:,0],firing_rateeq_mean[:,:,1] = np.mean(firing_rateeq[:,:,:NE],axis=2),np.mean(firing_rateeq[:,:,NE:],axis=2)
firing_rateeq_var[:,:,0],firing_rateeq_var[:,:,1] = np.var(firing_rateeq[:,:,:NE],axis=2),np.var(firing_rateeq[:,:,NE:],axis=2)

lowrank_eq_num_mean,lowrank_eq_num_var = np.zeros((trials,ntau,2)),np.zeros((trials,ntau,2))
lowrank_eq_num_mean[:,:,0],lowrank_eq_num_mean[:,:,1] = np.mean(lowrank_eq_num[:,:,:NE],axis=2),np.mean(lowrank_eq_num[:,:,NE:],axis=2)
lowrank_eq_num_var[:,:,0],lowrank_eq_num_var[:,:,1] = np.var(lowrank_eq_num[:,:,:NE],axis=2),np.var(lowrank_eq_num[:,:,NE:],axis=2)

In [ ]:
frate_real = firing_rateeq_mean[:,-1,0].copy()
### sorting eigvchn_real 
idx = np.argsort(frate_real)
idx_eff = idx[3:-3]
print('effective trials:',idx_eff,len(idx_eff))

In [ ]:
#### filter the firing rate
epsilon = 1e-3
thresh_low = 2.0/10.0
### select trials
ineffective = []
for ktrial in range(trials):
    if eigvchn_series[ktrial,-1,0].real>eigvchn_series[ktrial,-1,1].real:
        print('in...',ktrial)
        ineffective = np.append(ineffective,ktrial)
        continue 
    if np.abs(eigvchn_series[ktrial,-1,0].imag)>epsilon:
        ineffective = np.append(ineffective,ktrial)
        continue
    if np.abs(eigvchn_series[ktrial,-1,1].imag)>epsilon:
        ineffective = np.append(ineffective,ktrial)
        continue
effective_trials = np.setdiff1d(np.arange(trials),ineffective)   

cut = 3
for irr in range(2):
    for it in range(ntau):
        idxsort = np.argsort(lowrank_eq[:,it,0])
        idxnan = np.append(idxsort[:cut],idxsort[-cut:])
        lowrank_eq[idxnan,it,0] = np.nan 
        idxsort = np.argsort(lowrank_eq[:,it,1])
        idxnan = np.append(idxsort[:cut],idxsort[-cut:])
        lowrank_eq[idxnan,it,1] = np.nan 
        ### same for lowrank_eq_num_mean 
        idxsort = np.argsort(lowrank_eq_num_mean[:,it,0])
        idxnan = np.append(idxsort[:cut],idxsort[-cut:])
        lowrank_eq_num_mean[idxnan,it,0] = np.nan
        idxsort = np.argsort(lowrank_eq_num_mean[:,it,1])
        idxnan = np.append(idxsort[:cut],idxsort[-cut:])
        lowrank_eq_num_mean[idxnan,it,1] = np.nan
        
for irr in range(2):
    for ktrial in range(trials):
        for it in range(0,ntau): 
            if ktrial in ineffective:
                lowrank_eq[ktrial,it,0] = np.nan
                lowrank_eq[ktrial,it,1] = np.nan
                lowrank_eq_num_mean[ktrial,it,0] = np.nan
                lowrank_eq_num_mean[ktrial,it,1] = np.nan
                continue
            if np.abs(eigvchn_series[ktrial,it,irr].imag)>epsilon:
                lowrank_eq_num_mean[ktrial,it,0] = np.nan
                lowrank_eq_num_mean[ktrial,it,1] = np.nan
                lowrank_eq[ktrial,it,0] = np.nan
                lowrank_eq[ktrial,it,1] = np.nan
                continue
            
            ### keep the better match solutions
            if np.abs(lowrank_eq_num_mean[ktrial,it,0])>thresh_low:
                lowrank_eq_num_mean[ktrial,it,0] = np.nan
            if np.abs(lowrank_eq_num_mean[ktrial,it,1])>thresh_low:
                lowrank_eq_num_mean[ktrial,it,1] = np.nan
            if np.abs(lowrank_eq[ktrial,it,0])>thresh_low:
                lowrank_eq[ktrial,it,0] = np.nan
            if np.abs(lowrank_eq[ktrial,it,1])>thresh_low:
                lowrank_eq[ktrial,it,1] = np.nan
                
for it in range(ntau):
    idxsort = np.argsort(firing_rateeq_mean[:,it,0])
    idxnan = np.append(idxsort[:cut],idxsort[-cut:])
    firing_rateeq_mean[idxnan,it,0] = np.nan
    
    idxsort = np.argsort(firing_rateeq_mean[:,it,1])
    idxnan = np.append(idxsort[:cut],idxsort[-cut:])
    firing_rateeq_mean[idxnan,it,1] = np.nan

for ktrial in range(trials):
    for it in range(0,ntau):                
        if np.abs(firing_rateeq_mean[ktrial,it,0])>thresh_low:
            firing_rateeq_mean[ktrial,it,0] = np.nan
        if np.abs(firing_rateeq_mean[ktrial,it,1])>thresh_low:
            firing_rateeq_mean[ktrial,it,1] = np.nan
            
for irr in range(2):
    for it in range(ntau):
        idxsort = np.argsort(contributions_lr[:,it,irr,0])
        idxnan = np.append(idxsort[:cut],idxsort[-cut:])
        contributions_lr[idxnan,it,irr,0] = np.nan
        
        idxsort = np.argsort(contributions_lr[:,it,irr,1])
        idxnan = np.append(idxsort[:cut],idxsort[-cut:])
        contributions_lr[idxnan,it,irr,1] = np.nan
        
        idxsort = np.argsort(contributions_lr_num[:,it,irr,1])
        idxnan = np.append(idxsort[:cut],idxsort[-cut:])
        contributions_lr_num[idxnan,it,irr,1] = np.nan
        
        idxsort = np.argsort(contributions_lr_num[:,it,irr,0])
        idxnan = np.append(idxsort[:cut],idxsort[-cut:])
        contributions_lr_num[idxnan,it,irr,0] = np.nan
                
#### same for the contribution from different terms
for irr in range(2):
    for ktrial in range(trials):
        for it in range(0,ntau):
            if ktrial in ineffective:
                contributions_lr[ktrial,it,irr,0] = np.nan
                contributions_lr[ktrial,it,irr,1] = np.nan
                contributions_lr_num[ktrial,it,irr,0] = np.nan
                contributions_lr_num[ktrial,it,irr,1] = np.nan
                continue
            ### keep the better match solutions
            if np.abs(contributions_lr[ktrial,it,irr,0])>thresh_low:
                contributions_lr[ktrial,it,irr,0] = np.nan
            if np.abs(contributions_lr[ktrial,it,irr,1])>thresh_low:
                contributions_lr[ktrial,it,irr,1] = np.nan
            if np.abs(contributions_lr_num[ktrial,it,irr,0])>thresh_low:
                contributions_lr_num[ktrial,it,irr,0] = np.nan
            if np.abs(contributions_lr_num[ktrial,it,irr,1])>thresh_low:
                contributions_lr_num[ktrial,it,irr,1] = np.nan
                


In [ ]:
def list_to_dict(lst, string):
    """
    Transform a list of variables into a dictionary.
    Parameters
    ----------
    lst : list
        list with all variables.
    string : str
        string containing the names, separated by commas.
    Returns
    -------
    d : dict
        dictionary with items in which the keys and the values are specified
        in string and lst values respectively.
    """
    string = string[0]
    string = string.replace(']', '')
    string = string.replace('[', '')
    string = string.replace('\\', '')
    string = string.replace(' ', '')
    string = string.replace('\t', '')
    string = string.replace('\n', '')
    string = string.split(',')
    d = {s: v for s, v in zip(string, lst)}
    return d

In [ ]:
### compute the trial averaged mean 
# eigvchn_series = data['eigvchn_series']
# ntau = np.shape(eigvchn_series)[1]
# trials = np.shape(eigvchn_series)[0]
# NE, NI = params['NE'],params['NI']

mean_reigvec_series = np.zeros((trials,ntau,2,2)) ##rank, pop
mean_leigvec_series = np.zeros((trials,ntau,2,2)) ##rank, pop
mean_leig0vec_series = np.zeros((trials,ntau,2,2))
### numerical
mean_reigvec_num_series = np.zeros((trials,ntau,2,2))
mean_leigvec_num_series = np.zeros((trials,ntau,2,2))
mean_leig0vec_num_series = np.zeros((trials,ntau,2,2))
thl = 1.5
ths = 1.5
for ktrial in range(trials):
    for it in range(ntau):
        if it<2:
            threshold = ths
        else:
            threshold = thl
        for ir in range(2):
            mean_reigvec_series[ktrial,it,ir,0] = np.mean(intg_mean_series[ktrial,it,:NE,ir,1],axis=0)
            
            mean_reigvec_series[ktrial,it,ir,1] = np.mean(intg_mean_series[ktrial,it,NE:,ir,1],axis=0)
            
            mean_leigvec_series[ktrial,it,ir,0] = np.mean(intg_mean_series[ktrial,it,:NE,ir,0],axis=0)
            mean_leigvec_series[ktrial,it,ir,1] = np.mean(intg_mean_series[ktrial,it,NE:,ir,0],axis=0)
            
            mean_leig0vec_series[ktrial,it,ir,0] = np.mean(leig0mean_series[ktrial,it,:NE,ir])
            mean_leig0vec_series[ktrial,it,ir,1] = np.mean(leig0mean_series[ktrial,it,NE:,ir])
                
### for the numerical 
for ktrial in range(trials):
    for it in range(ntau):
        if it<2:
            threshold = ths
        else:
            threshold = thl
        for ir in range(2):
            mean_reigvec_num_series[ktrial,it,ir,0] = np.mean(eigrvec_series[ktrial,it,:NE,ir])
            mean_reigvec_num_series[ktrial,it,ir,1] = np.mean(eigrvec_series[ktrial,it,NE:,ir])
            mean_leigvec_num_series[ktrial,it,ir,0] = np.mean(eiglvec_series[ktrial,it,:NE,ir])
            mean_leigvec_num_series[ktrial,it,ir,1] = np.mean(eiglvec_series[ktrial,it,NE:,ir])
            
            mean_leig0vec_num_series[ktrial,it,ir,0] = np.mean(eiglvec0_series[ktrial,it,:NE,ir])
            mean_leig0vec_num_series[ktrial,it,ir,1] = np.mean(eiglvec0_series[ktrial,it,NE:,ir])
            
### select the middle 30 values 
kktrial = np.arange(trials)
cuts = 6#53,54#3#55#
for it in range(ntau):
    for ir in range(2):
        ### only keep the middle 30 values of mean_reigvec_series[:,it,ir,0/1], osrt mean_reigvec_series[:,it,ir,0]
        idxsort = np.argsort(mean_reigvec_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:]) 
        mean_reigvec_series[idxnan,it,ir,0] = np.nan 
        idxnan = np.where(np.abs(mean_reigvec_series[:,it,ir,0].real)>threshold)[0]
        mean_reigvec_series[idxnan,it,ir,0] = np.nan
        
        idxsort = np.argsort(mean_reigvec_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_reigvec_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_reigvec_series[:,it,ir,1].real)>threshold)[0]
        mean_reigvec_series[idxnan,it,ir,1] = np.nan
        
        
        idxsort = np.argsort(mean_leigvec_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leigvec_series[idxnan,it,ir,0] = np.nan
        idxnan = np.where(np.abs(mean_leigvec_series[:,it,ir,0].real)>threshold)[0]
        mean_leigvec_series[idxnan,it,ir,0] = np.nan
        if it>=6 and ir==1:
            idxpos=np.where(mean_leigvec_series[:,it,ir,0].real>0)[0]
            mean_leigvec_series[idxpos,it,ir,0] = np.nan
            
        
        idxsort = np.argsort(mean_leigvec_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leigvec_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_leigvec_series[:,it,ir,1].real)>threshold)[0]
        mean_leigvec_series[idxnan,it,ir,1] = np.nan
        if it>=6 and ir==1:
            idxneg=np.where(mean_leigvec_series[:,it,ir,1].real<0)[0]
            mean_leigvec_series[idxneg,it,ir,1] = np.nan
        
        idxsort = np.argsort(mean_reigvec_num_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:]) 
        mean_reigvec_num_series[idxnan,it,ir,0] = np.nan 
        idxnan = np.where(np.abs(mean_reigvec_num_series[:,it,ir,0].real)>threshold)[0]
        mean_reigvec_num_series[idxnan,it,ir,0] = np.nan
        
        idxsort = np.argsort(mean_reigvec_num_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_reigvec_num_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_reigvec_num_series[:,it,ir,1].real)>threshold)[0]
        mean_reigvec_num_series[idxnan,it,ir,1] = np.nan
        
        idxsort = np.argsort(mean_leigvec_num_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leigvec_num_series[idxnan,it,ir,0] = np.nan
        idxnan = np.where(np.abs(mean_leigvec_num_series[:,it,ir,0].real)>threshold)[0]
        mean_leigvec_num_series[idxnan,it,ir,0] = np.nan
        
        idxsort = np.argsort(mean_leigvec_num_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leigvec_num_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_leigvec_num_series[:,it,ir,1].real)>threshold)[0]
        mean_leigvec_num_series[idxnan,it,ir,1] = np.nan
        
        idxsort = np.argsort(mean_leig0vec_series[:,it,ir,0].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leig0vec_series[idxnan,it,ir,0] = np.nan
        idxnan = np.where(np.abs(mean_leig0vec_series[:,it,ir,0].real)>threshold)[0]
        mean_leig0vec_series[idxnan,it,ir,0] = np.nan
        
        idxsort = np.argsort(mean_leig0vec_series[:,it,ir,1].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:])
        mean_leig0vec_series[idxnan,it,ir,1] = np.nan
        idxnan = np.where(np.abs(mean_leig0vec_series[:,it,ir,1].real)>threshold)[0]
        mean_leig0vec_series[idxnan,it,ir,1] = np.nan
        
        

In [ ]:
### rescaled sqrt(N)
mean_leigvec_num_series *=np.sqrt(N)
mean_reigvec_num_series *=np.sqrt(N)
mean_leigvec_series *=np.sqrt(N)
mean_reigvec_series *=np.sqrt(N)

In [ ]:
dtau = tau_series[1]-tau_series[0]
### plot the mean of the left and right eigenvectors
tau_max = tau_series[-1]
fig,ax=plt.subplots(1,2,figsize=(8,3),sharex=True,sharey=True)
ax[0].fill_between(tau_series,np.nanmean(mean_reigvec_num_series[:,:,0,0],axis=0)-np.nanstd(mean_reigvec_num_series[:,:,0,0],axis=0),np.nanmean(mean_reigvec_num_series[:,:,0,0],axis=0)+np.nanstd(mean_reigvec_num_series[:,:,0,0],axis=0),facecolor='orange',alpha=0.3)
ax[0].plot(tau_series,np.nanmean(mean_reigvec_series[:,:,0,0],axis=0),marker='o',color='orange',lw=1.5)
### second rank
ax[0].plot(tau_series,np.nanmean(mean_reigvec_series[:,:,1,0],axis=0),marker='o',color='green',lw=1.5)
ax[0].fill_between(tau_series,np.nanmean(mean_reigvec_num_series[:,:,1,0],axis=0)-np.nanstd(mean_reigvec_num_series[:,:,1,0],axis=0),np.nanmean(mean_reigvec_num_series[:,:,1,0],axis=0)+np.nanstd(mean_reigvec_num_series[:,:,1,0],axis=0),facecolor='green',alpha=0.3)

### excitatory population
ax[0].set_title('Excitatory population',fontsize=12)
## move the x-axis to  the cecnter
ax[0].spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax[0].spines['top'].set_color('none')
ax[0].spines['right'].set_color('none')
ax[0].spines['left'].set_position(('data',0))
ax[0].set_xticks([0,0.2,0.4])
ax[0].set_xlim(0-dtau/8,tau_series[-1]+dtau/8)
# ax[0].set_ylim(-0.01,0.08)
# ax[0].set_yticks([0,0.05])
ax[0].set_ylim(-0.,1.5)
ax[0].set_yticks([0,1.5])
ax[0].set_xlabel('Chain-motif statistics \n'+r'$\tau$',fontsize=12)
ax[0].set_ylabel('Mean of right \n eigenvector(E)',fontsize=12)

### start the inhibitory population
ax[1].fill_between(tau_series,np.nanmean(mean_reigvec_num_series[:,:,0,1],axis=0)-np.nanstd(mean_reigvec_num_series[:,:,0,1],axis=0),np.nanmean(mean_reigvec_num_series[:,:,0,1],axis=0)+np.nanstd(mean_reigvec_num_series[:,:,0,1],axis=0),facecolor='orange',alpha=0.3)
ax[1].plot(tau_series,np.nanmean(mean_reigvec_series[:,:,0,1],axis=0),marker='o',color='orange',lw=1.5)
### second rank
ax[1].plot(tau_series,np.nanmean(mean_reigvec_series[:,:,1,1],axis=0),marker='o',color='green',lw=1.5)
ax[1].fill_between(tau_series,np.nanmean(mean_reigvec_num_series[:,:,1,1],axis=0)-np.nanstd(mean_reigvec_num_series[:,:,1,1],axis=0),np.nanmean(mean_reigvec_num_series[:,:,1,1],axis=0)+np.nanstd(mean_reigvec_num_series[:,:,1,1],axis=0),facecolor='green',alpha=0.3)
### move the x-axis to  the cecnter
ax[1].spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax[1].spines['top'].set_color('none')
ax[1].spines['right'].set_color('none')
ax[1].spines['left'].set_position(('data',0))
ax[1].set_title('Inhibitory population',fontsize=12)
ax[1].set_xticks([0,0.45])
ax[1].set_xlim(0-dtau/8,tau_series[-1]+dtau/8)
ax[1].set_ylim(-0.0,1.5)
ax[1].set_xlabel('Chain-motif statistics \n'+r'$\tau$',fontsize=12)
ax[1].set_ylabel('Mean of right \n eigenvector(I)',fontsize=12)

fig.tight_layout()


In [ ]:
### same plot but for the left eigenvector
dtau = tau_series[1]-tau_series[0]
fig,ax=plt.subplots(1,2,figsize=(8,3),sharex=True)
idx_eff = np.arange(trials)
ax[0].fill_between(tau_series,np.nanmean(mean_leigvec_num_series[:,:,0,0]*eigvchn_series[:,:,0],axis=0)-np.nanstd(mean_leigvec_num_series[:,:,0,0]*eigvchn_series[:,:,0],axis=0),np.nanmean(mean_leigvec_num_series[:,:,0,0]*eigvchn_series[:,:,0],axis=0)+np.nanstd(mean_leigvec_num_series[:,:,0,0]*eigvchn_series[:,:,0],axis=0),facecolor='orange',alpha=0.3)
ax[0].plot(tau_series,np.nanmean(mean_leigvec_series[:,:,0,0],axis=0),marker='o',color='orange',lw=1.5)
### second rank
ax[0].plot(tau_series,np.nanmean(mean_leigvec_series[:,:,1,0],axis=0),marker='o',color='green',lw=1.5)
ax[0].fill_between(tau_series,np.nanmean(mean_leigvec_num_series[:,:,1,0]*eigvchn_series[:,:,1],axis=0)-np.nanstd(mean_leigvec_num_series[:,:,1,0]*eigvchn_series[:,:,1],axis=0),np.nanmean(mean_leigvec_num_series[:,:,1,0]*eigvchn_series[:,:,1],axis=0)+np.nanstd(mean_leigvec_num_series[:,:,1,0]*eigvchn_series[:,:,1],axis=0),facecolor='green',alpha=0.3)
### excitatory population
### excitatory population
ax[0].set_title('Excitatory population',fontsize=12)
## move the x-axis to  the cecnter
ax[0].spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax[0].spines['top'].set_color('none')
ax[0].spines['right'].set_color('none')
ax[0].spines['left'].set_position(('data',0))
ax[0].set_xticks([0,0.2,0.4])
ax[0].set_xlim(0-dtau/8,tau_series[-1]+dtau/8)
ax[0].set_ylim(-0.2,0.35)
ax[0].set_yticks([0,0.3])

ax[0].set_ylim(-5,8)
ax[0].set_yticks([-5,8])
ax[0].set_xlabel('Chain-motif statistics \n'+r'$\tau$',fontsize=12)
ax[0].set_ylabel('Mean of left \n eigenvector(E)',fontsize=12)

### start the inhibitory population
ax[1].plot(tau_series,np.nanmean(mean_leigvec_series[:,:,0,1],axis=0),marker='o',color='orange',lw=1.5)
ax[1].fill_between(tau_series,np.nanmean(mean_leigvec_num_series[:,:,0,1]*eigvchn_series[:,:,0],axis=0)-np.nanstd(mean_leigvec_num_series[:,:,0,1]*eigvchn_series[:,:,0],axis=0),np.nanmean(mean_leigvec_num_series[:,:,0,1]*eigvchn_series[:,:,0],axis=0)+np.nanstd(mean_leigvec_num_series[:,:,0,1]*eigvchn_series[:,:,0],axis=0),facecolor='orange',alpha=0.3)
ax[1].plot(tau_series,np.nanmean(mean_leigvec_series[:,:,1,1],axis=0),marker='o',color='green',lw=1.5)
ax[1].fill_between(tau_series,np.nanmean(mean_leigvec_num_series[:,:,1,1]*eigvchn_series[:,:,1],axis=0)-np.nanstd(mean_leigvec_num_series[:,:,1,1]*eigvchn_series[:,:,1],axis=0),np.nanmean(mean_leigvec_num_series[:,:,1,1]*eigvchn_series[:,:,1],axis=0)+np.nanstd(mean_leigvec_num_series[:,:,1,1]*eigvchn_series[:,:,1],axis=0),facecolor='green',alpha=0.3)
### move the x-axis to  the cecnter
ax[1].spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax[1].spines['top'].set_color('none')
ax[1].spines['right'].set_color('none')
ax[1].spines['left'].set_position(('data',0))
ax[1].set_title('Inhibitory population',fontsize=12)
ax[1].set_xticks([0,0.2,0.4])
ax[1].set_xlim(0-dtau/8,tau_series[-1]+dtau/8)
# ax[1].set_ylim(-1.5,1)
# ax[1].set_yticks([-1,0])
ax[1].set_ylim(-40,15)
ax[1].set_yticks([-40,15])
ax[1].set_xlabel('Chain-motif statistics \n'+r'$\tau$',fontsize=12)
ax[1].set_ylabel('Mean of left \n eigenvector(I)',fontsize=12)

In [ ]:
### same plot but for the left eigenvector
dtau = tau_series[1]-tau_series[0]
fig,ax=plt.subplots(1,2,figsize=(8,3),sharex=True)
idx_eff = np.arange(trials)
ax[0].fill_between(tau_series,np.nanmean(mean_leigvec_num_series[:,:,0,0],axis=0)/np.sqrt(N)-np.nanstd(mean_leigvec_num_series[:,:,0,0],axis=0)/np.sqrt(N),np.nanmean(mean_leigvec_num_series[:,:,0,0],axis=0)/np.sqrt(N)+np.nanstd(mean_leigvec_num_series[:,:,0,0],axis=0)/np.sqrt(N),facecolor='orange',alpha=0.3)
ax[0].plot(tau_series,np.nanmean(mean_leig0vec_series[:,:,0,0],axis=0),marker='o',color='orange',lw=1.5)
### second rank
ax[0].plot(tau_series,np.nanmean(mean_leig0vec_series[:,:,1,0],axis=0),marker='o',color='green',lw=1.5)
ax[0].fill_between(tau_series,np.nanmean(mean_leigvec_num_series[:,:,1,0],axis=0)/np.sqrt(N)-np.nanstd(mean_leigvec_num_series[:,:,1,0],axis=0)/np.sqrt(N),np.nanmean(mean_leigvec_num_series[:,:,1,0],axis=0)/np.sqrt(N)+np.nanstd(mean_leigvec_num_series[:,:,1,0],axis=0)/np.sqrt(N),facecolor='green',alpha=0.3)
## excitatory population
ax[0].set_title('Excitatory population',fontsize=12)
## move the x-axis to  the cecnter
ax[0].spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax[0].spines['top'].set_color('none')
ax[0].spines['right'].set_color('none')
ax[0].spines['left'].set_position(('data',0))
ax[0].set_xticks([0,0.1])
ax[0].set_xlim(0-dtau/8,tau_series[-1]+dtau/8)
# ax[0].set_ylim(-0.01,0.18)
# ax[0].set_yticks([0,0.15])# noparadoxical
ax[0].set_ylim(-0.01,0.06)
ax[0].set_yticks([0,0.05])
# ax[0].set_ylim(-0.1,0.3)
# ax[0].set_yticks([-0.1,0.3])
ax[0].set_ylim(-0.3,0.3)
ax[0].set_yticks([-0.3,0.3])
# ax[0].set_ylim(-0.1,0.3)
# ax[0].set_yticks([-0.1,0.3])
ax[0].set_xlabel('Chain-motif statistics \n'+r'$\tau$',fontsize=12)
ax[0].set_ylabel('Mean of left \n eigenvector(E)',fontsize=12)

### start the inhibitory population

ax[1].plot(tau_series,np.nanmean(mean_leig0vec_series[:,:,0,1],axis=0),marker='o',color='orange',lw=1.5)
ax[1].fill_between(tau_series,np.nanmean(mean_leigvec_num_series[:,:,0,1],axis=0)/np.sqrt(N)-np.nanstd(mean_leigvec_num_series[:,:,0,1],axis=0)/np.sqrt(N),np.nanmean(mean_leigvec_num_series[:,:,0,1],axis=0)/np.sqrt(N)+np.nanstd(mean_leigvec_num_series[:,:,0,1],axis=0)/np.sqrt(N),facecolor='orange',alpha=0.3)
ax[1].plot(tau_series,np.nanmean(mean_leig0vec_series[:,:,1,1],axis=0),marker='o',color='green',lw=1.5)
ax[1].fill_between(tau_series,np.nanmean(mean_leigvec_num_series[:,:,1,1],axis=0)/np.sqrt(N)-np.nanstd(mean_leigvec_num_series[:,:,1,1],axis=0)/np.sqrt(N),np.nanmean(mean_leigvec_num_series[:,:,1,1],axis=0)/np.sqrt(N)+np.nanstd(mean_leigvec_num_series[:,:,1,1],axis=0)/np.sqrt(N),facecolor='green',alpha=0.3)
## move the x-axis to  the cecnter
ax[1].spines['bottom'].set_position(('data',0))
### set other x-axis invisible
ax[1].spines['top'].set_color('none')
ax[1].spines['right'].set_color('none')
ax[1].spines['left'].set_position(('data',0))
ax[1].set_title('Inhibitory population',fontsize=12)
ax[1].set_xticks([0,0.1])
ax[1].set_xlim(0-dtau/8,tau_series[-1]+dtau/8)
ax[1].set_ylim(-0.4,0.02)
ax[1].set_yticks([-0.4,0.])
# ax[1].set_ylim(-1,0.5)
# ax[1].set_yticks([-1,0.5])
ax[1].set_ylim(-1.0,1)
ax[1].set_yticks([-1,1])
ax[1].set_xlabel('Chain-motif statistics \n'+r'$\tau$',fontsize=12)
ax[1].set_ylabel('Mean of left \n eigenvector(I)',fontsize=12)

In [ ]:
### compute low-rank theoretical
contribution_theo = np.zeros((ntau,2,2))
for it in range(ntau):
    if it<2: # both are fine
        for ir in range(1):
            contribution_theo[it,ir,0] = np.squeeze((np.nanmean(mean_leigvec_series[:,it,ir,0],axis=0)/np.sqrt(N)*Inp[0]*NE+np.nanmean(mean_leigvec_series[:,it,ir,1],axis=0)/np.sqrt(N)*Inp[0]*NI)/(1-eigvchn_theo[it,ir].real)*np.nanmean(mean_reigvec_series[:,it,ir,0],axis=0))/np.sqrt(N)
            contribution_theo[it,ir,1] = np.squeeze((np.nanmean(mean_leigvec_series[:,it,ir,0],axis=0)/np.sqrt(N)*Inp[0]*NE+np.nanmean(mean_leigvec_series[:,it,ir,1],axis=0)*Inp[0]*NI)/np.sqrt(N)/(1-eigvchn_theo[it,ir].real)*np.nanmean(mean_reigvec_series[:,it,ir,1],axis=0))/np.sqrt(N)
    else:
        for ir in range(2):
            contribution_theo[it,ir,0] = np.squeeze((np.nanmean(mean_leigvec_series[:,it,ir,0],axis=0)/np.sqrt(N)*Inp[0]*NE+np.nanmean(mean_leigvec_series[:,it,ir,1],axis=0)/np.sqrt(N)*Inp[0]*NI)/(1-eigvchn_theo[it,ir].real)*np.nanmean(mean_reigvec_series[:,it,ir,0],axis=0))/np.sqrt(N)
            contribution_theo[it,ir,1] = np.squeeze((np.nanmean(mean_leigvec_series[:,it,ir,0],axis=0)/np.sqrt(N)*Inp[0]*NE+np.nanmean(mean_leigvec_series[:,it,ir,1],axis=0)/np.sqrt(N)*Inp[0]*NI)/(1-eigvchn_theo[it,ir].real)*np.nanmean(mean_reigvec_series[:,it,ir,1],axis=0))/np.sqrt(N)
firing_rate_lwtheo = np.ones((ntau,2))*Inp[0]
firing_rate_lwtheo +=np.sum(contribution_theo.copy(),axis=1)



In [ ]:
fig,ax=plt.subplots(figsize=(4,3))
htau = tau_series[1]-tau_series[0]
# ax.plot(tau_series, firing_rate0_mean[0]*np.ones((ntau,)),linestyle='--',color='gray',linewidth=1.5,alpha=0.5)
ax.fill_between(tau_series, trialavg_sparseiid_dyns[0]*np.ones((ntau,))-trialstd_sparseiid_dyns[0]*np.ones((ntau,)),trialavg_sparseiid_dyns[0]*np.ones((ntau,))+trialstd_sparseiid_dyns[0]*np.ones((ntau,)),facecolor='gray',alpha=0.3)
# ### start with the excitatory neuron population
# ax.plot(tau_series, np.nanmean(firing_rateeq_mean[:,:,0],axis=0),marker='o',linestyle='--',color='gray',alpha=1,lw=1.5)
### plot the errorbar of firing_rateeq_mean[:,:,0]
alphass=0.95
ax.errorbar(tau_series[:],np.nanmean(firing_rateeq_mean[:,:,0].real,axis=0),yerr=np.nanstd(firing_rateeq_mean[:,:,0].real,axis=0),fmt='x',color='tab:gray',ecolor='tab:gray',alpha=alphass,ls='',elinewidth=1.5)
ax.plot(tau_series, firing_rate_lwtheo[:,0],marker='o',color='black',alpha=1,lw=1.5)
### design the axes
ax.set_xlim(tau_series[0]-htau/8.0,tau_series[-1]+htau/8.0)
ax.set_xticks([tau_series[0],tau_series[-1]])
ax.set_ylim(-0.02,0.1)
ax.set_yticks([0,0.1])
### move the axes to the center
ax.spines['left'].set_position(('data', 0))
ax.spines['bottom'].set_position(('data', 0))
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.set_xlabel(r'$\tau$',fontsize=14)
ax.set_ylabel(r'$\nu_{E}$',fontsize=14)
ax.tick_params(labelsize=12)
# ax.set_title(r'$\nu_{E}$',fontsi

In [ ]:
### Plot the contributions from individual ranks
fig,ax=plt.subplots(figsize=(4,3))
htau = tau_series[1]-tau_series[0]
### start with the excitatory neuron population
ax.plot(tau_series, firing_rate_lwtheo[:,0],marker='o',color='black',alpha=1,lw=1.5)
### contribution from the first rank-1 component
ax.plot(tau_series, contribution_theo[:,0,0],marker='o',linestyle='--',color='gray',alpha=1,lw=1.5)
ax.plot(tau_series,contribution_theo[:,1,0],marker='o',linestyle='-',color='gray',alpha=1,lw=1.5)

### design the axes
ax.set_xlim(tau_series[0]-htau/8.0,tau_series[-1]+htau/8.0)
ax.set_xticks([tau_series[0],tau_series[-1]])
ax.set_ylim(-0.02,0.1)
ax.set_yticks([0,0.1])
### move the axes to the center
ax.spines['left'].set_position(('data', 0))
ax.spines['bottom'].set_position(('data', 0))
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.set_xlabel(r'$\tau$',fontsize=14)
ax.set_ylabel(r'$\nu_{I}$',fontsize=14)
ax.tick_params(labelsize=12)

In [ ]:
### select the middle 30 values 
kktrial = np.arange(trials)
cuts = 6
norm_4lvec_series_select = norm_4lvec_series.copy()
norm_4lvec_series_select_ = norm_4lvec_series_.copy()
for it in range(ntau):
    for ir in range(2):
        ### only keep the middle 30 values of mean_reigvec_series[:,it,ir,0/1], osrt mean_reigvec_series[:,it,ir,0]
        idxsort = np.argsort(norm_4lvec_series[:,it,ir].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:]) 
        norm_4lvec_series_select[idxnan,it,ir] = np.nan 
        
        idxsort = np.argsort(norm_4lvec_series_[:,it,ir].real)
        idxnan = np.append(idxsort[:cuts],idxsort[-cuts:]) 
        norm_4lvec_series_select_[idxnan,it,ir] = np.nan 

In [ ]:
#### plot the norm_4lvec and norm_4rvec change with tau^r   
fig, ax = plt.subplots(figsize=(4,2))
### plot error bar  
alphass = 0.95
ax.errorbar(tau_series[1:],np.nanmean(norm_4lvec_series_select[:,1:,0].real/norml0_series[:,1:-1,0],axis=0),yerr=np.nanstd(norm_4lvec_series_select[:,1:,0].real/norml0_series[:,1:-1,0],axis=0),fmt='x',color='tab:red',ecolor='tab:red',alpha=alphass,ls='',elinewidth=1.5)
ax.errorbar(tau_series[1:],np.mean(norm_4rvec_series[:,1:,0].real,axis=0),yerr=np.std(norm_4rvec_series[:,1:,0].real,axis=0),fmt='x',color='tab:green',ecolor='tab:green',alpha=alphass,ls='',elinewidth=1.5)
ax.set_xlim([tau_series[0]-0.005,tau_series[-1]+0.005])
ax.set_xticks([tau_series[0],tau_series[-1]])
ax.set_ylim([0.9,1.3])
ax.set_yticks([0.9,1.3])

#### plot the norm_4lvec and norm_4rvec change with tau^r   
fig, ax = plt.subplots(figsize=(4,2))
### plot error bar  
alphass = 0.95
ax.errorbar(tau_series[1:],np.nanmean(norm_4lvec_series_select_[:,1:,0].real,axis=0),yerr=np.nanstd(norm_4lvec_series_select_[:,1:,0].real,axis=0),fmt='x',color='tab:red',ecolor='tab:red',alpha=alphass,ls='',elinewidth=1.5)
ax.errorbar(tau_series[1:],np.mean(norm_4rvec_series[:,1:,0].real,axis=0),yerr=np.std(norm_4rvec_series[:,1:,0].real,axis=0),fmt='x',color='tab:green',ecolor='tab:green',alpha=alphass,ls='',elinewidth=1.5)
ax.set_xlim([tau_series[0]-0.005,tau_series[-1]+0.005])
ax.set_xticks([tau_series[0],tau_series[-1]])
ax.set_ylim([0.9,1.3])
ax.set_yticks([0.9,1.3])